In [1]:
import scrapy
import urllib
import json
import requests
from scrapy.crawler import CrawlerProcess

In [ ]:
def get_url_for_page(page):
    params = {
        "page": page,
        "rpp": 100,
        "active": "true",
        "max_price": 4600,
        "beds": 2,
        "baths": "",
        "neighborhoods": "",
        "buildingTypes": "",
        "features": "",
        "zip_codes": "",
        "web_ids": "",
        "addresses": "",
        "isRentals": "true",
        "isSaved": "false",
    }
    return 'https://api.coopercooper.com/CooperPlatform/unit//?beds=1&{}'.format(urllib.parse.urlencode(params))


def get_url_for_property_id(property_id):
    return 'https://api.coopercooper.com/CooperPlatform/unit/{}'.format(property_id)

    
class ListingSpider(scrapy.Spider):
    name = 'listings'
    start_urls = [get_url_for_page(1)]
    
    custom_settings = {
        'FEED_FORMAT':'json',
        'FEED_URI': 'listings.json'
    }
    
    def parse(self, response):
        data = json.loads(response.body)
        
        for listing in data['item']:
            property_id = listing['id']
            yield scrapy.Request(get_url_for_property_id(property_id), callback=self.parse_listing)
            
        if len(data['item']) > 0:
            yield scrapy.Request(get_url_for_page(data['page'] + 1), callback=self.parse)
            
    def parse_listing(self, response):
        data = json.loads(response.body)
        yield data
        
        
process = CrawlerProcess()
process.crawl(ListingSpider)
process.start(stop_after_crawl=False)

In [2]:
with open('listings-new.json', 'r') as f:
    listings = json.load(f)

In [3]:
from shapely.geometry import Polygon, MultiPolygon

def get_commute_polygons():
    cached_api_response_raw = '{"results":[{"search_id":"15_min","shapes":[{"shell":[{"lat":40.7397583,"lng":-73.9900909},{"lat":40.7397583,"lng":-73.99219512},{"lat":40.73885902,"lng":-73.9930944},{"lat":40.74065758,"lng":-73.99489296},{"lat":40.74065758,"lng":-73.99579224},{"lat":40.73885902,"lng":-73.9975908},{"lat":40.73885902,"lng":-73.99938936},{"lat":40.74020794,"lng":-74.00073828},{"lat":40.7420065,"lng":-74.00073828},{"lat":40.74245614,"lng":-74.00118792},{"lat":40.73930866,"lng":-74.0043354},{"lat":40.73840938,"lng":-74.00523468},{"lat":40.73706046,"lng":-74.00388576},{"lat":40.73795974,"lng":-74.00298648},{"lat":40.73795974,"lng":-74.0020872},{"lat":40.7375101,"lng":-74.00163756},{"lat":40.73481226,"lng":-74.00163756},{"lat":40.7330137,"lng":-74.00343612},{"lat":40.73031586,"lng":-74.00343612},{"lat":40.72941658,"lng":-74.00253684},{"lat":40.72761802,"lng":-74.00253684},{"lat":40.72671874,"lng":-74.00163756},{"lat":40.72132306,"lng":-74.00163756},{"lat":40.72087342,"lng":-74.0020872},{"lat":40.7217727,"lng":-74.00298648},{"lat":40.7217727,"lng":-74.00388576},{"lat":40.72132306,"lng":-74.0043354},{"lat":40.71772594,"lng":-74.0043354},{"lat":40.71637702,"lng":-74.00298648},{"lat":40.7172763,"lng":-74.0020872},{"lat":40.7172763,"lng":-74.00028864},{"lat":40.71817558,"lng":-73.99938936},{"lat":40.71817558,"lng":-73.9975908},{"lat":40.71547774,"lng":-73.99489296},{"lat":40.7172763,"lng":-73.9930944},{"lat":40.7172763,"lng":-73.99129584},{"lat":40.71862522,"lng":-73.98994692},{"lat":40.7195245,"lng":-73.98994692},{"lat":40.71997414,"lng":-73.99039656},{"lat":40.71997414,"lng":-73.99129584},{"lat":40.72042378,"lng":-73.99174548},{"lat":40.72132306,"lng":-73.99174548},{"lat":40.7217727,"lng":-73.99129584},{"lat":40.7217727,"lng":-73.98769872},{"lat":40.72476804,"lng":-73.98470338},{"lat":40.72476804,"lng":-73.985297},{"lat":40.72521768,"lng":-73.98574664},{"lat":40.72611696,"lng":-73.98574664},{"lat":40.72701624,"lng":-73.98484736},{"lat":40.72746588,"lng":-73.985297},{"lat":40.72746588,"lng":-73.98619628},{"lat":40.72791552,"lng":-73.98664592},{"lat":40.73061336,"lng":-73.98664592},{"lat":40.731063,"lng":-73.98709556},{"lat":40.731063,"lng":-73.98889412},{"lat":40.73151264,"lng":-73.98934376},{"lat":40.73241192,"lng":-73.98934376},{"lat":40.73286156,"lng":-73.98889412},{"lat":40.73286156,"lng":-73.98619628},{"lat":40.7333112,"lng":-73.98574664},{"lat":40.73735796,"lng":-73.9897934},{"lat":40.73735796,"lng":-73.99069268},{"lat":40.7378076,"lng":-73.99114232},{"lat":40.73870688,"lng":-73.99114232},{"lat":40.7397583,"lng":-73.9900909}],"holes":[]},{"shell":[{"lat":40.74155686,"lng":-73.98679806000001},{"lat":40.74155686,"lng":-73.98769872},{"lat":40.74110722,"lng":-73.98814836},{"lat":40.74020794,"lng":-73.98814836},{"lat":40.7400558,"lng":-73.9883005},{"lat":40.7400558,"lng":-73.98799484},{"lat":40.73915652,"lng":-73.98709556},{"lat":40.73915652,"lng":-73.985297},{"lat":40.73960616,"lng":-73.98484736},{"lat":40.74155686,"lng":-73.98679806000001}],"holes":[]},{"shell":[{"lat":40.747096519999985,"lng":-73.98095412},{"lat":40.74740218,"lng":-73.98095412},{"lat":40.74785182,"lng":-73.98140376},{"lat":40.74635076,"lng":-73.98290482},{"lat":40.74635076,"lng":-73.98169988},{"lat":40.747096519999985,"lng":-73.98095412}],"holes":[]},{"shell":[{"lat":40.74155686,"lng":-73.99129584},{"lat":40.7420065,"lng":-73.9908462},{"lat":40.74470434,"lng":-73.9908462},{"lat":40.74515398,"lng":-73.99129584},{"lat":40.7420065,"lng":-73.99444332},{"lat":40.74065758,"lng":-73.9930944},{"lat":40.74155686,"lng":-73.99219512},{"lat":40.74155686,"lng":-73.99129584}],"holes":[]}],"properties":{}},{"search_id":"20_min","shapes":[{"shell":[{"lat":40.75478074,"lng":-73.97646022},{"lat":40.75478074,"lng":-73.9755916},{"lat":40.755395730000004,"lng":-73.97497661},{"lat":40.75583004,"lng":-73.97541092},{"lat":40.75478074,"lng":-73.97646022}],"holes":[]},{"shell":[{"lat":40.75145258,"lng":-73.97316272},{"lat":40.75298218,"lng":-73.97469232},{"lat":40.75298218,"lng":-73.97675984},{"lat":40.75268256,"lng":-73.97675984},{"lat":40.75133364,"lng":-73.97810876},{"lat":40.75133364,"lng":-73.97990732},{"lat":40.750884,"lng":-73.98035696},{"lat":40.74998472,"lng":-73.98035696},{"lat":40.74953508,"lng":-73.9808066},{"lat":40.74953508,"lng":-73.98170588},{"lat":40.7486358,"lng":-73.98260516},{"lat":40.7486358,"lng":-73.98440372},{"lat":40.74998472,"lng":-73.98575264},{"lat":40.75178328,"lng":-73.98575264},{"lat":40.75223292,"lng":-73.985303},{"lat":40.75223292,"lng":-73.98350444},{"lat":40.75358184,"lng":-73.98215552},{"lat":40.7553804,"lng":-73.98215552},{"lat":40.75672932,"lng":-73.98350444},{"lat":40.75627968,"lng":-73.98395408},{"lat":40.7553804,"lng":-73.98395408},{"lat":40.75493076,"lng":-73.98440372},{"lat":40.75493076,"lng":-73.985303},{"lat":40.75583004,"lng":-73.98620228},{"lat":40.75448112,"lng":-73.9875512},{"lat":40.75403148,"lng":-73.98710156},{"lat":40.75403148,"lng":-73.98620228},{"lat":40.75358184,"lng":-73.98575264},{"lat":40.7531322,"lng":-73.98620228},{"lat":40.7531322,"lng":-73.98710156},{"lat":40.75223292,"lng":-73.98800084},{"lat":40.75223292,"lng":-73.9897994},{"lat":40.75133364,"lng":-73.99069868},{"lat":40.75133364,"lng":-73.99159796},{"lat":40.75178328,"lng":-73.9920476},{"lat":40.75268256,"lng":-73.9920476},{"lat":40.75403148,"lng":-73.99339652},{"lat":40.75178328,"lng":-73.99564472},{"lat":40.75043436,"lng":-73.9942958},{"lat":40.75043436,"lng":-73.99339652},{"lat":40.74998472,"lng":-73.99294688},{"lat":40.74908544,"lng":-73.99294688},{"lat":40.74773652,"lng":-73.9942958},{"lat":40.74773652,"lng":-73.99609436},{"lat":40.7486358,"lng":-73.99699364},{"lat":40.7486358,"lng":-73.9987922},{"lat":40.74773652,"lng":-73.99969148},{"lat":40.74773652,"lng":-74.00149004},{"lat":40.74728688,"lng":-74.00193968},{"lat":40.7463876,"lng":-74.0010404},{"lat":40.74548832,"lng":-74.0010404},{"lat":40.74503868,"lng":-74.00149004},{"lat":40.74503868,"lng":-74.00238932},{"lat":40.74324012,"lng":-74.00418788},{"lat":40.74324012,"lng":-74.00598644},{"lat":40.7441394,"lng":-74.00688572},{"lat":40.7418912,"lng":-74.00913392},{"lat":40.74009264,"lng":-74.00913392},{"lat":40.73919336,"lng":-74.0100332},{"lat":40.73829408,"lng":-74.00913392},{"lat":40.73649552,"lng":-74.00913392},{"lat":40.73604588,"lng":-74.00868428},{"lat":40.73604588,"lng":-74.007785},{"lat":40.73559624,"lng":-74.00733536},{"lat":40.7351466,"lng":-74.007785},{"lat":40.7351466,"lng":-74.00868428},{"lat":40.73469696,"lng":-74.00913392},{"lat":40.73379768,"lng":-74.00913392},{"lat":40.73334804,"lng":-74.00958356},{"lat":40.73334804,"lng":-74.01048284},{"lat":40.7328984,"lng":-74.01093248},{"lat":40.72975092,"lng":-74.007785},{"lat":40.72975092,"lng":-74.00688572},{"lat":40.72930128,"lng":-74.00643608},{"lat":40.72660344,"lng":-74.00643608},{"lat":40.72570416,"lng":-74.00733536},{"lat":40.72480488,"lng":-74.00643608},{"lat":40.72300632,"lng":-74.00643608},{"lat":40.72210704,"lng":-74.00733536},{"lat":40.72120776,"lng":-74.00733536},{"lat":40.72075812,"lng":-74.007785},{"lat":40.72075812,"lng":-74.00868428},{"lat":40.72030848,"lng":-74.00913392},{"lat":40.7194092,"lng":-74.00823464},{"lat":40.71671136,"lng":-74.00823464},{"lat":40.71626172,"lng":-74.00868428},{"lat":40.71626172,"lng":-74.00958356},{"lat":40.71401352,"lng":-74.01183176},{"lat":40.71311424,"lng":-74.01183176},{"lat":40.7126646,"lng":-74.0122814},{"lat":40.7126646,"lng":-74.01318068},{"lat":40.71221496,"lng":-74.01363032},{"lat":40.70951712,"lng":-74.01363032},{"lat":40.70906748,"lng":-74.01407996},{"lat":40.70906748,"lng":-74.01497924},{"lat":40.70771856,"lng":-74.01632816},{"lat":40.70726892,"lng":-74.01587852},{"lat":40.70681928,"lng":-74.01542888},{"lat":40.70502072,"lng":-74.01542888},{"lat":40.70457108,"lng":-74.01497924},{"lat":40.70457108,"lng":-74.01407996},{"lat":40.70412144,"lng":-74.01363032},{"lat":40.70322216,"lng":-74.01363032},{"lat":40.70187324,"lng":-74.0122814},{"lat":40.70232288,"lng":-74.01183176},{"lat":40.70322216,"lng":-74.01273104},{"lat":40.70412144,"lng":-74.01273104},{"lat":40.70592,"lng":-74.01093248},{"lat":40.70771856,"lng":-74.01093248},{"lat":40.7081682,"lng":-74.01048284},{"lat":40.7081682,"lng":-74.00868428},{"lat":40.70906748,"lng":-74.007785},{"lat":40.70906748,"lng":-74.00598644},{"lat":40.70996676,"lng":-74.00508716},{"lat":40.70996676,"lng":-74.00418788},{"lat":40.7104164,"lng":-74.00373824},{"lat":40.71131568,"lng":-74.00373824},{"lat":40.7126646,"lng":-74.00238932},{"lat":40.71176532,"lng":-74.00149004},{"lat":40.71176532,"lng":-74.00059076},{"lat":40.71221496,"lng":-74.00014112},{"lat":40.71311424,"lng":-74.00014112},{"lat":40.71446316,"lng":-73.9987922},{"lat":40.71401352,"lng":-73.99834256},{"lat":40.71311424,"lng":-73.99834256},{"lat":40.7126646,"lng":-73.99789292},{"lat":40.7126646,"lng":-73.99699364},{"lat":40.71356388,"lng":-73.99609436},{"lat":40.7126646,"lng":-73.99519508},{"lat":40.7126646,"lng":-73.99159796},{"lat":40.71356388,"lng":-73.99069868},{"lat":40.71356388,"lng":-73.98800084},{"lat":40.71401352,"lng":-73.9875512},{"lat":40.7149128,"lng":-73.9875512},{"lat":40.71581208,"lng":-73.98845048},{"lat":40.71626172,"lng":-73.98800084},{"lat":40.71626172,"lng":-73.98620228},{"lat":40.71671136,"lng":-73.98575264},{"lat":40.72030848,"lng":-73.98575264},{"lat":40.72075812,"lng":-73.985303},{"lat":40.72075812,"lng":-73.98440372},{"lat":40.72120776,"lng":-73.98395408},{"lat":40.72210704,"lng":-73.98395408},{"lat":40.72255668,"lng":-73.98350444},{"lat":40.72255668,"lng":-73.98260516},{"lat":40.72300632,"lng":-73.98215552},{"lat":40.7239056,"lng":-73.98215552},{"lat":40.72435524,"lng":-73.98170588},{"lat":40.72435524,"lng":-73.98143692},{"lat":40.7273527,"lng":-73.98143692},{"lat":40.72780234,"lng":-73.98188656},{"lat":40.72780234,"lng":-73.98278584},{"lat":40.72825198,"lng":-73.98323548},{"lat":40.73005054,"lng":-73.98323548},{"lat":40.73050018,"lng":-73.98278584},{"lat":40.72870162,"lng":-73.98098728},{"lat":40.72870162,"lng":-73.97918872},{"lat":40.7296009,"lng":-73.97828944},{"lat":40.7296009,"lng":-73.97739016},{"lat":40.73005054,"lng":-73.97694052},{"lat":40.73094982,"lng":-73.97694052},{"lat":40.7318491,"lng":-73.9778398},{"lat":40.73364766,"lng":-73.9778398},{"lat":40.73454694,"lng":-73.97873908},{"lat":40.73499658,"lng":-73.97828944},{"lat":40.73499658,"lng":-73.97739016},{"lat":40.735161930000004,"lng":-73.97722481},{"lat":40.73649552,"lng":-73.9785584},{"lat":40.73769442,"lng":-73.9785584},{"lat":40.73769442,"lng":-73.97918872},{"lat":40.73724478,"lng":-73.97963836},{"lat":40.73544622,"lng":-73.97963836},{"lat":40.7340973,"lng":-73.98098728},{"lat":40.7340973,"lng":-73.98188656},{"lat":40.7363455,"lng":-73.98413476},{"lat":40.73679514,"lng":-73.98368512},{"lat":40.73679514,"lng":-73.98278584},{"lat":40.73814406,"lng":-73.98143692},{"lat":40.74174118,"lng":-73.98143692},{"lat":40.74264046,"lng":-73.98053764},{"lat":40.74353974,"lng":-73.98053764},{"lat":40.74398938,"lng":-73.980088},{"lat":40.74398938,"lng":-73.97918872},{"lat":40.74324012,"lng":-73.97843945999999},{"lat":40.74324012,"lng":-73.97813941999999},{"lat":40.74353974,"lng":-73.9778398},{"lat":40.74443902,"lng":-73.9778398},{"lat":40.7446197,"lng":-73.97765912},{"lat":40.74548832,"lng":-73.97765912},{"lat":40.7463876,"lng":-73.97675984},{"lat":40.74683724,"lng":-73.97720948},{"lat":40.74683724,"lng":-73.97810876},{"lat":40.74728688,"lng":-73.9785584},{"lat":40.74908544,"lng":-73.9785584},{"lat":40.74953508,"lng":-73.97810876},{"lat":40.74953508,"lng":-73.97720948},{"lat":40.75043436,"lng":-73.9763102},{"lat":40.75043436,"lng":-73.97451164},{"lat":40.75028434,"lng":-73.97436162},{"lat":40.75028434,"lng":-73.97316272},{"lat":40.75145258,"lng":-73.97316272}],"holes":[]},{"shell":[{"lat":40.75764393,"lng":-73.97182913},{"lat":40.75702894,"lng":-73.97244412},{"lat":40.7565793,"lng":-73.97199448},{"lat":40.7565793,"lng":-73.9707645},{"lat":40.75764393,"lng":-73.97182913}],"holes":[]},{"shell":[{"lat":40.76392356,"lng":-73.96854738},{"lat":40.76287426,"lng":-73.96749808},{"lat":40.76287426,"lng":-73.9662681},{"lat":40.76392356,"lng":-73.9673174},{"lat":40.76392356,"lng":-73.96854738}],"holes":[]},{"shell":[{"lat":40.71744529,"lng":-73.95804031},{"lat":40.7161117,"lng":-73.95670672},{"lat":40.71746062,"lng":-73.9553578},{"lat":40.71791026,"lng":-73.95580744},{"lat":40.71791026,"lng":-73.95757534},{"lat":40.71761064,"lng":-73.95787496},{"lat":40.71744529,"lng":-73.95804031}],"holes":[]},{"shell":[{"lat":40.75942716,"lng":-73.98170588},{"lat":40.75942716,"lng":-73.98260516},{"lat":40.75897752,"lng":-73.9830548},{"lat":40.7576286,"lng":-73.98170588},{"lat":40.75897752,"lng":-73.98035696},{"lat":40.75942716,"lng":-73.9808066},{"lat":40.75942716,"lng":-73.98170588}],"holes":[]},{"shell":[{"lat":40.75852788,"lng":-73.98890012},{"lat":40.75852788,"lng":-73.99069868},{"lat":40.75807824,"lng":-73.99114832},{"lat":40.75672932,"lng":-73.9897994},{"lat":40.75807824,"lng":-73.98845048},{"lat":40.75852788,"lng":-73.98890012}],"holes":[]}],"properties":{}},{"search_id":"25_min","shapes":[{"shell":[{"lat":40.73920626,"lng":-74.0122932},{"lat":40.73851524,"lng":-74.0122932},{"lat":40.7380656,"lng":-74.01274284},{"lat":40.7380656,"lng":-74.01346026},{"lat":40.73785734,"lng":-74.01366852},{"lat":40.7374077,"lng":-74.01321888},{"lat":40.7374077,"lng":-74.0123196},{"lat":40.73695806,"lng":-74.01186996},{"lat":40.73605878,"lng":-74.01186996},{"lat":40.7351595,"lng":-74.01276924},{"lat":40.73426022,"lng":-74.01276924},{"lat":40.73381058,"lng":-74.01321888},{"lat":40.73381058,"lng":-74.01411816},{"lat":40.73336094,"lng":-74.0145678},{"lat":40.73111274,"lng":-74.0123196},{"lat":40.73111274,"lng":-74.01142032},{"lat":40.7306631,"lng":-74.01097068},{"lat":40.72976382,"lng":-74.01097068},{"lat":40.72886454,"lng":-74.01186996},{"lat":40.7284149,"lng":-74.01142032},{"lat":40.7284149,"lng":-74.01052104},{"lat":40.72796526,"lng":-74.0100714},{"lat":40.72706598,"lng":-74.01097068},{"lat":40.72526742,"lng":-74.01097068},{"lat":40.72436814,"lng":-74.01186996},{"lat":40.7216703,"lng":-74.01186996},{"lat":40.72077102,"lng":-74.01276924},{"lat":40.71987174,"lng":-74.01186996},{"lat":40.71897246,"lng":-74.01186996},{"lat":40.71852282,"lng":-74.0123196},{"lat":40.71852282,"lng":-74.01411816},{"lat":40.71807318,"lng":-74.0145678},{"lat":40.7171739,"lng":-74.0145678},{"lat":40.71672426,"lng":-74.01501744},{"lat":40.71672426,"lng":-74.016816},{"lat":40.71627462,"lng":-74.01726564},{"lat":40.71447606,"lng":-74.01546708},{"lat":40.71357678,"lng":-74.01546708},{"lat":40.71312714,"lng":-74.01591672},{"lat":40.71312714,"lng":-74.016816},{"lat":40.70908038,"lng":-74.02086276},{"lat":40.70728182,"lng":-74.0190642},{"lat":40.7036847,"lng":-74.0190642},{"lat":40.70233578,"lng":-74.01771528},{"lat":40.70233578,"lng":-74.016816},{"lat":40.70188614,"lng":-74.01636636},{"lat":40.70098686,"lng":-74.01636636},{"lat":40.69873866,"lng":-74.01411816},{"lat":40.69873866,"lng":-74.01052104},{"lat":40.6991883,"lng":-74.0100714},{"lat":40.70008758,"lng":-74.0100714},{"lat":40.70188614,"lng":-74.00827284},{"lat":40.70458398,"lng":-74.00827284},{"lat":40.70503362,"lng":-74.0078232},{"lat":40.70503362,"lng":-74.00692392},{"lat":40.70413434,"lng":-74.00602464},{"lat":40.70413434,"lng":-74.00512536},{"lat":40.70458398,"lng":-74.00467572},{"lat":40.70548326,"lng":-74.00467572},{"lat":40.70773146,"lng":-74.00242752},{"lat":40.70683218,"lng":-74.00152824},{"lat":40.70683218,"lng":-73.9988304},{"lat":40.70728182,"lng":-73.99838076},{"lat":40.7081811,"lng":-73.99838076},{"lat":40.70908038,"lng":-73.99928004},{"lat":40.70953002,"lng":-73.9988304},{"lat":40.70953002,"lng":-73.99703184},{"lat":40.7104293,"lng":-73.99613256},{"lat":40.70953002,"lng":-73.99523328},{"lat":40.70953002,"lng":-73.99343472},{"lat":40.70863074,"lng":-73.99253544},{"lat":40.70863074,"lng":-73.99181802000001},{"lat":40.70928864,"lng":-73.99116012},{"lat":40.70863074,"lng":-73.99050222000001},{"lat":40.70863074,"lng":-73.98893832},{"lat":40.71402642,"lng":-73.98354264},{"lat":40.71402642,"lng":-73.98264336},{"lat":40.71447606,"lng":-73.98219372},{"lat":40.71537534,"lng":-73.98219372},{"lat":40.71582498,"lng":-73.98174408},{"lat":40.71582498,"lng":-73.9808448},{"lat":40.71627462,"lng":-73.98039516},{"lat":40.71672426,"lng":-73.9808448},{"lat":40.71672426,"lng":-73.98174408},{"lat":40.7171739,"lng":-73.98219372},{"lat":40.7194221,"lng":-73.97994552},{"lat":40.71852282,"lng":-73.97904624},{"lat":40.71852282,"lng":-73.97814696},{"lat":40.71897246,"lng":-73.97769732},{"lat":40.72077102,"lng":-73.97769732},{"lat":40.72122066,"lng":-73.97814696},{"lat":40.72122066,"lng":-73.97904624},{"lat":40.7216703,"lng":-73.97949588},{"lat":40.7239185,"lng":-73.97724768},{"lat":40.7239185,"lng":-73.97365056},{"lat":40.7261667,"lng":-73.97140236},{"lat":40.72796526,"lng":-73.97140236},{"lat":40.72886454,"lng":-73.97050308},{"lat":40.7306631,"lng":-73.97050308},{"lat":40.73156238,"lng":-73.97140236},{"lat":40.731744240000005,"lng":-73.97140236},{"lat":40.73087136,"lng":-73.97227524},{"lat":40.73222028,"lng":-73.97362416},{"lat":40.73311956,"lng":-73.97272488},{"lat":40.73468346,"lng":-73.97272488},{"lat":40.73695806,"lng":-73.97499948},{"lat":40.73899127999999,"lng":-73.97499948},{"lat":40.73941452,"lng":-73.97542272},{"lat":40.7403138,"lng":-73.97542272},{"lat":40.740737040000006,"lng":-73.97499948},{"lat":40.74145446,"lng":-73.97499948},{"lat":40.7441523,"lng":-73.97230164},{"lat":40.74460194,"lng":-73.97275128},{"lat":40.74460194,"lng":-73.97365056},{"lat":40.74505158,"lng":-73.9741002},{"lat":40.74595086,"lng":-73.9741002},{"lat":40.74685014,"lng":-73.97499948},{"lat":40.74774942,"lng":-73.97499948},{"lat":40.74819906,"lng":-73.97454984},{"lat":40.74819906,"lng":-73.971852},{"lat":40.74795768,"lng":-73.97161062},{"lat":40.74795768,"lng":-73.97137596},{"lat":40.74729978,"lng":-73.97071806},{"lat":40.74729978,"lng":-73.96915416},{"lat":40.74774942,"lng":-73.96870452},{"lat":40.74954798,"lng":-73.96870452},{"lat":40.75044726,"lng":-73.9696038},{"lat":40.75068192,"lng":-73.9696038},{"lat":40.75200444,"lng":-73.97092632},{"lat":40.75290372,"lng":-73.97092632},{"lat":40.75332696,"lng":-73.97050308},{"lat":40.75404438,"lng":-73.97050308},{"lat":40.75449402,"lng":-73.97005344},{"lat":40.75449402,"lng":-73.96915416},{"lat":40.75413531,"lng":-73.96879545},{"lat":40.75515192,"lng":-73.96777884},{"lat":40.75515192,"lng":-73.96690596},{"lat":40.7576415,"lng":-73.96690596},{"lat":40.75809114,"lng":-73.96645632},{"lat":40.75809114,"lng":-73.96555704},{"lat":40.7598897,"lng":-73.96375848},{"lat":40.7598897,"lng":-73.96373208},{"lat":40.76009796,"lng":-73.96373208},{"lat":40.7605476,"lng":-73.96328244},{"lat":40.7605476,"lng":-73.96238316},{"lat":40.761420480000005,"lng":-73.96151028},{"lat":40.7621379,"lng":-73.96151028},{"lat":40.76483574,"lng":-73.96420812},{"lat":40.76573502,"lng":-73.96420812},{"lat":40.76618466,"lng":-73.96375848},{"lat":40.76618466,"lng":-73.96106064},{"lat":40.7666343,"lng":-73.960611},{"lat":40.76753358,"lng":-73.96151028},{"lat":40.76933214,"lng":-73.96151028},{"lat":40.76978178,"lng":-73.96195992},{"lat":40.76978178,"lng":-73.96555704},{"lat":40.76843286,"lng":-73.96690596},{"lat":40.7666343,"lng":-73.96690596},{"lat":40.76618466,"lng":-73.9673556},{"lat":40.76618466,"lng":-73.96915416},{"lat":40.76528538,"lng":-73.97005344},{"lat":40.76528538,"lng":-73.971852},{"lat":40.76618466,"lng":-73.97275128},{"lat":40.76393646,"lng":-73.97499948},{"lat":40.7621379,"lng":-73.97320092},{"lat":40.76123862,"lng":-73.97320092},{"lat":40.76078898,"lng":-73.97365056},{"lat":40.76078898,"lng":-73.97454984},{"lat":40.7598897,"lng":-73.97544912},{"lat":40.7598897,"lng":-73.9763484},{"lat":40.76033934,"lng":-73.97679804},{"lat":40.76303718,"lng":-73.97679804},{"lat":40.76393646,"lng":-73.97589876},{"lat":40.76528538,"lng":-73.97724768},{"lat":40.76528538,"lng":-73.97904624},{"lat":40.76618466,"lng":-73.97994552},{"lat":40.76618466,"lng":-73.9808448},{"lat":40.7666343,"lng":-73.98129444},{"lat":40.76708394,"lng":-73.9808448},{"lat":40.76708394,"lng":-73.97994552},{"lat":40.76753358,"lng":-73.97949588},{"lat":40.76933214,"lng":-73.97949588},{"lat":40.77068106,"lng":-73.9808448},{"lat":40.76978178,"lng":-73.98174408},{"lat":40.76978178,"lng":-73.98354264},{"lat":40.76933214,"lng":-73.98399228},{"lat":40.76843286,"lng":-73.983093},{"lat":40.76573502,"lng":-73.983093},{"lat":40.76528538,"lng":-73.98354264},{"lat":40.76528538,"lng":-73.98444192},{"lat":40.7643861,"lng":-73.9853412},{"lat":40.7643861,"lng":-73.98713976},{"lat":40.76303718,"lng":-73.98848868},{"lat":40.76123862,"lng":-73.98848868},{"lat":40.7598897,"lng":-73.9898376},{"lat":40.7598897,"lng":-73.99163616},{"lat":40.76078898,"lng":-73.99253544},{"lat":40.7598897,"lng":-73.99343472},{"lat":40.7598897,"lng":-73.994334},{"lat":40.75944006,"lng":-73.99478364},{"lat":40.75854078,"lng":-73.99478364},{"lat":40.75809114,"lng":-73.994334},{"lat":40.75809114,"lng":-73.99343472},{"lat":40.7576415,"lng":-73.99298508},{"lat":40.75584294,"lng":-73.99298508},{"lat":40.7553933,"lng":-73.99343472},{"lat":40.7553933,"lng":-73.99523328},{"lat":40.75449402,"lng":-73.99613256},{"lat":40.75449402,"lng":-73.9988304},{"lat":40.75404438,"lng":-73.99928004},{"lat":40.7531451,"lng":-73.99838076},{"lat":40.75134654,"lng":-73.99838076},{"lat":40.7508969,"lng":-73.9988304},{"lat":40.7508969,"lng":-74.00242752},{"lat":40.75044726,"lng":-74.00287716},{"lat":40.74954798,"lng":-74.00287716},{"lat":40.74909834,"lng":-74.0033268},{"lat":40.74909834,"lng":-74.00422608},{"lat":40.74774942,"lng":-74.005575},{"lat":40.74685014,"lng":-74.005575},{"lat":40.7464005,"lng":-74.00602464},{"lat":40.74685014,"lng":-74.00647428},{"lat":40.74774942,"lng":-74.00647428},{"lat":40.74909834,"lng":-74.0078232},{"lat":40.7486487,"lng":-74.00827284},{"lat":40.74685014,"lng":-74.00827284},{"lat":40.74505158,"lng":-74.0100714},{"lat":40.74325302,"lng":-74.0100714},{"lat":40.74235374,"lng":-74.01097068},{"lat":40.7396559,"lng":-74.01097068},{"lat":40.73920626,"lng":-74.01142032},{"lat":40.73920626,"lng":-74.0122932}],"holes":[[{"lat":40.72637496,"lng":-73.97497308},{"lat":40.72637496,"lng":-73.97317452},{"lat":40.7268246,"lng":-73.97272488},{"lat":40.72817352,"lng":-73.9740738},{"lat":40.7268246,"lng":-73.97542272},{"lat":40.72637496,"lng":-73.97497308}]]},{"shell":[{"lat":40.78021443,"lng":-73.95260841},{"lat":40.77943248,"lng":-73.95339036},{"lat":40.77943248,"lng":-73.95518892},{"lat":40.78033176,"lng":-73.9560882},{"lat":40.77988212,"lng":-73.95653784},{"lat":40.77808356,"lng":-73.95473928},{"lat":40.7778489,"lng":-73.95473928},{"lat":40.77697602,"lng":-73.9538664},{"lat":40.77742566,"lng":-73.95341676},{"lat":40.77922422,"lng":-73.95341676},{"lat":40.7801235,"lng":-73.95251748},{"lat":40.78021443,"lng":-73.95260841}],"holes":[]},{"shell":[{"lat":40.78561011,"lng":-73.94901129},{"lat":40.78437852,"lng":-73.95024288},{"lat":40.78428759,"lng":-73.95015195},{"lat":40.78551918,"lng":-73.94892036},{"lat":40.78561011,"lng":-73.94901129}],"holes":[]},{"shell":[{"lat":40.682578019999994,"lng":-73.9808184},{"lat":40.6818606,"lng":-73.9808184},{"lat":40.68165234,"lng":-73.98061014000001},{"lat":40.68165234,"lng":-73.97904624},{"lat":40.68255162,"lng":-73.97814696},{"lat":40.68255162,"lng":-73.9763484},{"lat":40.68300126,"lng":-73.97589876},{"lat":40.68524946,"lng":-73.97814696},{"lat":40.68524946,"lng":-73.97994552},{"lat":40.68479982,"lng":-73.98039516},{"lat":40.68300126,"lng":-73.98039516},{"lat":40.682578019999994,"lng":-73.9808184}],"holes":[]},{"shell":[{"lat":40.71220146,"lng":-73.95384},{"lat":40.71243612,"lng":-73.95384},{"lat":40.71288576,"lng":-73.95428964},{"lat":40.71288576,"lng":-73.9545243},{"lat":40.71220146,"lng":-73.95384}],"holes":[]},{"shell":[{"lat":40.7133354,"lng":-73.94664576},{"lat":40.71378504,"lng":-73.94619612},{"lat":40.71378504,"lng":-73.94529684},{"lat":40.7133354,"lng":-73.9448472},{"lat":40.71243612,"lng":-73.9448472},{"lat":40.7110872,"lng":-73.94349828},{"lat":40.7110872,"lng":-73.94169972},{"lat":40.71018792,"lng":-73.94080044},{"lat":40.71243612,"lng":-73.93855224},{"lat":40.71378504,"lng":-73.93990116},{"lat":40.71378504,"lng":-73.94080044},{"lat":40.71423468,"lng":-73.94125008},{"lat":40.71513396,"lng":-73.94125008},{"lat":40.7155836,"lng":-73.94169972},{"lat":40.71468432,"lng":-73.942599},{"lat":40.71468432,"lng":-73.94349828},{"lat":40.71513396,"lng":-73.94394792},{"lat":40.71603324,"lng":-73.94394792},{"lat":40.71648288,"lng":-73.94439756},{"lat":40.7155836,"lng":-73.94529684},{"lat":40.7155836,"lng":-73.94889396},{"lat":40.71648288,"lng":-73.94979324},{"lat":40.71603324,"lng":-73.95024288},{"lat":40.71513396,"lng":-73.95024288},{"lat":40.71468432,"lng":-73.95069252},{"lat":40.71468432,"lng":-73.95249108},{"lat":40.71513396,"lng":-73.95294072},{"lat":40.71963036,"lng":-73.95294072},{"lat":40.72008,"lng":-73.95339036},{"lat":40.72008,"lng":-73.9560882},{"lat":40.71918072,"lng":-73.95698748},{"lat":40.72008,"lng":-73.95788676},{"lat":40.72008,"lng":-73.9605846},{"lat":40.71963036,"lng":-73.96103424},{"lat":40.71603324,"lng":-73.96103424},{"lat":40.7155836,"lng":-73.9605846},{"lat":40.7155836,"lng":-73.95968532},{"lat":40.71468432,"lng":-73.95878604},{"lat":40.71468432,"lng":-73.95698748},{"lat":40.71378504,"lng":-73.9560882},{"lat":40.71378504,"lng":-73.95339036},{"lat":40.71288576,"lng":-73.95249108},{"lat":40.71288576,"lng":-73.95069252},{"lat":40.71198648,"lng":-73.94979324},{"lat":40.71288576,"lng":-73.94889396},{"lat":40.71288576,"lng":-73.9470954},{"lat":40.7133354,"lng":-73.94664576}],"holes":[]},{"shell":[{"lat":40.74346128,"lng":-73.95272574},{"lat":40.74346128,"lng":-73.95500706},{"lat":40.74325302,"lng":-73.95521532},{"lat":40.74235374,"lng":-73.95521532},{"lat":40.7419041,"lng":-73.95476568},{"lat":40.7419041,"lng":-73.94937},{"lat":40.74235374,"lng":-73.94892036},{"lat":40.742562,"lng":-73.94912862000001},{"lat":40.742562,"lng":-73.94979324},{"lat":40.74280338,"lng":-73.95003462},{"lat":40.74280338,"lng":-73.95206784},{"lat":40.74346128,"lng":-73.95272574}],"holes":[]},{"shell":[{"lat":40.69064514,"lng":-73.98803904},{"lat":40.69064514,"lng":-73.98624048},{"lat":40.69199406,"lng":-73.98489156},{"lat":40.69424226,"lng":-73.98713976},{"lat":40.69199406,"lng":-73.98938796},{"lat":40.69064514,"lng":-73.98803904}],"holes":[]},{"shell":[{"lat":40.70008758,"lng":-73.98399228},{"lat":40.70098686,"lng":-73.983093},{"lat":40.70188614,"lng":-73.983093},{"lat":40.70233578,"lng":-73.98354264},{"lat":40.70233578,"lng":-73.9853412},{"lat":40.70278542,"lng":-73.98579084},{"lat":40.7036847,"lng":-73.98579084},{"lat":40.70413434,"lng":-73.98624048},{"lat":40.70413434,"lng":-73.98893832},{"lat":40.70278542,"lng":-73.99028724},{"lat":40.70098686,"lng":-73.99028724},{"lat":40.69963794,"lng":-73.98893832},{"lat":40.69963794,"lng":-73.98803904},{"lat":40.69873866,"lng":-73.98713976},{"lat":40.69873866,"lng":-73.9853412},{"lat":40.70008758,"lng":-73.98399228}],"holes":[]},{"shell":[{"lat":40.77427818,"lng":-73.9763484},{"lat":40.77472782,"lng":-73.97589876},{"lat":40.77652638,"lng":-73.97589876},{"lat":40.77697602,"lng":-73.9763484},{"lat":40.7756271,"lng":-73.97769732},{"lat":40.77427818,"lng":-73.9763484}],"holes":[]}],"properties":{}},{"search_id":"30_min","shapes":[{"shell":[{"lat":40.73723388,"lng":-74.01511692},{"lat":40.73687604,"lng":-74.01547476},{"lat":40.73687604,"lng":-74.01691222},{"lat":40.7340864,"lng":-74.01970186},{"lat":40.73300962,"lng":-74.01970186},{"lat":40.73282928,"lng":-74.01952152},{"lat":40.7321075,"lng":-74.01952152},{"lat":40.73093892,"lng":-74.01835294},{"lat":40.73093892,"lng":-74.0156551},{"lat":40.73048928,"lng":-74.01520546},{"lat":40.72959,"lng":-74.01520546},{"lat":40.72869072,"lng":-74.01610474},{"lat":40.7273418,"lng":-74.01475582},{"lat":40.72869072,"lng":-74.0134069},{"lat":40.73048928,"lng":-74.0134069},{"lat":40.73093892,"lng":-74.01295726},{"lat":40.73093892,"lng":-74.01205798},{"lat":40.73048928,"lng":-74.01160834},{"lat":40.72779144,"lng":-74.01160834},{"lat":40.7250936,"lng":-74.01430618},{"lat":40.72239576,"lng":-74.01430618},{"lat":40.72149648,"lng":-74.01520546},{"lat":40.71969792,"lng":-74.01520546},{"lat":40.71924828,"lng":-74.0156551},{"lat":40.71924828,"lng":-74.01655438},{"lat":40.71879864,"lng":-74.01700402},{"lat":40.71700008,"lng":-74.01700402},{"lat":40.71655044,"lng":-74.01745366},{"lat":40.71655044,"lng":-74.01925222},{"lat":40.71520152,"lng":-74.02060114},{"lat":40.71430224,"lng":-74.01970186},{"lat":40.71340296,"lng":-74.01970186},{"lat":40.71295332,"lng":-74.0201515},{"lat":40.71295332,"lng":-74.02284934},{"lat":40.71070512,"lng":-74.02509754},{"lat":40.70980584,"lng":-74.02599682},{"lat":40.7093562,"lng":-74.02554718},{"lat":40.7093562,"lng":-74.0246479},{"lat":40.70890656,"lng":-74.02419826},{"lat":40.70441016,"lng":-74.02419826},{"lat":40.70351088,"lng":-74.02329898},{"lat":40.70081304,"lng":-74.02329898},{"lat":40.69946412,"lng":-74.02195006},{"lat":40.69946412,"lng":-74.0201515},{"lat":40.69901448,"lng":-74.01970186},{"lat":40.6981152,"lng":-74.01970186},{"lat":40.695867,"lng":-74.01745366},{"lat":40.695867,"lng":-74.0156551},{"lat":40.69496772,"lng":-74.01475582},{"lat":40.69496772,"lng":-74.01062051999999},{"lat":40.69667774,"lng":-74.0089105},{"lat":40.69721592,"lng":-74.0089105},{"lat":40.69901448,"lng":-74.00711194},{"lat":40.69991376,"lng":-74.00711194},{"lat":40.7003634,"lng":-74.0066623},{"lat":40.7003634,"lng":-74.00486374},{"lat":40.70171232,"lng":-74.00351482},{"lat":40.70351088,"lng":-74.00351482},{"lat":40.70396052,"lng":-74.00306518},{"lat":40.70396052,"lng":-74.00126662},{"lat":40.7048598,"lng":-74.00036734},{"lat":40.7048598,"lng":-73.99856878},{"lat":40.70755764,"lng":-73.99587094},{"lat":40.70755764,"lng":-73.99407238},{"lat":40.70665836,"lng":-73.9931731},{"lat":40.70665836,"lng":-73.98957598},{"lat":40.70755764,"lng":-73.9886767},{"lat":40.707108,"lng":-73.98822706},{"lat":40.70620872,"lng":-73.98822706},{"lat":40.70575908,"lng":-73.9886767},{"lat":40.70575908,"lng":-73.99137454},{"lat":40.70396052,"lng":-73.9931731},{"lat":40.70396052,"lng":-73.99407238},{"lat":40.70351088,"lng":-73.99452202},{"lat":40.69991376,"lng":-73.99452202},{"lat":40.6981152,"lng":-73.99632058},{"lat":40.69631664,"lng":-73.99632058},{"lat":40.69451808,"lng":-73.99811914},{"lat":40.69182024,"lng":-73.9954213},{"lat":40.69092096,"lng":-73.9954213},{"lat":40.69047132,"lng":-73.99497166},{"lat":40.69047132,"lng":-73.99137454},{"lat":40.69002168,"lng":-73.9909249},{"lat":40.68867276,"lng":-73.99227382},{"lat":40.68867276,"lng":-73.9931731},{"lat":40.68822312,"lng":-73.99362274},{"lat":40.68732384,"lng":-73.99362274},{"lat":40.68642456,"lng":-73.99452202},{"lat":40.684626,"lng":-73.99452202},{"lat":40.68372672,"lng":-73.99362274},{"lat":40.6823778,"lng":-73.99497166},{"lat":40.6823778,"lng":-73.99677022},{"lat":40.68192816,"lng":-73.99721986},{"lat":40.67833104,"lng":-73.99721986},{"lat":40.6778814,"lng":-73.99677022},{"lat":40.6778814,"lng":-73.99497166},{"lat":40.67878068,"lng":-73.99407238},{"lat":40.6778814,"lng":-73.9931731},{"lat":40.6778814,"lng":-73.99227382},{"lat":40.67833104,"lng":-73.99182418},{"lat":40.68102888,"lng":-73.99182418},{"lat":40.68192816,"lng":-73.99272346},{"lat":40.68327708,"lng":-73.99137454},{"lat":40.68327708,"lng":-73.98777742},{"lat":40.68372672,"lng":-73.98732778},{"lat":40.68552528,"lng":-73.98732778},{"lat":40.68597492,"lng":-73.98687814},{"lat":40.684626,"lng":-73.98552922},{"lat":40.68282744,"lng":-73.98552922},{"lat":40.67923032,"lng":-73.9819321},{"lat":40.67833104,"lng":-73.9819321},{"lat":40.6778814,"lng":-73.98148246},{"lat":40.6778814,"lng":-73.98058318},{"lat":40.67967996,"lng":-73.97878462},{"lat":40.67967996,"lng":-73.97608678},{"lat":40.67878068,"lng":-73.9751875},{"lat":40.6801296,"lng":-73.97383858},{"lat":40.68192816,"lng":-73.97383858},{"lat":40.6823778,"lng":-73.97338894},{"lat":40.6823778,"lng":-73.97159038},{"lat":40.68282744,"lng":-73.97114074},{"lat":40.68372672,"lng":-73.97114074},{"lat":40.68417636,"lng":-73.97159038},{"lat":40.68417636,"lng":-73.97338894},{"lat":40.684626,"lng":-73.97383858},{"lat":40.68822312,"lng":-73.97383858},{"lat":40.68867276,"lng":-73.97428822},{"lat":40.68867276,"lng":-73.9796839},{"lat":40.6891224,"lng":-73.98013354},{"lat":40.69182024,"lng":-73.98013354},{"lat":40.69226988,"lng":-73.98058318},{"lat":40.69226988,"lng":-73.98148246},{"lat":40.69271952,"lng":-73.9819321},{"lat":40.6936188,"lng":-73.9819321},{"lat":40.69406844,"lng":-73.98238174},{"lat":40.69406844,"lng":-73.98328102},{"lat":40.69541736,"lng":-73.98462994},{"lat":40.69766556,"lng":-73.98238174},{"lat":40.69766556,"lng":-73.98058318},{"lat":40.69901448,"lng":-73.97923426},{"lat":40.70081304,"lng":-73.97923426},{"lat":40.70171232,"lng":-73.98013354},{"lat":40.70351088,"lng":-73.98013354},{"lat":40.7048598,"lng":-73.98148246},{"lat":40.7048598,"lng":-73.98328102},{"lat":40.70530944,"lng":-73.98373066},{"lat":40.70620872,"lng":-73.98373066},{"lat":40.70665836,"lng":-73.9841803},{"lat":40.70665836,"lng":-73.98507958},{"lat":40.707108,"lng":-73.98552922},{"lat":40.70890656,"lng":-73.98552922},{"lat":40.7093562,"lng":-73.98507958},{"lat":40.7093562,"lng":-73.98328102},{"lat":40.71115476,"lng":-73.98148246},{"lat":40.71115476,"lng":-73.9796839},{"lat":40.71205404,"lng":-73.97878462},{"lat":40.71205404,"lng":-73.97788534},{"lat":40.71250368,"lng":-73.9774357},{"lat":40.71340296,"lng":-73.9774357},{"lat":40.7161008,"lng":-73.97473786},{"lat":40.71969792,"lng":-73.97473786},{"lat":40.7205972,"lng":-73.97383858},{"lat":40.72239576,"lng":-73.97383858},{"lat":40.7228454,"lng":-73.97338894},{"lat":40.7228454,"lng":-73.9706911},{"lat":40.72329504,"lng":-73.97024146},{"lat":40.72419432,"lng":-73.97114074},{"lat":40.7250936,"lng":-73.97114074},{"lat":40.72599288,"lng":-73.97204002},{"lat":40.72644252,"lng":-73.97159038},{"lat":40.72644252,"lng":-73.96799326},{"lat":40.72689216,"lng":-73.96754362},{"lat":40.72779144,"lng":-73.96754362},{"lat":40.72869072,"lng":-73.9684429},{"lat":40.73048928,"lng":-73.9684429},{"lat":40.73085017,"lng":-73.96808201},{"lat":40.73103072,"lng":-73.96826256},{"lat":40.7321075,"lng":-73.96826256},{"lat":40.73363676,"lng":-73.96979182},{"lat":40.73363676,"lng":-73.9706911},{"lat":40.7340864,"lng":-73.97114074},{"lat":40.73498568,"lng":-73.97114074},{"lat":40.73588496,"lng":-73.97024146},{"lat":40.74217992,"lng":-73.97024146},{"lat":40.74262956,"lng":-73.96979182},{"lat":40.74262956,"lng":-73.96889254},{"lat":40.74299045,"lng":-73.96853165},{"lat":40.74362064,"lng":-73.96916184},{"lat":40.74379814,"lng":-73.96916184},{"lat":40.74397848,"lng":-73.96934218},{"lat":40.7475756,"lng":-73.96934218},{"lat":40.74892452,"lng":-73.96799326},{"lat":40.74892452,"lng":-73.96565652},{"lat":40.7499156,"lng":-73.96466544},{"lat":40.7500931,"lng":-73.96466544},{"lat":40.75117272,"lng":-73.96574506},{"lat":40.752072,"lng":-73.96574506},{"lat":40.75252164,"lng":-73.96529542},{"lat":40.75252164,"lng":-73.96259758},{"lat":40.75297128,"lng":-73.96214794},{"lat":40.75476984,"lng":-73.96214794},{"lat":40.75495018,"lng":-73.9619676},{"lat":40.75531128,"lng":-73.9619676},{"lat":40.75576092,"lng":-73.96151796},{"lat":40.75576092,"lng":-73.96134046},{"lat":40.7565684,"lng":-73.96214794},{"lat":40.75746768,"lng":-73.96214794},{"lat":40.75791732,"lng":-73.9616983},{"lat":40.75791732,"lng":-73.96079902},{"lat":40.75701804,"lng":-73.95989974},{"lat":40.75764802,"lng":-73.95926976},{"lat":40.7589084,"lng":-73.95926976},{"lat":40.76025732,"lng":-73.95792084},{"lat":40.76025732,"lng":-73.95684406000001},{"lat":40.76196408,"lng":-73.95855082},{"lat":40.76376264,"lng":-73.95855082},{"lat":40.76466192,"lng":-73.95765154},{"lat":40.7655612,"lng":-73.95855082},{"lat":40.76735976,"lng":-73.95855082},{"lat":40.76825904,"lng":-73.9594501},{"lat":40.7700576,"lng":-73.9594501},{"lat":40.77050724,"lng":-73.95900046},{"lat":40.77050724,"lng":-73.9572019},{"lat":40.76960796,"lng":-73.95630262},{"lat":40.77095688,"lng":-73.9549537},{"lat":40.77275544,"lng":-73.9549537},{"lat":40.77365472,"lng":-73.95585298},{"lat":40.774554,"lng":-73.95585298},{"lat":40.77500364,"lng":-73.95540334},{"lat":40.77500364,"lng":-73.9527055},{"lat":40.77590292,"lng":-73.95180622},{"lat":40.77590292,"lng":-73.95000766},{"lat":40.77500364,"lng":-73.94910838},{"lat":40.77635256,"lng":-73.94775946},{"lat":40.77815112,"lng":-73.94775946},{"lat":40.7790504,"lng":-73.94865874},{"lat":40.78084896,"lng":-73.94865874},{"lat":40.7812986,"lng":-73.94910838},{"lat":40.7812986,"lng":-73.95000766},{"lat":40.78174824,"lng":-73.9504573},{"lat":40.78219788,"lng":-73.95000766},{"lat":40.78219788,"lng":-73.94730982},{"lat":40.78444608,"lng":-73.94506162},{"lat":40.78624464,"lng":-73.94686018},{"lat":40.78714392,"lng":-73.94686018},{"lat":40.78759356,"lng":-73.94641054},{"lat":40.78759356,"lng":-73.94551126},{"lat":40.78894248,"lng":-73.94416234},{"lat":40.78930358,"lng":-73.94416234},{"lat":40.78903428,"lng":-73.94443164},{"lat":40.78903428,"lng":-73.9462302},{"lat":40.78948392,"lng":-73.94667984},{"lat":40.79128248,"lng":-73.94667984},{"lat":40.79173212,"lng":-73.94712948},{"lat":40.7903832,"lng":-73.9484784},{"lat":40.78948392,"lng":-73.94757912},{"lat":40.78858464,"lng":-73.94757912},{"lat":40.788135,"lng":-73.94802876},{"lat":40.788135,"lng":-73.94892804},{"lat":40.78903428,"lng":-73.94982732},{"lat":40.788135,"lng":-73.9507266},{"lat":40.788135,"lng":-73.95432372},{"lat":40.78849284,"lng":-73.95468156000001},{"lat":40.78849284,"lng":-73.95558084000001},{"lat":40.78678608,"lng":-73.95387408},{"lat":40.78318896,"lng":-73.95387408},{"lat":40.78273932,"lng":-73.95432372},{"lat":40.78273932,"lng":-73.9594501},{"lat":40.78174824,"lng":-73.9594501},{"lat":40.78004148,"lng":-73.96115685999999},{"lat":40.78004148,"lng":-73.9597194},{"lat":40.77869256,"lng":-73.95837048},{"lat":40.776894,"lng":-73.95837048},{"lat":40.77644436,"lng":-73.95882012},{"lat":40.77644436,"lng":-73.96061868},{"lat":40.777163089999995,"lng":-73.96133741},{"lat":40.7768022,"lng":-73.9616983},{"lat":40.7768022,"lng":-73.96439614},{"lat":40.7765329,"lng":-73.96466544},{"lat":40.77617222,"lng":-73.96466544},{"lat":40.77545328,"lng":-73.9639465},{"lat":40.77365472,"lng":-73.9639465},{"lat":40.7723058,"lng":-73.96529542},{"lat":40.7723058,"lng":-73.96709398},{"lat":40.76870868,"lng":-73.9706911},{"lat":40.76870868,"lng":-73.97248966},{"lat":40.76960796,"lng":-73.97338894},{"lat":40.76870868,"lng":-73.97428822},{"lat":40.76870868,"lng":-73.97608678},{"lat":40.76915832,"lng":-73.97653642},{"lat":40.77185616,"lng":-73.97653642},{"lat":40.7723058,"lng":-73.97608678},{"lat":40.7723058,"lng":-73.97338894},{"lat":40.77365472,"lng":-73.97204002},{"lat":40.77815112,"lng":-73.97204002},{"lat":40.77860076,"lng":-73.97159038},{"lat":40.77860076,"lng":-73.9706911},{"lat":40.77770148,"lng":-73.96979182},{"lat":40.77815112,"lng":-73.96934218},{"lat":40.77994968,"lng":-73.96934218},{"lat":40.78084896,"lng":-73.9684429},{"lat":40.78264752,"lng":-73.9684429},{"lat":40.7835468,"lng":-73.96934218},{"lat":40.78444608,"lng":-73.96934218},{"lat":40.78489572,"lng":-73.96889254},{"lat":40.78489572,"lng":-73.96799326},{"lat":40.78399644,"lng":-73.96709398},{"lat":40.78444608,"lng":-73.96664434},{"lat":40.78534536,"lng":-73.96754362},{"lat":40.78714392,"lng":-73.96754362},{"lat":40.7880432,"lng":-73.96664434},{"lat":40.78849284,"lng":-73.96709398},{"lat":40.78849284,"lng":-73.96889254},{"lat":40.78759356,"lng":-73.96979182},{"lat":40.78759356,"lng":-73.97159038},{"lat":40.78714392,"lng":-73.97204002},{"lat":40.78624464,"lng":-73.97114074},{"lat":40.78444608,"lng":-73.97114074},{"lat":40.78309716,"lng":-73.97248966},{"lat":40.78309716,"lng":-73.97428822},{"lat":40.78174824,"lng":-73.97563714},{"lat":40.7812986,"lng":-73.9751875},{"lat":40.7812986,"lng":-73.97428822},{"lat":40.78084896,"lng":-73.97383858},{"lat":40.77994968,"lng":-73.97383858},{"lat":40.77770148,"lng":-73.97608678},{"lat":40.77770148,"lng":-73.97788534},{"lat":40.7812986,"lng":-73.98148246},{"lat":40.78039932,"lng":-73.98238174},{"lat":40.78039932,"lng":-73.9841803},{"lat":40.7790504,"lng":-73.98552922},{"lat":40.77860076,"lng":-73.98507958},{"lat":40.77860076,"lng":-73.9841803},{"lat":40.77815112,"lng":-73.98373066},{"lat":40.77725184,"lng":-73.98373066},{"lat":40.77635256,"lng":-73.98283138},{"lat":40.77545328,"lng":-73.98283138},{"lat":40.77500364,"lng":-73.98328102},{"lat":40.77500364,"lng":-73.9841803},{"lat":40.774554,"lng":-73.98462994},{"lat":40.77275544,"lng":-73.98462994},{"lat":40.77185616,"lng":-73.98552922},{"lat":40.77095688,"lng":-73.98552922},{"lat":40.77050724,"lng":-73.98597886},{"lat":40.77050724,"lng":-73.98687814},{"lat":40.7700576,"lng":-73.98732778},{"lat":40.76825904,"lng":-73.98732778},{"lat":40.7678094,"lng":-73.98777742},{"lat":40.7678094,"lng":-73.9886767},{"lat":40.76735976,"lng":-73.98912634},{"lat":40.7655612,"lng":-73.98912634},{"lat":40.76511156,"lng":-73.98957598},{"lat":40.76601084,"lng":-73.99047526},{"lat":40.76601084,"lng":-73.99227382},{"lat":40.76466192,"lng":-73.99362274},{"lat":40.76286336,"lng":-73.99362274},{"lat":40.76151444,"lng":-73.99497166},{"lat":40.76151444,"lng":-73.9976695},{"lat":40.76016552,"lng":-73.99901842},{"lat":40.75926624,"lng":-73.99811914},{"lat":40.75746768,"lng":-73.99811914},{"lat":40.75701804,"lng":-73.99856878},{"lat":40.75701804,"lng":-73.99946806},{"lat":40.75746768,"lng":-73.9999177},{"lat":40.75836696,"lng":-73.9999177},{"lat":40.7588166,"lng":-74.00036734},{"lat":40.75791732,"lng":-74.00126662},{"lat":40.75791732,"lng":-74.00306518},{"lat":40.7588166,"lng":-74.00396446},{"lat":40.7565684,"lng":-74.00621266},{"lat":40.752072,"lng":-74.00621266},{"lat":40.75162236,"lng":-74.0066623},{"lat":40.75162236,"lng":-74.00756158},{"lat":40.75252164,"lng":-74.00846086},{"lat":40.74847488,"lng":-74.01250762},{"lat":40.74667632,"lng":-74.01250762},{"lat":40.74487776,"lng":-74.01430618},{"lat":40.74352884,"lng":-74.01295726},{"lat":40.74352884,"lng":-74.01205798},{"lat":40.7430792,"lng":-74.01160834},{"lat":40.74262956,"lng":-74.01205798},{"lat":40.74262956,"lng":-74.01295726},{"lat":40.74128064,"lng":-74.01430618},{"lat":40.74038136,"lng":-74.0134069},{"lat":40.7385828,"lng":-74.0134069},{"lat":40.73723388,"lng":-74.01475582},{"lat":40.73723388,"lng":-74.01511692}],"holes":[]},{"shell":[{"lat":40.65576029,"lng":-74.00180501},{"lat":40.65684012,"lng":-74.00288484},{"lat":40.65684012,"lng":-74.0046834},{"lat":40.65639048,"lng":-74.00513304},{"lat":40.65557995,"lng":-74.00594357},{"lat":40.65494976,"lng":-74.00531338},{"lat":40.65477226,"lng":-74.00531338},{"lat":40.65351209,"lng":-74.00405321},{"lat":40.65576029,"lng":-74.00180501}],"holes":[]},{"shell":[{"lat":40.69469842,"lng":-73.95027696},{"lat":40.69416024,"lng":-73.95027696},{"lat":40.6937106,"lng":-73.94982732},{"lat":40.69316916,"lng":-73.94928587999999},{"lat":40.69316916,"lng":-73.9482091},{"lat":40.6936188,"lng":-73.94775946},{"lat":40.69613914,"lng":-73.94775946},{"lat":40.69640844,"lng":-73.94802876},{"lat":40.69640844,"lng":-73.94946622},{"lat":40.69631664,"lng":-73.94955802},{"lat":40.69541736,"lng":-73.94955802},{"lat":40.69469842,"lng":-73.95027696}],"holes":[]},{"shell":[{"lat":40.702522849999994,"lng":-73.95144533},{"lat":40.70225376,"lng":-73.95117624},{"lat":40.7004552,"lng":-73.95117624},{"lat":40.70000556,"lng":-73.9507266},{"lat":40.70000556,"lng":-73.94946622},{"lat":40.7003634,"lng":-73.94910838},{"lat":40.7003634,"lng":-73.9485702},{"lat":40.70090484,"lng":-73.94802876},{"lat":40.700724290000004,"lng":-73.94784821},{"lat":40.70081304,"lng":-73.94775946},{"lat":40.70171232,"lng":-73.94775946},{"lat":40.70216196,"lng":-73.94730982},{"lat":40.70216196,"lng":-73.9437127},{"lat":40.70306124,"lng":-73.94281342},{"lat":40.7026116,"lng":-73.94236378},{"lat":40.69901448,"lng":-73.94236378},{"lat":40.69856484,"lng":-73.94191414},{"lat":40.69856484,"lng":-73.94011558},{"lat":40.69901448,"lng":-73.93966594},{"lat":40.70081304,"lng":-73.93966594},{"lat":40.70126268,"lng":-73.9392163},{"lat":40.70126268,"lng":-73.93292134},{"lat":40.701623569999995,"lng":-73.93256045},{"lat":40.70180412,"lng":-73.932741},{"lat":40.70180412,"lng":-73.93453956},{"lat":40.70315304,"lng":-73.93588848},{"lat":40.70405232,"lng":-73.93588848},{"lat":40.70450196,"lng":-73.93543884},{"lat":40.70360268,"lng":-73.93453956},{"lat":40.70360268,"lng":-73.93364028},{"lat":40.7027034,"lng":-73.932741},{"lat":40.70315304,"lng":-73.93229136},{"lat":40.70405232,"lng":-73.93229136},{"lat":40.70450196,"lng":-73.93184172},{"lat":40.70450196,"lng":-73.93094244},{"lat":40.70585088,"lng":-73.92959352},{"lat":40.70899836,"lng":-73.932741},{"lat":40.70899836,"lng":-73.93543884},{"lat":40.709448,"lng":-73.93588848},{"lat":40.71124656,"lng":-73.93408992},{"lat":40.71304512,"lng":-73.93408992},{"lat":40.7139444,"lng":-73.9349892},{"lat":40.71484368,"lng":-73.9349892},{"lat":40.71529332,"lng":-73.93543884},{"lat":40.71529332,"lng":-73.93633812},{"lat":40.71574296,"lng":-73.93678776},{"lat":40.71664224,"lng":-73.93678776},{"lat":40.71709188,"lng":-73.9372374},{"lat":40.71709188,"lng":-73.93813668},{"lat":40.71889044,"lng":-73.93993524},{"lat":40.71889044,"lng":-73.9417338},{"lat":40.71799116,"lng":-73.94263308},{"lat":40.7184408,"lng":-73.94308272},{"lat":40.72023936,"lng":-73.94308272},{"lat":40.720689,"lng":-73.94353236},{"lat":40.720689,"lng":-73.94533092},{"lat":40.72158828,"lng":-73.9462302},{"lat":40.72158828,"lng":-73.94712948},{"lat":40.72113864,"lng":-73.94757912},{"lat":40.72023936,"lng":-73.94757912},{"lat":40.71978972,"lng":-73.94802876},{"lat":40.71978972,"lng":-73.94982732},{"lat":40.72023936,"lng":-73.95027696},{"lat":40.72113864,"lng":-73.94937768},{"lat":40.72203792,"lng":-73.94937768},{"lat":40.72248756,"lng":-73.94982732},{"lat":40.72248756,"lng":-73.95252516},{"lat":40.72338684,"lng":-73.95342444},{"lat":40.72338684,"lng":-73.955223},{"lat":40.72158828,"lng":-73.95702156},{"lat":40.72338684,"lng":-73.95882012},{"lat":40.72338684,"lng":-73.96061868},{"lat":40.72248756,"lng":-73.96151796},{"lat":40.72248756,"lng":-73.9642158},{"lat":40.72203792,"lng":-73.96466544},{"lat":40.71754152,"lng":-73.96466544},{"lat":40.71664224,"lng":-73.96376616},{"lat":40.71484368,"lng":-73.96376616},{"lat":40.71349476,"lng":-73.96511508},{"lat":40.71349476,"lng":-73.96574506},{"lat":40.70980584,"lng":-73.96574506},{"lat":40.7093562,"lng":-73.96529542},{"lat":40.7093562,"lng":-73.96349686},{"lat":40.70845692,"lng":-73.96259758},{"lat":40.707108,"lng":-73.96124866},{"lat":40.70530944,"lng":-73.96124866},{"lat":40.7048598,"lng":-73.96079902},{"lat":40.7048598,"lng":-73.95900046},{"lat":40.70575908,"lng":-73.95810118},{"lat":40.7048598,"lng":-73.9572019},{"lat":40.7048598,"lng":-73.95360478},{"lat":40.7026116,"lng":-73.95135658},{"lat":40.702522849999994,"lng":-73.95144533}],"holes":[[{"lat":40.70899836,"lng":-73.94712948},{"lat":40.70854872,"lng":-73.94757912},{"lat":40.7071998,"lng":-73.9462302},{"lat":40.7071998,"lng":-73.94443164},{"lat":40.70764944,"lng":-73.943982},{"lat":40.709448,"lng":-73.943982},{"lat":40.70989764,"lng":-73.94443164},{"lat":40.70989764,"lng":-73.9462302},{"lat":40.70899836,"lng":-73.94712948}]]},{"shell":[{"lat":40.74415882,"lng":-73.95926976},{"lat":40.74272136,"lng":-73.95926976},{"lat":40.74199958,"lng":-73.95926976},{"lat":40.74173028,"lng":-73.95900046},{"lat":40.74137244,"lng":-73.95864261999999},{"lat":40.74137244,"lng":-73.955223},{"lat":40.73957388,"lng":-73.95342444},{"lat":40.73957388,"lng":-73.95162588},{"lat":40.73939333,"lng":-73.95144533},{"lat":40.73993172,"lng":-73.95090694},{"lat":40.73993172,"lng":-73.95000766},{"lat":40.74038136,"lng":-73.94955802},{"lat":40.74128064,"lng":-73.94955802},{"lat":40.74173028,"lng":-73.94910838},{"lat":40.74173028,"lng":-73.9482091},{"lat":40.74217992,"lng":-73.94775946},{"lat":40.74397848,"lng":-73.94775946},{"lat":40.7453274,"lng":-73.94641054},{"lat":40.7453274,"lng":-73.94281342},{"lat":40.74667632,"lng":-73.9414645},{"lat":40.7475756,"lng":-73.9414645},{"lat":40.74802524,"lng":-73.94101486},{"lat":40.74802524,"lng":-73.93651846},{"lat":40.74847488,"lng":-73.93606882},{"lat":40.75027344,"lng":-73.93606882},{"lat":40.75342092,"lng":-73.9392163},{"lat":40.75252164,"lng":-73.94011558},{"lat":40.75252164,"lng":-73.94101486},{"lat":40.75297128,"lng":-73.9414645},{"lat":40.75387056,"lng":-73.9414645},{"lat":40.75521948,"lng":-73.94281342},{"lat":40.75476984,"lng":-73.94326306},{"lat":40.75387056,"lng":-73.94236378},{"lat":40.752072,"lng":-73.94236378},{"lat":40.75117272,"lng":-73.94326306},{"lat":40.75027344,"lng":-73.94326306},{"lat":40.7498238,"lng":-73.9437127},{"lat":40.7498238,"lng":-73.94461198},{"lat":40.74892452,"lng":-73.94551126},{"lat":40.74892452,"lng":-73.9482091},{"lat":40.74847488,"lng":-73.94865874},{"lat":40.74577704,"lng":-73.94865874},{"lat":40.7453274,"lng":-73.94910838},{"lat":40.7453274,"lng":-73.95036876},{"lat":40.74496956,"lng":-73.9507266},{"lat":40.74496956,"lng":-73.95252516},{"lat":40.7453274,"lng":-73.952883},{"lat":40.7453274,"lng":-73.95360478},{"lat":40.74442812,"lng":-73.95450406},{"lat":40.7453274,"lng":-73.95540334},{"lat":40.7453274,"lng":-73.95630262},{"lat":40.74442812,"lng":-73.9572019},{"lat":40.74442812,"lng":-73.95900046},{"lat":40.74415882,"lng":-73.95926976}],"holes":[]},{"shell":[{"lat":40.74137244,"lng":-73.9460527},{"lat":40.74137244,"lng":-73.94496982000001},{"lat":40.74217992,"lng":-73.94416234},{"lat":40.74397848,"lng":-73.94416234},{"lat":40.74442812,"lng":-73.94461198},{"lat":40.74217992,"lng":-73.94686018},{"lat":40.74137244,"lng":-73.9460527}],"holes":[]},{"shell":[{"lat":40.70540124,"lng":-73.92105036},{"lat":40.70585088,"lng":-73.92060072},{"lat":40.7071998,"lng":-73.92194964},{"lat":40.7071998,"lng":-73.9237482},{"lat":40.70809908,"lng":-73.92464748},{"lat":40.70764944,"lng":-73.92509712},{"lat":40.70540124,"lng":-73.92284892},{"lat":40.70540124,"lng":-73.92105036}],"holes":[]},{"shell":[{"lat":40.71565116,"lng":-74.0336407},{"lat":40.71475188,"lng":-74.03274142},{"lat":40.7161008,"lng":-74.0313925},{"lat":40.71789936,"lng":-74.0313925},{"lat":40.718349,"lng":-74.03184214},{"lat":40.71744972,"lng":-74.03274142},{"lat":40.71744972,"lng":-74.03453998},{"lat":40.71700008,"lng":-74.03498962},{"lat":40.71565116,"lng":-74.0336407}],"holes":[]},{"shell":[{"lat":40.73453604,"lng":-74.03004358},{"lat":40.7340864,"lng":-74.02959394},{"lat":40.73318712,"lng":-74.02959394},{"lat":40.73273748,"lng":-74.0291443},{"lat":40.73273748,"lng":-74.02554718},{"lat":40.7340864,"lng":-74.02419826},{"lat":40.73588496,"lng":-74.02419826},{"lat":40.73993172,"lng":-74.02824502},{"lat":40.73903244,"lng":-74.0291443},{"lat":40.73903244,"lng":-74.03094286},{"lat":40.73588496,"lng":-74.03409034},{"lat":40.73453604,"lng":-74.03274142},{"lat":40.73453604,"lng":-74.03004358}],"holes":[]},{"shell":[{"lat":40.68327708,"lng":-74.00486374},{"lat":40.68327708,"lng":-74.00576302},{"lat":40.68282744,"lng":-74.00621266},{"lat":40.67878068,"lng":-74.0021659},{"lat":40.67923032,"lng":-74.00171626},{"lat":40.6801296,"lng":-74.00261554},{"lat":40.68192816,"lng":-74.00261554},{"lat":40.68282744,"lng":-74.00171626},{"lat":40.68327708,"lng":-74.0021659},{"lat":40.68327708,"lng":-74.00486374}],"holes":[]},{"shell":[{"lat":40.66169436,"lng":-73.9976695},{"lat":40.66304328,"lng":-73.99632058},{"lat":40.6643922,"lng":-73.9976695},{"lat":40.6643922,"lng":-73.99946806},{"lat":40.66394256,"lng":-73.9999177},{"lat":40.66169436,"lng":-73.9976695}],"holes":[]},{"shell":[{"lat":40.673385,"lng":-73.99677022},{"lat":40.67248572,"lng":-73.99587094},{"lat":40.67293536,"lng":-73.9954213},{"lat":40.67473392,"lng":-73.9954213},{"lat":40.67518356,"lng":-73.99587094},{"lat":40.67383464,"lng":-73.99721986},{"lat":40.673385,"lng":-73.99677022}],"holes":[]},{"shell":[{"lat":40.6778814,"lng":-73.97159038},{"lat":40.67743176,"lng":-73.97204002},{"lat":40.6756332,"lng":-73.97204002},{"lat":40.67428428,"lng":-73.9706911},{"lat":40.6756332,"lng":-73.96934218},{"lat":40.6778814,"lng":-73.97159038}],"holes":[]},{"shell":[{"lat":40.7902914,"lng":-73.96349686},{"lat":40.7902914,"lng":-73.96259758},{"lat":40.79074104,"lng":-73.96214794},{"lat":40.79298924,"lng":-73.96439614},{"lat":40.79164032,"lng":-73.96574506},{"lat":40.7902914,"lng":-73.96439614},{"lat":40.7902914,"lng":-73.96349686}],"holes":[]},{"shell":[{"lat":40.75836696,"lng":-73.95135658},{"lat":40.76016552,"lng":-73.95135658},{"lat":40.76151444,"lng":-73.9527055},{"lat":40.75836696,"lng":-73.95585298},{"lat":40.75701804,"lng":-73.95450406},{"lat":40.75791732,"lng":-73.95360478},{"lat":40.75791732,"lng":-73.95180622},{"lat":40.75836696,"lng":-73.95135658}],"holes":[]}],"properties":{}},{"search_id":"bill_15_min","shapes":[{"shell":[{"lat":40.73930866,"lng":-74.0043354},{"lat":40.74245614,"lng":-74.00118792},{"lat":40.7420065,"lng":-74.00073828000001},{"lat":40.74020794,"lng":-74.00073828000001},{"lat":40.73885902,"lng":-73.99938936000001},{"lat":40.73885902,"lng":-73.99759080000001},{"lat":40.740657580000004,"lng":-73.99579224},{"lat":40.740657580000004,"lng":-73.99489296},{"lat":40.73885902,"lng":-73.9930944},{"lat":40.7397583,"lng":-73.99219512},{"lat":40.7397583,"lng":-73.98859800000001},{"lat":40.74020794,"lng":-73.98814836000001},{"lat":40.74110722,"lng":-73.98814836000001},{"lat":40.74155686,"lng":-73.98769872000001},{"lat":40.74155686,"lng":-73.98590016},{"lat":40.7420065,"lng":-73.98545052},{"lat":40.74380506,"lng":-73.98545052},{"lat":40.74785182,"lng":-73.98140376},{"lat":40.74740218,"lng":-73.98095412},{"lat":40.745603620000004,"lng":-73.98095412},{"lat":40.74470434,"lng":-73.9818534},{"lat":40.74290578,"lng":-73.9818534},{"lat":40.74155686,"lng":-73.98320232},{"lat":40.74155686,"lng":-73.9841016},{"lat":40.74110722,"lng":-73.98455124},{"lat":40.73840938,"lng":-73.98455124},{"lat":40.7375101,"lng":-73.98545052},{"lat":40.73481226,"lng":-73.98545052},{"lat":40.73436262,"lng":-73.98500088},{"lat":40.73436262,"lng":-73.98140376},{"lat":40.7330137,"lng":-73.98005484000001},{"lat":40.73211442,"lng":-73.98005484000001},{"lat":40.73121514,"lng":-73.97915556000001},{"lat":40.73031586,"lng":-73.97915556000001},{"lat":40.72941658,"lng":-73.98005484000001},{"lat":40.72761802,"lng":-73.98005484000001},{"lat":40.7262691,"lng":-73.98140376},{"lat":40.7262691,"lng":-73.98320232},{"lat":40.7217727,"lng":-73.98769872000001},{"lat":40.7217727,"lng":-73.99129584},{"lat":40.72132306,"lng":-73.99174548},{"lat":40.72042378,"lng":-73.99174548},{"lat":40.71997414,"lng":-73.99129584},{"lat":40.71997414,"lng":-73.99039656000001},{"lat":40.7195245,"lng":-73.98994692000001},{"lat":40.71862522,"lng":-73.98994692000001},{"lat":40.7172763,"lng":-73.99129584},{"lat":40.7172763,"lng":-73.9930944},{"lat":40.71547774,"lng":-73.99489296},{"lat":40.71817558,"lng":-73.99759080000001},{"lat":40.71817558,"lng":-73.99938936000001},{"lat":40.7172763,"lng":-74.00028864000001},{"lat":40.7172763,"lng":-74.0020872},{"lat":40.716377019999996,"lng":-74.00298648},{"lat":40.71772594,"lng":-74.0043354},{"lat":40.72132306,"lng":-74.0043354},{"lat":40.7217727,"lng":-74.00388576},{"lat":40.7217727,"lng":-74.00298648},{"lat":40.72087342,"lng":-74.0020872},{"lat":40.72132306,"lng":-74.00163756},{"lat":40.72671874,"lng":-74.00163756},{"lat":40.72761802,"lng":-74.00253684},{"lat":40.72941658,"lng":-74.00253684},{"lat":40.73031586,"lng":-74.00343612},{"lat":40.7330137,"lng":-74.00343612},{"lat":40.73481226,"lng":-74.00163756},{"lat":40.7375101,"lng":-74.00163756},{"lat":40.73795974,"lng":-74.0020872},{"lat":40.73795974,"lng":-74.00298648},{"lat":40.73706046,"lng":-74.00388576},{"lat":40.73840938,"lng":-74.00523468},{"lat":40.73930866,"lng":-74.0043354}],"holes":[]},{"shell":[{"lat":40.74155686,"lng":-73.99129584},{"lat":40.74155686,"lng":-73.99219512},{"lat":40.740657580000004,"lng":-73.9930944},{"lat":40.7420065,"lng":-73.99444332},{"lat":40.745153980000005,"lng":-73.99129584},{"lat":40.74470434,"lng":-73.99084620000001},{"lat":40.7420065,"lng":-73.99084620000001},{"lat":40.74155686,"lng":-73.99129584}],"holes":[]}],"properties":{}},{"search_id":"bill_20_min","shapes":[{"shell":[{"lat":40.70726892,"lng":-74.01587852},{"lat":40.707718559999996,"lng":-74.01632816},{"lat":40.70906748,"lng":-74.01497924},{"lat":40.70906748,"lng":-74.01407996},{"lat":40.70951712,"lng":-74.01363032},{"lat":40.71221496,"lng":-74.01363032},{"lat":40.7126646,"lng":-74.01318068},{"lat":40.7126646,"lng":-74.01228139999999},{"lat":40.71311424,"lng":-74.01183175999999},{"lat":40.71401352,"lng":-74.01183175999999},{"lat":40.71626172,"lng":-74.00958356},{"lat":40.71626172,"lng":-74.00868428},{"lat":40.71671136,"lng":-74.00823464},{"lat":40.7194092,"lng":-74.00823464},{"lat":40.72030848,"lng":-74.00913392},{"lat":40.72075812,"lng":-74.00868428},{"lat":40.72075812,"lng":-74.007785},{"lat":40.72120776,"lng":-74.00733536},{"lat":40.72210704,"lng":-74.00733536},{"lat":40.72300632,"lng":-74.00643608},{"lat":40.72480488,"lng":-74.00643608},{"lat":40.72570416,"lng":-74.00733536},{"lat":40.72660344,"lng":-74.00643608},{"lat":40.72930128,"lng":-74.00643608},{"lat":40.72975092,"lng":-74.00688572},{"lat":40.72975092,"lng":-74.007785},{"lat":40.7328984,"lng":-74.01093248},{"lat":40.73334804,"lng":-74.01048284},{"lat":40.73334804,"lng":-74.00958356},{"lat":40.73379768,"lng":-74.00913392},{"lat":40.73469696,"lng":-74.00913392},{"lat":40.7351466,"lng":-74.00868428},{"lat":40.7351466,"lng":-74.007785},{"lat":40.73559624,"lng":-74.00733536},{"lat":40.73604588,"lng":-74.007785},{"lat":40.73604588,"lng":-74.00868428},{"lat":40.73649552,"lng":-74.00913392},{"lat":40.73829408,"lng":-74.00913392},{"lat":40.73919336,"lng":-74.0100332},{"lat":40.74009264,"lng":-74.00913392},{"lat":40.7418912,"lng":-74.00913392},{"lat":40.7441394,"lng":-74.00688572},{"lat":40.74324012,"lng":-74.00598644},{"lat":40.74324012,"lng":-74.00418788},{"lat":40.74503868,"lng":-74.00238931999999},{"lat":40.74503868,"lng":-74.00149004},{"lat":40.74548832,"lng":-74.0010404},{"lat":40.7463876,"lng":-74.0010404},{"lat":40.74728688,"lng":-74.00193967999999},{"lat":40.747736520000004,"lng":-74.00149004},{"lat":40.747736520000004,"lng":-73.99969148},{"lat":40.7486358,"lng":-73.9987922},{"lat":40.7486358,"lng":-73.99699364},{"lat":40.747736520000004,"lng":-73.99609436},{"lat":40.747736520000004,"lng":-73.9942958},{"lat":40.74908544,"lng":-73.99294688},{"lat":40.74998472,"lng":-73.99294688},{"lat":40.75043436,"lng":-73.99339652},{"lat":40.75043436,"lng":-73.9942958},{"lat":40.75178328,"lng":-73.99564472},{"lat":40.75403148,"lng":-73.99339652},{"lat":40.752682560000004,"lng":-73.99204759999999},{"lat":40.75178328,"lng":-73.99204759999999},{"lat":40.75133364,"lng":-73.99159796},{"lat":40.75133364,"lng":-73.99069868},{"lat":40.75223292,"lng":-73.9897994},{"lat":40.75223292,"lng":-73.98800084},{"lat":40.7531322,"lng":-73.98710156},{"lat":40.7531322,"lng":-73.98620228},{"lat":40.75358184,"lng":-73.98575264},{"lat":40.75403148,"lng":-73.98620228},{"lat":40.75403148,"lng":-73.98710156},{"lat":40.75448112,"lng":-73.9875512},{"lat":40.75583004,"lng":-73.98620228},{"lat":40.75493076,"lng":-73.985303},{"lat":40.75493076,"lng":-73.98440372},{"lat":40.7553804,"lng":-73.98395408},{"lat":40.75627968,"lng":-73.98395408},{"lat":40.75672932,"lng":-73.98350444},{"lat":40.7553804,"lng":-73.98215551999999},{"lat":40.75358184,"lng":-73.98215551999999},{"lat":40.75223292,"lng":-73.98350444},{"lat":40.75223292,"lng":-73.985303},{"lat":40.75178328,"lng":-73.98575264},{"lat":40.74998472,"lng":-73.98575264},{"lat":40.7486358,"lng":-73.98440372},{"lat":40.7486358,"lng":-73.98260515999999},{"lat":40.74953508,"lng":-73.98170587999999},{"lat":40.74953508,"lng":-73.9808066},{"lat":40.74998472,"lng":-73.98035696},{"lat":40.750884,"lng":-73.98035696},{"lat":40.75133364,"lng":-73.97990732},{"lat":40.75133364,"lng":-73.97810876},{"lat":40.752682560000004,"lng":-73.97675984},{"lat":40.75448112,"lng":-73.97675984},{"lat":40.75583004,"lng":-73.97541092},{"lat":40.75448112,"lng":-73.974062},{"lat":40.752682560000004,"lng":-73.974062},{"lat":40.75178328,"lng":-73.97316272},{"lat":40.74998472,"lng":-73.97316272},{"lat":40.74953508,"lng":-73.97361236},{"lat":40.75043436,"lng":-73.97451164},{"lat":40.75043436,"lng":-73.9763102},{"lat":40.74953508,"lng":-73.97720948},{"lat":40.74953508,"lng":-73.97810876},{"lat":40.74908544,"lng":-73.9785584},{"lat":40.74728688,"lng":-73.9785584},{"lat":40.74683724,"lng":-73.97810876},{"lat":40.74683724,"lng":-73.97720948},{"lat":40.7463876,"lng":-73.97675984},{"lat":40.74548832,"lng":-73.97765912},{"lat":40.74368976,"lng":-73.97765912},{"lat":40.74324012,"lng":-73.97810876},{"lat":40.74324012,"lng":-73.97900804},{"lat":40.742790480000004,"lng":-73.97945768},{"lat":40.7418912,"lng":-73.97945768},{"lat":40.74099192,"lng":-73.98035696},{"lat":40.73919336,"lng":-73.98035696},{"lat":40.73874372,"lng":-73.97990732},{"lat":40.73874372,"lng":-73.97900804},{"lat":40.73829408,"lng":-73.9785584},{"lat":40.73649552,"lng":-73.9785584},{"lat":40.73379768,"lng":-73.97586056},{"lat":40.73199912,"lng":-73.97586056},{"lat":40.73154948,"lng":-73.97541092},{"lat":40.73154948,"lng":-73.97451164},{"lat":40.73109984,"lng":-73.974062},{"lat":40.73020056,"lng":-73.974062},{"lat":40.72930128,"lng":-73.97496128},{"lat":40.72750272,"lng":-73.97496128},{"lat":40.72705308,"lng":-73.97541092},{"lat":40.72705308,"lng":-73.97900804},{"lat":40.72795236,"lng":-73.97990732},{"lat":40.72750272,"lng":-73.98035696},{"lat":40.72480488,"lng":-73.98035696},{"lat":40.72435524,"lng":-73.9808066},{"lat":40.72435524,"lng":-73.98170587999999},{"lat":40.7239056,"lng":-73.98215551999999},{"lat":40.72300632,"lng":-73.98215551999999},{"lat":40.72255668,"lng":-73.98260515999999},{"lat":40.72255668,"lng":-73.98350444},{"lat":40.72210704,"lng":-73.98395408},{"lat":40.72120776,"lng":-73.98395408},{"lat":40.72075812,"lng":-73.98440372},{"lat":40.72075812,"lng":-73.985303},{"lat":40.72030848,"lng":-73.98575264},{"lat":40.71671136,"lng":-73.98575264},{"lat":40.71626172,"lng":-73.98620228},{"lat":40.71626172,"lng":-73.98800084},{"lat":40.71581208,"lng":-73.98845048},{"lat":40.7149128,"lng":-73.9875512},{"lat":40.71401352,"lng":-73.9875512},{"lat":40.71356388,"lng":-73.98800084},{"lat":40.71356388,"lng":-73.99069868},{"lat":40.7126646,"lng":-73.99159796},{"lat":40.7126646,"lng":-73.99519508},{"lat":40.71356388,"lng":-73.99609436},{"lat":40.7126646,"lng":-73.99699364},{"lat":40.7126646,"lng":-73.99789292},{"lat":40.71311424,"lng":-73.99834256},{"lat":40.71401352,"lng":-73.99834256},{"lat":40.71446316,"lng":-73.9987922},{"lat":40.71311424,"lng":-74.00014112},{"lat":40.71221496,"lng":-74.00014112},{"lat":40.71176532,"lng":-74.00059076},{"lat":40.71176532,"lng":-74.00149004},{"lat":40.7126646,"lng":-74.00238931999999},{"lat":40.71131568,"lng":-74.00373824},{"lat":40.7104164,"lng":-74.00373824},{"lat":40.70996676,"lng":-74.00418788},{"lat":40.70996676,"lng":-74.00508716},{"lat":40.70906748,"lng":-74.00598644},{"lat":40.70906748,"lng":-74.007785},{"lat":40.7081682,"lng":-74.00868428},{"lat":40.7081682,"lng":-74.01048284},{"lat":40.707718559999996,"lng":-74.01093248},{"lat":40.70592,"lng":-74.01093248},{"lat":40.70412144,"lng":-74.01273104},{"lat":40.70322216,"lng":-74.01273104},{"lat":40.70232288,"lng":-74.01183175999999},{"lat":40.70187324,"lng":-74.01228139999999},{"lat":40.70322216,"lng":-74.01363032},{"lat":40.70412144,"lng":-74.01363032},{"lat":40.70457108,"lng":-74.01407996},{"lat":40.70457108,"lng":-74.01497924},{"lat":40.70502072,"lng":-74.01542888},{"lat":40.70681928,"lng":-74.01542888},{"lat":40.70726892,"lng":-74.01587852}],"holes":[]},{"shell":[{"lat":40.75583004,"lng":-73.97181379999999},{"lat":40.75583004,"lng":-73.97271308},{"lat":40.75717896,"lng":-73.974062},{"lat":40.75852788,"lng":-73.97271308},{"lat":40.75627968,"lng":-73.97046488},{"lat":40.75583004,"lng":-73.97091452},{"lat":40.75583004,"lng":-73.97181379999999}],"holes":[]},{"shell":[{"lat":40.76122572,"lng":-73.96821668},{"lat":40.76122572,"lng":-73.96911596},{"lat":40.762574640000004,"lng":-73.97046488},{"lat":40.76392356,"lng":-73.96911596},{"lat":40.76392356,"lng":-73.9673174},{"lat":40.76167536,"lng":-73.9650692},{"lat":40.76122572,"lng":-73.96551884},{"lat":40.76122572,"lng":-73.96821668}],"holes":[]},{"shell":[{"lat":40.71761064,"lng":-73.95787496},{"lat":40.71895956,"lng":-73.95652604},{"lat":40.71895956,"lng":-73.95472748},{"lat":40.71850992,"lng":-73.95427784},{"lat":40.71671136,"lng":-73.95427784},{"lat":40.71581208,"lng":-73.95337856},{"lat":40.71536244,"lng":-73.9538282},{"lat":40.71536244,"lng":-73.95742532},{"lat":40.71671136,"lng":-73.95877424},{"lat":40.71761064,"lng":-73.95787496}],"holes":[]},{"shell":[{"lat":40.71401352,"lng":-73.95068072},{"lat":40.71536244,"lng":-73.9493318},{"lat":40.7149128,"lng":-73.94888216},{"lat":40.71311424,"lng":-73.94888216},{"lat":40.7126646,"lng":-73.9493318},{"lat":40.71401352,"lng":-73.95068072}],"holes":[]},{"shell":[{"lat":40.71086604,"lng":-73.940339},{"lat":40.71221496,"lng":-73.94168791999999},{"lat":40.7126646,"lng":-73.94123828},{"lat":40.7126646,"lng":-73.93943972},{"lat":40.71221496,"lng":-73.93899008},{"lat":40.71086604,"lng":-73.940339}],"holes":[]},{"shell":[{"lat":40.75942716,"lng":-73.98170587999999},{"lat":40.75942716,"lng":-73.9808066},{"lat":40.75897752,"lng":-73.98035696},{"lat":40.757628600000004,"lng":-73.98170587999999},{"lat":40.75897752,"lng":-73.9830548},{"lat":40.75942716,"lng":-73.98260515999999},{"lat":40.75942716,"lng":-73.98170587999999}],"holes":[]},{"shell":[{"lat":40.75852788,"lng":-73.98890012},{"lat":40.75807824,"lng":-73.98845048},{"lat":40.75672932,"lng":-73.9897994},{"lat":40.75807824,"lng":-73.99114832},{"lat":40.75852788,"lng":-73.99069868},{"lat":40.75852788,"lng":-73.98890012}],"holes":[]}],"properties":{}},{"search_id":"bill_25_min","shapes":[{"shell":[{"lat":40.70908038,"lng":-74.02086276},{"lat":40.713127140000005,"lng":-74.016816},{"lat":40.713127140000005,"lng":-74.01591672},{"lat":40.713576780000004,"lng":-74.01546708},{"lat":40.71447606,"lng":-74.01546708},{"lat":40.71627462,"lng":-74.01726564},{"lat":40.71672426,"lng":-74.016816},{"lat":40.71672426,"lng":-74.01501744},{"lat":40.7171739,"lng":-74.0145678},{"lat":40.718073180000005,"lng":-74.0145678},{"lat":40.718522820000004,"lng":-74.01411816},{"lat":40.718522820000004,"lng":-74.0123196},{"lat":40.71897246,"lng":-74.01186996},{"lat":40.71987174,"lng":-74.01186996},{"lat":40.72077102,"lng":-74.01276924},{"lat":40.7216703,"lng":-74.01186996},{"lat":40.72436814,"lng":-74.01186996},{"lat":40.72526742,"lng":-74.01097068},{"lat":40.72706598,"lng":-74.01097068},{"lat":40.72796526,"lng":-74.0100714},{"lat":40.728414900000004,"lng":-74.01052104},{"lat":40.728414900000004,"lng":-74.01142032},{"lat":40.728864540000004,"lng":-74.01186996},{"lat":40.72976382,"lng":-74.01097068},{"lat":40.7306631,"lng":-74.01097068},{"lat":40.73111274,"lng":-74.01142032},{"lat":40.73111274,"lng":-74.0123196},{"lat":40.733360940000004,"lng":-74.0145678},{"lat":40.733810580000004,"lng":-74.01411816},{"lat":40.733810580000004,"lng":-74.01321888},{"lat":40.73426022,"lng":-74.01276924},{"lat":40.7351595,"lng":-74.01276924},{"lat":40.73605878,"lng":-74.01186996},{"lat":40.73695806,"lng":-74.01186996},{"lat":40.7374077,"lng":-74.0123196},{"lat":40.7374077,"lng":-74.01321888},{"lat":40.73785734,"lng":-74.01366852},{"lat":40.73920626,"lng":-74.0123196},{"lat":40.73920626,"lng":-74.01142032},{"lat":40.7396559,"lng":-74.01097068},{"lat":40.74235374,"lng":-74.01097068},{"lat":40.743253020000004,"lng":-74.0100714},{"lat":40.74505158,"lng":-74.0100714},{"lat":40.74685014,"lng":-74.00827284},{"lat":40.748648700000004,"lng":-74.00827284},{"lat":40.74909834,"lng":-74.0078232},{"lat":40.74774942,"lng":-74.00647428},{"lat":40.74685014,"lng":-74.00647428},{"lat":40.7464005,"lng":-74.00602464},{"lat":40.74685014,"lng":-74.005575},{"lat":40.74774942,"lng":-74.005575},{"lat":40.74909834,"lng":-74.00422608},{"lat":40.74909834,"lng":-74.0033268},{"lat":40.74954798,"lng":-74.00287716},{"lat":40.75044726,"lng":-74.00287716},{"lat":40.7508969,"lng":-74.00242752},{"lat":40.7508969,"lng":-73.9988304},{"lat":40.75134654,"lng":-73.99838076},{"lat":40.7531451,"lng":-73.99838076},{"lat":40.75404438,"lng":-73.99928004},{"lat":40.75449402,"lng":-73.9988304},{"lat":40.75449402,"lng":-73.99613256},{"lat":40.7553933,"lng":-73.99523328},{"lat":40.7553933,"lng":-73.99343472},{"lat":40.75584294,"lng":-73.99298508},{"lat":40.7576415,"lng":-73.99298508},{"lat":40.75809114,"lng":-73.99343472},{"lat":40.75809114,"lng":-73.994334},{"lat":40.758540780000004,"lng":-73.99478364},{"lat":40.75944006,"lng":-73.99478364},{"lat":40.7598897,"lng":-73.994334},{"lat":40.7598897,"lng":-73.99343472},{"lat":40.76078898,"lng":-73.99253544},{"lat":40.7598897,"lng":-73.99163616},{"lat":40.7598897,"lng":-73.9898376},{"lat":40.76123862,"lng":-73.98848868},{"lat":40.76303718,"lng":-73.98848868},{"lat":40.7643861,"lng":-73.98713976},{"lat":40.7643861,"lng":-73.9853412},{"lat":40.76528538,"lng":-73.98444192},{"lat":40.76528538,"lng":-73.98354264},{"lat":40.76573502,"lng":-73.983093},{"lat":40.768432860000004,"lng":-73.983093},{"lat":40.76933214,"lng":-73.98399228},{"lat":40.76978178,"lng":-73.98354264},{"lat":40.76978178,"lng":-73.98174408},{"lat":40.77068106,"lng":-73.9808448},{"lat":40.76933214,"lng":-73.97949588},{"lat":40.76753358,"lng":-73.97949588},{"lat":40.76708394,"lng":-73.97994552},{"lat":40.76708394,"lng":-73.9808448},{"lat":40.7666343,"lng":-73.98129444},{"lat":40.76618466,"lng":-73.9808448},{"lat":40.76618466,"lng":-73.97994552},{"lat":40.76528538,"lng":-73.97904624},{"lat":40.76528538,"lng":-73.97724768},{"lat":40.763936460000004,"lng":-73.97589876},{"lat":40.76303718,"lng":-73.97679804},{"lat":40.76033934,"lng":-73.97679804},{"lat":40.7598897,"lng":-73.9763484},{"lat":40.7598897,"lng":-73.97544912000001},{"lat":40.76078898,"lng":-73.97454984},{"lat":40.76078898,"lng":-73.97365056},{"lat":40.76123862,"lng":-73.97320092},{"lat":40.7621379,"lng":-73.97320092},{"lat":40.763936460000004,"lng":-73.97499948},{"lat":40.76618466,"lng":-73.97275128},{"lat":40.76528538,"lng":-73.971852},{"lat":40.76528538,"lng":-73.97005344},{"lat":40.76618466,"lng":-73.96915416},{"lat":40.76618466,"lng":-73.9673556},{"lat":40.7666343,"lng":-73.96690596},{"lat":40.768432860000004,"lng":-73.96690596},{"lat":40.76978178,"lng":-73.96555704000001},{"lat":40.76978178,"lng":-73.96195992},{"lat":40.76933214,"lng":-73.96151028},{"lat":40.76753358,"lng":-73.96151028},{"lat":40.7666343,"lng":-73.960611},{"lat":40.76618466,"lng":-73.96106064},{"lat":40.76618466,"lng":-73.96375848},{"lat":40.76573502,"lng":-73.96420812},{"lat":40.76483574,"lng":-73.96420812},{"lat":40.7621379,"lng":-73.96151028},{"lat":40.76033934,"lng":-73.96151028},{"lat":40.7598897,"lng":-73.96195992},{"lat":40.7598897,"lng":-73.96375848},{"lat":40.75809114,"lng":-73.96555704000001},{"lat":40.75809114,"lng":-73.96645632},{"lat":40.7576415,"lng":-73.96690596},{"lat":40.75494366,"lng":-73.96690596},{"lat":40.753594740000004,"lng":-73.96825488},{"lat":40.75449402,"lng":-73.96915416},{"lat":40.75449402,"lng":-73.97005344},{"lat":40.75404438,"lng":-73.97050308},{"lat":40.7531451,"lng":-73.97050308},{"lat":40.75224582,"lng":-73.9696038},{"lat":40.75044726,"lng":-73.9696038},{"lat":40.74954798,"lng":-73.96870452},{"lat":40.74774942,"lng":-73.96870452},{"lat":40.74729978,"lng":-73.96915416},{"lat":40.74729978,"lng":-73.97095272},{"lat":40.748199060000005,"lng":-73.971852},{"lat":40.748199060000005,"lng":-73.97454984},{"lat":40.74774942,"lng":-73.97499948},{"lat":40.74685014,"lng":-73.97499948},{"lat":40.74595086,"lng":-73.9741002},{"lat":40.74505158,"lng":-73.9741002},{"lat":40.74460194,"lng":-73.97365056},{"lat":40.74460194,"lng":-73.97275128},{"lat":40.7441523,"lng":-73.97230164},{"lat":40.74145446,"lng":-73.97499948},{"lat":40.73695806,"lng":-73.97499948},{"lat":40.733360940000004,"lng":-73.97140236},{"lat":40.73156238,"lng":-73.97140236},{"lat":40.7306631,"lng":-73.97050308},{"lat":40.728864540000004,"lng":-73.97050308},{"lat":40.72796526,"lng":-73.97140236},{"lat":40.7261667,"lng":-73.97140236},{"lat":40.7239185,"lng":-73.97365056},{"lat":40.7239185,"lng":-73.97724768},{"lat":40.7216703,"lng":-73.97949588},{"lat":40.72122066,"lng":-73.97904624},{"lat":40.72122066,"lng":-73.97814696},{"lat":40.72077102,"lng":-73.97769732},{"lat":40.71897246,"lng":-73.97769732},{"lat":40.718522820000004,"lng":-73.97814696},{"lat":40.718522820000004,"lng":-73.97904624},{"lat":40.7194221,"lng":-73.97994552},{"lat":40.7171739,"lng":-73.98219372},{"lat":40.71672426,"lng":-73.98174408},{"lat":40.71672426,"lng":-73.9808448},{"lat":40.71627462,"lng":-73.98039516},{"lat":40.71582498,"lng":-73.9808448},{"lat":40.71582498,"lng":-73.98174408},{"lat":40.71537534,"lng":-73.98219372},{"lat":40.71447606,"lng":-73.98219372},{"lat":40.71402642,"lng":-73.98264336},{"lat":40.71402642,"lng":-73.98354264},{"lat":40.708630740000004,"lng":-73.98893832},{"lat":40.708630740000004,"lng":-73.99253544},{"lat":40.70953002,"lng":-73.99343472},{"lat":40.70953002,"lng":-73.99523328},{"lat":40.7104293,"lng":-73.99613256},{"lat":40.70953002,"lng":-73.99703184},{"lat":40.70953002,"lng":-73.9988304},{"lat":40.70908038,"lng":-73.99928004},{"lat":40.708181100000004,"lng":-73.99838076},{"lat":40.70728182,"lng":-73.99838076},{"lat":40.70683218,"lng":-73.9988304},{"lat":40.70683218,"lng":-74.00152824},{"lat":40.70773146,"lng":-74.00242752},{"lat":40.70548326,"lng":-74.00467572},{"lat":40.70458398,"lng":-74.00467572},{"lat":40.70413434,"lng":-74.00512536},{"lat":40.70413434,"lng":-74.00602464},{"lat":40.70503362,"lng":-74.00692392},{"lat":40.70503362,"lng":-74.0078232},{"lat":40.70458398,"lng":-74.00827284},{"lat":40.70188614,"lng":-74.00827284},{"lat":40.70008758,"lng":-74.0100714},{"lat":40.6991883,"lng":-74.0100714},{"lat":40.698738660000004,"lng":-74.01052104},{"lat":40.698738660000004,"lng":-74.01411816},{"lat":40.70098686,"lng":-74.01636636},{"lat":40.70188614,"lng":-74.01636636},{"lat":40.70233578,"lng":-74.016816},{"lat":40.70233578,"lng":-74.01771528},{"lat":40.703684700000004,"lng":-74.0190642},{"lat":40.70728182,"lng":-74.0190642},{"lat":40.70908038,"lng":-74.02086276}],"holes":[]},{"shell":[{"lat":40.69064514,"lng":-73.98803904},{"lat":40.69199406,"lng":-73.98938796},{"lat":40.69424226,"lng":-73.98713976},{"lat":40.69199406,"lng":-73.98489156},{"lat":40.69064514,"lng":-73.98624048},{"lat":40.69064514,"lng":-73.98803904}],"holes":[]},{"shell":[{"lat":40.70008758,"lng":-73.98399228},{"lat":40.698738660000004,"lng":-73.9853412},{"lat":40.698738660000004,"lng":-73.98713976},{"lat":40.69963794,"lng":-73.98803904},{"lat":40.69963794,"lng":-73.98893832},{"lat":40.70098686,"lng":-73.99028724},{"lat":40.70278542,"lng":-73.99028724},{"lat":40.70413434,"lng":-73.98893832},{"lat":40.70413434,"lng":-73.98624048},{"lat":40.703684700000004,"lng":-73.98579084},{"lat":40.70278542,"lng":-73.98579084},{"lat":40.70233578,"lng":-73.9853412},{"lat":40.70233578,"lng":-73.98354264},{"lat":40.70188614,"lng":-73.983093},{"lat":40.70098686,"lng":-73.983093},{"lat":40.70008758,"lng":-73.98399228}],"holes":[]},{"shell":[{"lat":40.68165234,"lng":-73.9808448},{"lat":40.68210198,"lng":-73.98129444},{"lat":40.683001260000005,"lng":-73.98039516},{"lat":40.68479982,"lng":-73.98039516},{"lat":40.68524946,"lng":-73.97994552},{"lat":40.68524946,"lng":-73.97814696},{"lat":40.683001260000005,"lng":-73.97589876},{"lat":40.68255162,"lng":-73.9763484},{"lat":40.68255162,"lng":-73.97814696},{"lat":40.68165234,"lng":-73.97904624},{"lat":40.68165234,"lng":-73.9808448}],"holes":[]},{"shell":[{"lat":40.71402642,"lng":-73.96195992},{"lat":40.71447606,"lng":-73.96240956},{"lat":40.71627462,"lng":-73.96240956},{"lat":40.7171739,"lng":-73.96330884},{"lat":40.71987174,"lng":-73.96330884},{"lat":40.72032138,"lng":-73.9628592},{"lat":40.72032138,"lng":-73.96195992},{"lat":40.72122066,"lng":-73.96106064},{"lat":40.72122066,"lng":-73.95926208},{"lat":40.72211994,"lng":-73.9583628},{"lat":40.72211994,"lng":-73.95656424},{"lat":40.72122066,"lng":-73.95566496000001},{"lat":40.72211994,"lng":-73.95476568},{"lat":40.72211994,"lng":-73.95296712},{"lat":40.72122066,"lng":-73.95206784},{"lat":40.72122066,"lng":-73.95116856},{"lat":40.72077102,"lng":-73.95071892},{"lat":40.71987174,"lng":-73.95071892},{"lat":40.71897246,"lng":-73.9516182},{"lat":40.718522820000004,"lng":-73.95116856},{"lat":40.718522820000004,"lng":-73.95026928},{"lat":40.71672426,"lng":-73.94847072},{"lat":40.71672426,"lng":-73.94577288},{"lat":40.7171739,"lng":-73.94532324000001},{"lat":40.718073180000005,"lng":-73.94532324000001},{"lat":40.718522820000004,"lng":-73.9448736},{"lat":40.71762354,"lng":-73.94397432},{"lat":40.71762354,"lng":-73.94217576},{"lat":40.71582498,"lng":-73.9403772},{"lat":40.71582498,"lng":-73.93857864},{"lat":40.713576780000004,"lng":-73.93633044},{"lat":40.7126775,"lng":-73.93722972},{"lat":40.71087894,"lng":-73.93722972},{"lat":40.70997966,"lng":-73.93633044},{"lat":40.70953002,"lng":-73.93678008},{"lat":40.70953002,"lng":-73.93767936},{"lat":40.70908038,"lng":-73.938129},{"lat":40.708630740000004,"lng":-73.93767936},{"lat":40.708630740000004,"lng":-73.93498152},{"lat":40.70638254,"lng":-73.93273332},{"lat":40.7059329,"lng":-73.93318296},{"lat":40.70683218,"lng":-73.93408224},{"lat":40.70683218,"lng":-73.93678008},{"lat":40.70638254,"lng":-73.93722972},{"lat":40.70548326,"lng":-73.93722972},{"lat":40.70503362,"lng":-73.93767936},{"lat":40.70503362,"lng":-73.94127648},{"lat":40.70548326,"lng":-73.94172612},{"lat":40.70638254,"lng":-73.94172612},{"lat":40.70683218,"lng":-73.94217576},{"lat":40.70683218,"lng":-73.94307504},{"lat":40.70728182,"lng":-73.94352468},{"lat":40.708181100000004,"lng":-73.9426254},{"lat":40.70908038,"lng":-73.9426254},{"lat":40.70953002,"lng":-73.94307504},{"lat":40.70953002,"lng":-73.94397432},{"lat":40.70997966,"lng":-73.94442396},{"lat":40.71087894,"lng":-73.94442396},{"lat":40.71132858,"lng":-73.9448736},{"lat":40.71132858,"lng":-73.94757144},{"lat":40.70953002,"lng":-73.94937},{"lat":40.70997966,"lng":-73.94981964},{"lat":40.71087894,"lng":-73.94981964},{"lat":40.71132858,"lng":-73.95026928},{"lat":40.71132858,"lng":-73.95116856},{"lat":40.7104293,"lng":-73.95206784},{"lat":40.713127140000005,"lng":-73.95476568},{"lat":40.713127140000005,"lng":-73.96106064},{"lat":40.71402642,"lng":-73.96195992}],"holes":[]},{"shell":[{"lat":40.7868681,"lng":-73.95026928},{"lat":40.78551918,"lng":-73.94892036},{"lat":40.78417026,"lng":-73.95026928},{"lat":40.78417026,"lng":-73.95206784},{"lat":40.7846199,"lng":-73.95251748},{"lat":40.78641846,"lng":-73.95251748},{"lat":40.7868681,"lng":-73.95206784},{"lat":40.7868681,"lng":-73.95026928}],"holes":[]},{"shell":[{"lat":40.78147242,"lng":-73.9538664},{"lat":40.7801235,"lng":-73.95251748},{"lat":40.77922422,"lng":-73.95341676},{"lat":40.77742566,"lng":-73.95341676},{"lat":40.77697602,"lng":-73.9538664},{"lat":40.7778753,"lng":-73.95476568},{"lat":40.7778753,"lng":-73.95656424},{"lat":40.77697602,"lng":-73.95746352},{"lat":40.77742566,"lng":-73.95791316},{"lat":40.77922422,"lng":-73.95791316},{"lat":40.7801235,"lng":-73.95881244},{"lat":40.78147242,"lng":-73.95746352},{"lat":40.78147242,"lng":-73.95566496000001},{"lat":40.7823717,"lng":-73.95476568},{"lat":40.78147242,"lng":-73.9538664}],"holes":[]},{"shell":[{"lat":40.74280338,"lng":-73.95206784},{"lat":40.74280338,"lng":-73.94937},{"lat":40.74235374,"lng":-73.94892036},{"lat":40.7419041,"lng":-73.94937},{"lat":40.7419041,"lng":-73.95476568},{"lat":40.74235374,"lng":-73.95521532},{"lat":40.743253020000004,"lng":-73.95521532},{"lat":40.743702660000004,"lng":-73.95476568},{"lat":40.743702660000004,"lng":-73.95296712},{"lat":40.74280338,"lng":-73.95206784}],"holes":[]},{"shell":[{"lat":40.77292926,"lng":-73.95791316},{"lat":40.77247962,"lng":-73.9583628},{"lat":40.77247962,"lng":-73.96016136},{"lat":40.77472782,"lng":-73.96240956},{"lat":40.77517746,"lng":-73.96195992},{"lat":40.77427818,"lng":-73.96106064},{"lat":40.77427818,"lng":-73.95926208},{"lat":40.77517746,"lng":-73.9583628},{"lat":40.77472782,"lng":-73.95791316},{"lat":40.77292926,"lng":-73.95791316}],"holes":[]},{"shell":[{"lat":40.77427818,"lng":-73.9763484},{"lat":40.7756271,"lng":-73.97769732},{"lat":40.77697602,"lng":-73.9763484},{"lat":40.77652638,"lng":-73.97589876},{"lat":40.77472782,"lng":-73.97589876},{"lat":40.77427818,"lng":-73.9763484}],"holes":[]}],"properties":{}},{"search_id":"bill_30_min","shapes":[{"shell":[{"lat":40.71565116,"lng":-74.03364069999999},{"lat":40.717000080000005,"lng":-74.03498961999999},{"lat":40.717449720000005,"lng":-74.03453997999999},{"lat":40.717449720000005,"lng":-74.03274142},{"lat":40.718349,"lng":-74.03184214},{"lat":40.717899360000004,"lng":-74.0313925},{"lat":40.71610080000001,"lng":-74.0313925},{"lat":40.71475188,"lng":-74.03274142},{"lat":40.71565116,"lng":-74.03364069999999}],"holes":[]},{"shell":[{"lat":40.73453604,"lng":-74.03004358},{"lat":40.73453604,"lng":-74.03274142},{"lat":40.73588496,"lng":-74.03409033999999},{"lat":40.73903244,"lng":-74.03094286},{"lat":40.73903244,"lng":-74.0291443},{"lat":40.73993172,"lng":-74.02824502},{"lat":40.73588496,"lng":-74.02419825999999},{"lat":40.7340864,"lng":-74.02419825999999},{"lat":40.732737480000004,"lng":-74.02554717999999},{"lat":40.732737480000004,"lng":-74.0291443},{"lat":40.733187120000004,"lng":-74.02959394},{"lat":40.7340864,"lng":-74.02959394},{"lat":40.73453604,"lng":-74.03004358}],"holes":[]},{"shell":[{"lat":40.71070512000001,"lng":-74.02509753999999},{"lat":40.712953320000004,"lng":-74.02284934},{"lat":40.712953320000004,"lng":-74.0201515},{"lat":40.71340296,"lng":-74.01970186},{"lat":40.71430224,"lng":-74.01970186},{"lat":40.71520152,"lng":-74.02060114},{"lat":40.716550440000006,"lng":-74.01925222},{"lat":40.716550440000006,"lng":-74.01745365999999},{"lat":40.717000080000005,"lng":-74.01700401999999},{"lat":40.71879864,"lng":-74.01700401999999},{"lat":40.71924828,"lng":-74.01655437999999},{"lat":40.71924828,"lng":-74.01565509999999},{"lat":40.71969792,"lng":-74.01520545999999},{"lat":40.721496480000006,"lng":-74.01520545999999},{"lat":40.722395760000005,"lng":-74.01430617999999},{"lat":40.7250936,"lng":-74.01430617999999},{"lat":40.727791440000004,"lng":-74.01160834},{"lat":40.73048928,"lng":-74.01160834},{"lat":40.73093892000001,"lng":-74.01205798},{"lat":40.73093892000001,"lng":-74.01295726},{"lat":40.73048928,"lng":-74.01340689999999},{"lat":40.72869072,"lng":-74.01340689999999},{"lat":40.727341800000005,"lng":-74.01475581999999},{"lat":40.72869072,"lng":-74.01610473999999},{"lat":40.72959,"lng":-74.01520545999999},{"lat":40.73048928,"lng":-74.01520545999999},{"lat":40.73093892000001,"lng":-74.01565509999999},{"lat":40.73093892000001,"lng":-74.01835294},{"lat":40.732287840000005,"lng":-74.01970186},{"lat":40.7340864,"lng":-74.01970186},{"lat":40.737233880000005,"lng":-74.01655437999999},{"lat":40.737233880000005,"lng":-74.01475581999999},{"lat":40.7385828,"lng":-74.01340689999999},{"lat":40.74038136,"lng":-74.01340689999999},{"lat":40.74128064000001,"lng":-74.01430617999999},{"lat":40.742629560000005,"lng":-74.01295726},{"lat":40.742629560000005,"lng":-74.01205798},{"lat":40.743079200000004,"lng":-74.01160834},{"lat":40.74352884,"lng":-74.01205798},{"lat":40.74352884,"lng":-74.01295726},{"lat":40.74487776,"lng":-74.01430617999999},{"lat":40.746676320000006,"lng":-74.01250762},{"lat":40.74847488,"lng":-74.01250762},{"lat":40.752521640000005,"lng":-74.00846086},{"lat":40.751622360000006,"lng":-74.00756157999999},{"lat":40.751622360000006,"lng":-74.00666229999999},{"lat":40.752072000000005,"lng":-74.00621265999999},{"lat":40.756568400000006,"lng":-74.00621265999999},{"lat":40.7588166,"lng":-74.00396445999999},{"lat":40.757917320000004,"lng":-74.00306518},{"lat":40.757917320000004,"lng":-74.00126662},{"lat":40.7588166,"lng":-74.00036734},{"lat":40.758366960000004,"lng":-73.9999177},{"lat":40.757467680000005,"lng":-73.9999177},{"lat":40.757018040000005,"lng":-73.99946806},{"lat":40.757018040000005,"lng":-73.99856878},{"lat":40.757467680000005,"lng":-73.99811914},{"lat":40.75926624,"lng":-73.99811914},{"lat":40.76016552,"lng":-73.99901842},{"lat":40.761514440000006,"lng":-73.99766949999999},{"lat":40.761514440000006,"lng":-73.99497165999999},{"lat":40.762863360000004,"lng":-73.99362273999999},{"lat":40.76466192,"lng":-73.99362273999999},{"lat":40.76601084,"lng":-73.99227382},{"lat":40.76601084,"lng":-73.99047526},{"lat":40.76511156,"lng":-73.98957598},{"lat":40.7655612,"lng":-73.98912634},{"lat":40.767359760000005,"lng":-73.98912634},{"lat":40.767809400000004,"lng":-73.9886767},{"lat":40.767809400000004,"lng":-73.98777742},{"lat":40.768259040000004,"lng":-73.98732777999999},{"lat":40.7700576,"lng":-73.98732777999999},{"lat":40.77050724,"lng":-73.98687813999999},{"lat":40.77050724,"lng":-73.98597885999999},{"lat":40.77095688,"lng":-73.98552921999999},{"lat":40.771856160000006,"lng":-73.98552921999999},{"lat":40.772755440000005,"lng":-73.98462993999999},{"lat":40.774554,"lng":-73.98462993999999},{"lat":40.77500364,"lng":-73.98418029999999},{"lat":40.77500364,"lng":-73.98328101999999},{"lat":40.77545328,"lng":-73.98283138},{"lat":40.77635256,"lng":-73.98283138},{"lat":40.777251840000005,"lng":-73.98373065999999},{"lat":40.778151120000004,"lng":-73.98373065999999},{"lat":40.77860076,"lng":-73.98418029999999},{"lat":40.77860076,"lng":-73.98507957999999},{"lat":40.7790504,"lng":-73.98552921999999},{"lat":40.78039932,"lng":-73.98418029999999},{"lat":40.78039932,"lng":-73.98238174},{"lat":40.7812986,"lng":-73.98148246},{"lat":40.777701480000005,"lng":-73.97788534},{"lat":40.777701480000005,"lng":-73.97608677999999},{"lat":40.77994968,"lng":-73.97383857999999},{"lat":40.78084896,"lng":-73.97383857999999},{"lat":40.7812986,"lng":-73.97428821999999},{"lat":40.7812986,"lng":-73.97518749999999},{"lat":40.781748240000006,"lng":-73.97563713999999},{"lat":40.783097160000004,"lng":-73.97428821999999},{"lat":40.783097160000004,"lng":-73.97248966},{"lat":40.78444608,"lng":-73.97114074},{"lat":40.78624464,"lng":-73.97114074},{"lat":40.787143920000005,"lng":-73.97204002},{"lat":40.787593560000005,"lng":-73.97159038},{"lat":40.787593560000005,"lng":-73.96979182},{"lat":40.78849284,"lng":-73.96889254},{"lat":40.78849284,"lng":-73.96709397999999},{"lat":40.788043200000004,"lng":-73.96664433999999},{"lat":40.787143920000005,"lng":-73.96754362},{"lat":40.78534536,"lng":-73.96754362},{"lat":40.78444608,"lng":-73.96664433999999},{"lat":40.78399644,"lng":-73.96709397999999},{"lat":40.78489572,"lng":-73.96799326},{"lat":40.78489572,"lng":-73.96889254},{"lat":40.78444608,"lng":-73.96934218},{"lat":40.7835468,"lng":-73.96934218},{"lat":40.782647520000005,"lng":-73.9684429},{"lat":40.78084896,"lng":-73.9684429},{"lat":40.77994968,"lng":-73.96934218},{"lat":40.778151120000004,"lng":-73.96934218},{"lat":40.777701480000005,"lng":-73.96979182},{"lat":40.77860076,"lng":-73.9706911},{"lat":40.77860076,"lng":-73.97159038},{"lat":40.778151120000004,"lng":-73.97204002},{"lat":40.77365472,"lng":-73.97204002},{"lat":40.772305800000005,"lng":-73.97338893999999},{"lat":40.772305800000005,"lng":-73.97608677999999},{"lat":40.771856160000006,"lng":-73.97653641999999},{"lat":40.76915832,"lng":-73.97653641999999},{"lat":40.76870868,"lng":-73.97608677999999},{"lat":40.76870868,"lng":-73.97428821999999},{"lat":40.76960796,"lng":-73.97338893999999},{"lat":40.76870868,"lng":-73.97248966},{"lat":40.76870868,"lng":-73.9706911},{"lat":40.772305800000005,"lng":-73.96709397999999},{"lat":40.772305800000005,"lng":-73.96529541999999},{"lat":40.77365472,"lng":-73.96394649999999},{"lat":40.77545328,"lng":-73.96394649999999},{"lat":40.77635256,"lng":-73.96484577999999},{"lat":40.776802200000006,"lng":-73.96439613999999},{"lat":40.776802200000006,"lng":-73.9616983},{"lat":40.777251840000005,"lng":-73.96124866},{"lat":40.77994968,"lng":-73.96124866},{"lat":40.781748240000006,"lng":-73.9594501},{"lat":40.7835468,"lng":-73.9594501},{"lat":40.78399644,"lng":-73.95900046},{"lat":40.78399644,"lng":-73.95630261999999},{"lat":40.78444608,"lng":-73.95585297999999},{"lat":40.787143920000005,"lng":-73.95585297999999},{"lat":40.788043200000004,"lng":-73.95675225999999},{"lat":40.78849284,"lng":-73.95630261999999},{"lat":40.78849284,"lng":-73.95450405999999},{"lat":40.78939212,"lng":-73.95360477999999},{"lat":40.78939212,"lng":-73.95180622},{"lat":40.78984176,"lng":-73.95135658},{"lat":40.791640320000006,"lng":-73.95135658},{"lat":40.792089960000006,"lng":-73.95090694},{"lat":40.792089960000006,"lng":-73.94910838},{"lat":40.792989240000004,"lng":-73.9482091},{"lat":40.792989240000004,"lng":-73.94730982},{"lat":40.793438880000004,"lng":-73.94686017999999},{"lat":40.79433816,"lng":-73.94686017999999},{"lat":40.79523744,"lng":-73.94775946},{"lat":40.79658636,"lng":-73.94641053999999},{"lat":40.79658636,"lng":-73.94461197999999},{"lat":40.797036000000006,"lng":-73.94416233999999},{"lat":40.79883456,"lng":-73.94416233999999},{"lat":40.80018348,"lng":-73.94281342},{"lat":40.80018348,"lng":-73.94101486},{"lat":40.80108276,"lng":-73.94011558},{"lat":40.80063312,"lng":-73.93966594},{"lat":40.797036000000006,"lng":-73.93966594},{"lat":40.79658636,"lng":-73.94011558},{"lat":40.79658636,"lng":-73.94101486},{"lat":40.79613672,"lng":-73.9414645},{"lat":40.79523744,"lng":-73.9414645},{"lat":40.79433816,"lng":-73.94056522},{"lat":40.79388852,"lng":-73.94101486},{"lat":40.79388852,"lng":-73.94281342},{"lat":40.792989240000004,"lng":-73.94371269999999},{"lat":40.792989240000004,"lng":-73.94461197999999},{"lat":40.792539600000005,"lng":-73.94506161999999},{"lat":40.791640320000006,"lng":-73.94506161999999},{"lat":40.79074104,"lng":-73.94416233999999},{"lat":40.78894248,"lng":-73.94416233999999},{"lat":40.787593560000005,"lng":-73.94551125999999},{"lat":40.787593560000005,"lng":-73.94641053999999},{"lat":40.787143920000005,"lng":-73.94686017999999},{"lat":40.78624464,"lng":-73.94686017999999},{"lat":40.78444608,"lng":-73.94506161999999},{"lat":40.782197880000005,"lng":-73.94730982},{"lat":40.782197880000005,"lng":-73.95000766},{"lat":40.781748240000006,"lng":-73.9504573},{"lat":40.7812986,"lng":-73.95000766},{"lat":40.7812986,"lng":-73.94910838},{"lat":40.78084896,"lng":-73.94865874},{"lat":40.7790504,"lng":-73.94865874},{"lat":40.778151120000004,"lng":-73.94775946},{"lat":40.77635256,"lng":-73.94775946},{"lat":40.77500364,"lng":-73.94910838},{"lat":40.77590292,"lng":-73.95000766},{"lat":40.77590292,"lng":-73.95180622},{"lat":40.77500364,"lng":-73.9527055},{"lat":40.77500364,"lng":-73.95540333999999},{"lat":40.774554,"lng":-73.95585297999999},{"lat":40.77365472,"lng":-73.95585297999999},{"lat":40.772755440000005,"lng":-73.95495369999999},{"lat":40.77095688,"lng":-73.95495369999999},{"lat":40.76960796,"lng":-73.95630261999999},{"lat":40.77050724,"lng":-73.95720189999999},{"lat":40.77050724,"lng":-73.95900046},{"lat":40.7700576,"lng":-73.9594501},{"lat":40.768259040000004,"lng":-73.9594501},{"lat":40.767359760000005,"lng":-73.95855082},{"lat":40.7655612,"lng":-73.95855082},{"lat":40.76466192,"lng":-73.95765154},{"lat":40.76376264,"lng":-73.95855082},{"lat":40.761964080000006,"lng":-73.95855082},{"lat":40.76016552,"lng":-73.95675225999999},{"lat":40.757018040000005,"lng":-73.95989974},{"lat":40.757917320000004,"lng":-73.96079902},{"lat":40.757917320000004,"lng":-73.9616983},{"lat":40.757467680000005,"lng":-73.96214794},{"lat":40.756568400000006,"lng":-73.96214794},{"lat":40.75566912,"lng":-73.96124866},{"lat":40.75476984,"lng":-73.96214794},{"lat":40.752971280000004,"lng":-73.96214794},{"lat":40.752521640000005,"lng":-73.96259758},{"lat":40.752521640000005,"lng":-73.96529541999999},{"lat":40.752072000000005,"lng":-73.96574505999999},{"lat":40.75117272000001,"lng":-73.96574505999999},{"lat":40.74937416,"lng":-73.96394649999999},{"lat":40.74892452,"lng":-73.96439613999999},{"lat":40.74892452,"lng":-73.96799326},{"lat":40.747575600000005,"lng":-73.96934218},{"lat":40.74397848,"lng":-73.96934218},{"lat":40.743079200000004,"lng":-73.9684429},{"lat":40.742629560000005,"lng":-73.96889254},{"lat":40.742629560000005,"lng":-73.96979182},{"lat":40.742179920000005,"lng":-73.97024146},{"lat":40.73588496,"lng":-73.97024146},{"lat":40.73498568,"lng":-73.97114074},{"lat":40.7340864,"lng":-73.97114074},{"lat":40.73363676,"lng":-73.9706911},{"lat":40.73363676,"lng":-73.96979182},{"lat":40.731388560000006,"lng":-73.96754362},{"lat":40.73048928,"lng":-73.9684429},{"lat":40.72869072,"lng":-73.9684429},{"lat":40.727791440000004,"lng":-73.96754362},{"lat":40.726892160000006,"lng":-73.96754362},{"lat":40.726442520000006,"lng":-73.96799326},{"lat":40.726442520000006,"lng":-73.97159038},{"lat":40.72599288000001,"lng":-73.97204002},{"lat":40.7250936,"lng":-73.97114074},{"lat":40.72419432,"lng":-73.97114074},{"lat":40.723295040000004,"lng":-73.97024146},{"lat":40.722845400000004,"lng":-73.9706911},{"lat":40.722845400000004,"lng":-73.97338893999999},{"lat":40.722395760000005,"lng":-73.97383857999999},{"lat":40.7205972,"lng":-73.97383857999999},{"lat":40.71969792,"lng":-73.97473785999999},{"lat":40.71610080000001,"lng":-73.97473785999999},{"lat":40.71340296,"lng":-73.97743569999999},{"lat":40.712503680000005,"lng":-73.97743569999999},{"lat":40.712054040000005,"lng":-73.97788534},{"lat":40.712054040000005,"lng":-73.97878462},{"lat":40.71115476000001,"lng":-73.9796839},{"lat":40.71115476000001,"lng":-73.98148246},{"lat":40.7093562,"lng":-73.98328101999999},{"lat":40.7093562,"lng":-73.98507957999999},{"lat":40.70890656,"lng":-73.98552921999999},{"lat":40.707108000000005,"lng":-73.98552921999999},{"lat":40.706658360000006,"lng":-73.98507957999999},{"lat":40.706658360000006,"lng":-73.98418029999999},{"lat":40.70620872000001,"lng":-73.98373065999999},{"lat":40.70530944,"lng":-73.98373065999999},{"lat":40.7048598,"lng":-73.98328101999999},{"lat":40.7048598,"lng":-73.98148246},{"lat":40.70351088,"lng":-73.98013354},{"lat":40.701712320000006,"lng":-73.98013354},{"lat":40.70081304000001,"lng":-73.97923426},{"lat":40.69901448,"lng":-73.97923426},{"lat":40.697665560000004,"lng":-73.98058318},{"lat":40.697665560000004,"lng":-73.98238174},{"lat":40.69541736,"lng":-73.98462993999999},{"lat":40.69406844,"lng":-73.98328101999999},{"lat":40.69406844,"lng":-73.98238174},{"lat":40.6936188,"lng":-73.9819321},{"lat":40.692719520000004,"lng":-73.9819321},{"lat":40.692269880000005,"lng":-73.98148246},{"lat":40.692269880000005,"lng":-73.98058318},{"lat":40.691820240000006,"lng":-73.98013354},{"lat":40.6891224,"lng":-73.98013354},{"lat":40.68867276,"lng":-73.9796839},{"lat":40.68867276,"lng":-73.97428821999999},{"lat":40.68822312,"lng":-73.97383857999999},{"lat":40.684626,"lng":-73.97383857999999},{"lat":40.68417636,"lng":-73.97338893999999},{"lat":40.68417636,"lng":-73.97159038},{"lat":40.68372672,"lng":-73.97114074},{"lat":40.682827440000004,"lng":-73.97114074},{"lat":40.682377800000005,"lng":-73.97159038},{"lat":40.682377800000005,"lng":-73.97338893999999},{"lat":40.681928160000005,"lng":-73.97383857999999},{"lat":40.6801296,"lng":-73.97383857999999},{"lat":40.67878068,"lng":-73.97518749999999},{"lat":40.67967996,"lng":-73.97608677999999},{"lat":40.67967996,"lng":-73.97878462},{"lat":40.677881400000004,"lng":-73.98058318},{"lat":40.677881400000004,"lng":-73.98148246},{"lat":40.67833104,"lng":-73.9819321},{"lat":40.67923032,"lng":-73.9819321},{"lat":40.682827440000004,"lng":-73.98552921999999},{"lat":40.684626,"lng":-73.98552921999999},{"lat":40.68597492000001,"lng":-73.98687813999999},{"lat":40.68552528000001,"lng":-73.98732777999999},{"lat":40.68372672,"lng":-73.98732777999999},{"lat":40.68327708,"lng":-73.98777742},{"lat":40.68327708,"lng":-73.99137454},{"lat":40.681928160000005,"lng":-73.99272346},{"lat":40.68102888000001,"lng":-73.99182418},{"lat":40.67833104,"lng":-73.99182418},{"lat":40.677881400000004,"lng":-73.99227382},{"lat":40.677881400000004,"lng":-73.99317309999999},{"lat":40.67878068,"lng":-73.99407237999999},{"lat":40.677881400000004,"lng":-73.99497165999999},{"lat":40.677881400000004,"lng":-73.99677021999999},{"lat":40.67833104,"lng":-73.99721985999999},{"lat":40.681928160000005,"lng":-73.99721985999999},{"lat":40.682377800000005,"lng":-73.99677021999999},{"lat":40.682377800000005,"lng":-73.99497165999999},{"lat":40.68372672,"lng":-73.99362273999999},{"lat":40.684626,"lng":-73.99452201999999},{"lat":40.686424560000006,"lng":-73.99452201999999},{"lat":40.687323840000005,"lng":-73.99362273999999},{"lat":40.68822312,"lng":-73.99362273999999},{"lat":40.68867276,"lng":-73.99317309999999},{"lat":40.68867276,"lng":-73.99227382},{"lat":40.69002168,"lng":-73.9909249},{"lat":40.69047132000001,"lng":-73.99137454},{"lat":40.69047132000001,"lng":-73.99497165999999},{"lat":40.69092096000001,"lng":-73.99542129999999},{"lat":40.691820240000006,"lng":-73.99542129999999},{"lat":40.69451808,"lng":-73.99811914},{"lat":40.696316640000006,"lng":-73.99632057999999},{"lat":40.698115200000004,"lng":-73.99632057999999},{"lat":40.69991376,"lng":-73.99452201999999},{"lat":40.70351088,"lng":-73.99452201999999},{"lat":40.70396052,"lng":-73.99407237999999},{"lat":40.70396052,"lng":-73.99317309999999},{"lat":40.70575908000001,"lng":-73.99137454},{"lat":40.70575908000001,"lng":-73.9886767},{"lat":40.70620872000001,"lng":-73.98822706},{"lat":40.707108000000005,"lng":-73.98822706},{"lat":40.707557640000005,"lng":-73.9886767},{"lat":40.706658360000006,"lng":-73.98957598},{"lat":40.706658360000006,"lng":-73.99317309999999},{"lat":40.707557640000005,"lng":-73.99407237999999},{"lat":40.707557640000005,"lng":-73.99587093999999},{"lat":40.7048598,"lng":-73.99856878},{"lat":40.7048598,"lng":-74.00036734},{"lat":40.70396052,"lng":-74.00126662},{"lat":40.70396052,"lng":-74.00306518},{"lat":40.70351088,"lng":-74.00351481999999},{"lat":40.701712320000006,"lng":-74.00351481999999},{"lat":40.7003634,"lng":-74.00486373999999},{"lat":40.7003634,"lng":-74.00666229999999},{"lat":40.69991376,"lng":-74.00711193999999},{"lat":40.69901448,"lng":-74.00711193999999},{"lat":40.697215920000005,"lng":-74.0089105},{"lat":40.69541736,"lng":-74.0089105},{"lat":40.69496772,"lng":-74.00936014},{"lat":40.69496772,"lng":-74.01475581999999},{"lat":40.69586700000001,"lng":-74.01565509999999},{"lat":40.69586700000001,"lng":-74.01745365999999},{"lat":40.698115200000004,"lng":-74.01970186},{"lat":40.69901448,"lng":-74.01970186},{"lat":40.69946412,"lng":-74.0201515},{"lat":40.69946412,"lng":-74.02195006},{"lat":40.70081304000001,"lng":-74.02329897999999},{"lat":40.70351088,"lng":-74.02329897999999},{"lat":40.70441016,"lng":-74.02419825999999},{"lat":40.70890656,"lng":-74.02419825999999},{"lat":40.7093562,"lng":-74.02464789999999},{"lat":40.7093562,"lng":-74.02554717999999},{"lat":40.70980584,"lng":-74.02599681999999},{"lat":40.71070512000001,"lng":-74.02509753999999}],"holes":[]},{"shell":[{"lat":40.68327708,"lng":-74.00486373999999},{"lat":40.68327708,"lng":-74.0021659},{"lat":40.682827440000004,"lng":-74.00171626},{"lat":40.681928160000005,"lng":-74.00261554},{"lat":40.6801296,"lng":-74.00261554},{"lat":40.67923032,"lng":-74.00171626},{"lat":40.67878068,"lng":-74.0021659},{"lat":40.682827440000004,"lng":-74.00621265999999},{"lat":40.68327708,"lng":-74.00576301999999},{"lat":40.68327708,"lng":-74.00486373999999}],"holes":[]},{"shell":[{"lat":40.652701560000004,"lng":-74.00486373999999},{"lat":40.6531512,"lng":-74.00531337999999},{"lat":40.65494976,"lng":-74.00531337999999},{"lat":40.65584904000001,"lng":-74.00621265999999},{"lat":40.657197960000005,"lng":-74.00486373999999},{"lat":40.657197960000005,"lng":-74.00306518},{"lat":40.65584904000001,"lng":-74.00171626},{"lat":40.652701560000004,"lng":-74.00486373999999}],"holes":[]},{"shell":[{"lat":40.661694360000006,"lng":-73.99766949999999},{"lat":40.66394256,"lng":-73.9999177},{"lat":40.6643922,"lng":-73.99946806},{"lat":40.6643922,"lng":-73.99766949999999},{"lat":40.663043280000004,"lng":-73.99632057999999},{"lat":40.661694360000006,"lng":-73.99766949999999}],"holes":[]},{"shell":[{"lat":40.673385,"lng":-73.99677021999999},{"lat":40.67383464,"lng":-73.99721985999999},{"lat":40.67518356,"lng":-73.99587093999999},{"lat":40.67473392,"lng":-73.99542129999999},{"lat":40.672935360000004,"lng":-73.99542129999999},{"lat":40.672485720000005,"lng":-73.99587093999999},{"lat":40.673385,"lng":-73.99677021999999}],"holes":[]},{"shell":[{"lat":40.677881400000004,"lng":-73.97159038},{"lat":40.67563320000001,"lng":-73.96934218},{"lat":40.67428428,"lng":-73.9706911},{"lat":40.67563320000001,"lng":-73.97204002},{"lat":40.677431760000005,"lng":-73.97204002},{"lat":40.677881400000004,"lng":-73.97159038}],"holes":[]},{"shell":[{"lat":40.70845692,"lng":-73.96259758},{"lat":40.7093562,"lng":-73.96349685999999},{"lat":40.7093562,"lng":-73.96529541999999},{"lat":40.70980584,"lng":-73.96574505999999},{"lat":40.71520152,"lng":-73.96574505999999},{"lat":40.71610080000001,"lng":-73.96484577999999},{"lat":40.716550440000006,"lng":-73.96529541999999},{"lat":40.716550440000006,"lng":-73.96619469999999},{"lat":40.717000080000005,"lng":-73.96664433999999},{"lat":40.717899360000004,"lng":-73.96664433999999},{"lat":40.71879864,"lng":-73.96574505999999},{"lat":40.71969792,"lng":-73.96664433999999},{"lat":40.721496480000006,"lng":-73.96664433999999},{"lat":40.72374468,"lng":-73.96439613999999},{"lat":40.72374468,"lng":-73.95360477999999},{"lat":40.72419432,"lng":-73.95315513999999},{"lat":40.726892160000006,"lng":-73.95315513999999},{"lat":40.727341800000005,"lng":-73.9527055},{"lat":40.727341800000005,"lng":-73.95180622},{"lat":40.726442520000006,"lng":-73.95090694},{"lat":40.726442520000006,"lng":-73.95000766},{"lat":40.72599288000001,"lng":-73.94955802},{"lat":40.7250936,"lng":-73.94955802},{"lat":40.72374468,"lng":-73.9482091},{"lat":40.72374468,"lng":-73.94641053999999},{"lat":40.723295040000004,"lng":-73.94596089999999},{"lat":40.721496480000006,"lng":-73.94596089999999},{"lat":40.72104684000001,"lng":-73.94551125999999},{"lat":40.72104684000001,"lng":-73.94011558},{"lat":40.72014756,"lng":-73.9392163},{"lat":40.72014756,"lng":-73.93741774},{"lat":40.717899360000004,"lng":-73.93516953999999},{"lat":40.71610080000001,"lng":-73.93516953999999},{"lat":40.71475188,"lng":-73.93382061999999},{"lat":40.71565116,"lng":-73.93292134},{"lat":40.71565116,"lng":-73.93112278},{"lat":40.71520152,"lng":-73.93067314},{"lat":40.71340296,"lng":-73.93067314},{"lat":40.71070512000001,"lng":-73.93337097999999},{"lat":40.71025548,"lng":-73.93292134},{"lat":40.71025548,"lng":-73.93112278},{"lat":40.7093562,"lng":-73.9302235},{"lat":40.7093562,"lng":-73.92842494},{"lat":40.70890656,"lng":-73.9279753},{"lat":40.708007280000004,"lng":-73.9279753},{"lat":40.707557640000005,"lng":-73.92752566},{"lat":40.707557640000005,"lng":-73.92662637999999},{"lat":40.708007280000004,"lng":-73.92617673999999},{"lat":40.70890656,"lng":-73.92617673999999},{"lat":40.7093562,"lng":-73.92572709999999},{"lat":40.7093562,"lng":-73.92482781999999},{"lat":40.70845692,"lng":-73.92392853999999},{"lat":40.70845692,"lng":-73.92302925999999},{"lat":40.71025548,"lng":-73.9212307},{"lat":40.71025548,"lng":-73.91943214},{"lat":40.70980584,"lng":-73.9189825},{"lat":40.708007280000004,"lng":-73.9189825},{"lat":40.707108000000005,"lng":-73.91988178},{"lat":40.70620872000001,"lng":-73.91988178},{"lat":40.70575908000001,"lng":-73.91943214},{"lat":40.70575908000001,"lng":-73.91673429999999},{"lat":40.706658360000006,"lng":-73.91583501999999},{"lat":40.70620872000001,"lng":-73.91538537999999},{"lat":40.702611600000004,"lng":-73.91538537999999},{"lat":40.701262680000006,"lng":-73.91673429999999},{"lat":40.701262680000006,"lng":-73.92033142},{"lat":40.703061240000004,"lng":-73.92212998},{"lat":40.703061240000004,"lng":-73.92392853999999},{"lat":40.7048598,"lng":-73.92572709999999},{"lat":40.7048598,"lng":-73.92752566},{"lat":40.702161960000005,"lng":-73.9302235},{"lat":40.702161960000005,"lng":-73.93202206},{"lat":40.701262680000006,"lng":-73.93292134},{"lat":40.701262680000006,"lng":-73.9392163},{"lat":40.70081304000001,"lng":-73.93966594},{"lat":40.69901448,"lng":-73.93966594},{"lat":40.69856484,"lng":-73.94011558},{"lat":40.69856484,"lng":-73.94191414},{"lat":40.69901448,"lng":-73.94236378},{"lat":40.702611600000004,"lng":-73.94236378},{"lat":40.703061240000004,"lng":-73.94281342},{"lat":40.702161960000005,"lng":-73.94371269999999},{"lat":40.702161960000005,"lng":-73.94730982},{"lat":40.701712320000006,"lng":-73.94775946},{"lat":40.70081304000001,"lng":-73.94775946},{"lat":40.7003634,"lng":-73.9482091},{"lat":40.7003634,"lng":-73.94910838},{"lat":40.69991376,"lng":-73.94955802},{"lat":40.69901448,"lng":-73.94955802},{"lat":40.69856484,"lng":-73.95000766},{"lat":40.69856484,"lng":-73.95090694},{"lat":40.69991376,"lng":-73.95225586},{"lat":40.701712320000006,"lng":-73.95225586},{"lat":40.702611600000004,"lng":-73.95135658},{"lat":40.7048598,"lng":-73.95360477999999},{"lat":40.7048598,"lng":-73.95720189999999},{"lat":40.70575908000001,"lng":-73.95810118},{"lat":40.7048598,"lng":-73.95900046},{"lat":40.7048598,"lng":-73.96079902},{"lat":40.70530944,"lng":-73.96124866},{"lat":40.707108000000005,"lng":-73.96124866},{"lat":40.70845692,"lng":-73.96259758}],"holes":[]},{"shell":[{"lat":40.741730280000006,"lng":-73.95900046},{"lat":40.743079200000004,"lng":-73.96034938},{"lat":40.74442812,"lng":-73.95900046},{"lat":40.74442812,"lng":-73.95720189999999},{"lat":40.7453274,"lng":-73.95630261999999},{"lat":40.7453274,"lng":-73.95540333999999},{"lat":40.74442812,"lng":-73.95450405999999},{"lat":40.7453274,"lng":-73.95360477999999},{"lat":40.7453274,"lng":-73.94910838},{"lat":40.74577704,"lng":-73.94865874},{"lat":40.74847488,"lng":-73.94865874},{"lat":40.74892452,"lng":-73.9482091},{"lat":40.74892452,"lng":-73.94551125999999},{"lat":40.7498238,"lng":-73.94461197999999},{"lat":40.7498238,"lng":-73.94371269999999},{"lat":40.75027344,"lng":-73.94326305999999},{"lat":40.75117272000001,"lng":-73.94326305999999},{"lat":40.752072000000005,"lng":-73.94236378},{"lat":40.75387056,"lng":-73.94236378},{"lat":40.75476984,"lng":-73.94326305999999},{"lat":40.75521948,"lng":-73.94281342},{"lat":40.75387056,"lng":-73.9414645},{"lat":40.752971280000004,"lng":-73.9414645},{"lat":40.752521640000005,"lng":-73.94101486},{"lat":40.752521640000005,"lng":-73.94011558},{"lat":40.75342092,"lng":-73.9392163},{"lat":40.75027344,"lng":-73.93606881999999},{"lat":40.74847488,"lng":-73.93606881999999},{"lat":40.748025240000004,"lng":-73.93651845999999},{"lat":40.748025240000004,"lng":-73.94101486},{"lat":40.747575600000005,"lng":-73.9414645},{"lat":40.746676320000006,"lng":-73.9414645},{"lat":40.7453274,"lng":-73.94281342},{"lat":40.7453274,"lng":-73.94641053999999},{"lat":40.74397848,"lng":-73.94775946},{"lat":40.742179920000005,"lng":-73.94775946},{"lat":40.741730280000006,"lng":-73.9482091},{"lat":40.741730280000006,"lng":-73.94910838},{"lat":40.74128064000001,"lng":-73.94955802},{"lat":40.74038136,"lng":-73.94955802},{"lat":40.73993172,"lng":-73.95000766},{"lat":40.73993172,"lng":-73.95090694},{"lat":40.73903244,"lng":-73.95180622},{"lat":40.73903244,"lng":-73.95360477999999},{"lat":40.740831,"lng":-73.95540333999999},{"lat":40.740831,"lng":-73.95810118},{"lat":40.741730280000006,"lng":-73.95900046}],"holes":[]},{"shell":[{"lat":40.740831,"lng":-73.94551125999999},{"lat":40.742179920000005,"lng":-73.94686017999999},{"lat":40.74442812,"lng":-73.94461197999999},{"lat":40.74397848,"lng":-73.94416233999999},{"lat":40.742179920000005,"lng":-73.94416233999999},{"lat":40.740831,"lng":-73.94551125999999}],"holes":[]},{"shell":[{"lat":40.81412232,"lng":-73.93067314},{"lat":40.81457196,"lng":-73.9302235},{"lat":40.813223040000004,"lng":-73.92887458},{"lat":40.811874120000006,"lng":-73.9302235},{"lat":40.813223040000004,"lng":-73.93157242},{"lat":40.81412232,"lng":-73.93067314}],"holes":[]},{"shell":[{"lat":40.69856484,"lng":-73.9122379},{"lat":40.69991376,"lng":-73.91358681999999},{"lat":40.701262680000006,"lng":-73.9122379},{"lat":40.701262680000006,"lng":-73.91043934},{"lat":40.69991376,"lng":-73.90909042},{"lat":40.69946412,"lng":-73.90954006},{"lat":40.69946412,"lng":-73.91133862},{"lat":40.69856484,"lng":-73.9122379}],"holes":[]},{"shell":[{"lat":40.696766280000006,"lng":-73.94910838},{"lat":40.696766280000006,"lng":-73.9482091},{"lat":40.696316640000006,"lng":-73.94775946},{"lat":40.6936188,"lng":-73.94775946},{"lat":40.693169160000004,"lng":-73.9482091},{"lat":40.693169160000004,"lng":-73.95000766},{"lat":40.6936188,"lng":-73.9504573},{"lat":40.69451808,"lng":-73.9504573},{"lat":40.69541736,"lng":-73.94955802},{"lat":40.696316640000006,"lng":-73.94955802},{"lat":40.696766280000006,"lng":-73.94910838}],"holes":[]},{"shell":[{"lat":40.80512952,"lng":-73.93516953999999},{"lat":40.803330960000004,"lng":-73.93516953999999},{"lat":40.802881320000004,"lng":-73.93561917999999},{"lat":40.8037806,"lng":-73.93651845999999},{"lat":40.8037806,"lng":-73.93741774},{"lat":40.803330960000004,"lng":-73.93786738},{"lat":40.802431680000005,"lng":-73.93786738},{"lat":40.801982040000006,"lng":-73.93831702},{"lat":40.803330960000004,"lng":-73.93966594},{"lat":40.80512952,"lng":-73.93966594},{"lat":40.80557916,"lng":-73.9392163},{"lat":40.80557916,"lng":-73.93741774},{"lat":40.807377720000005,"lng":-73.93561917999999},{"lat":40.806928080000006,"lng":-73.93516953999999},{"lat":40.80512952,"lng":-73.93516953999999}],"holes":[]},{"shell":[{"lat":40.7902914,"lng":-73.96349685999999},{"lat":40.7902914,"lng":-73.96439613999999},{"lat":40.791640320000006,"lng":-73.96574505999999},{"lat":40.792989240000004,"lng":-73.96439613999999},{"lat":40.79074104,"lng":-73.96214794},{"lat":40.7902914,"lng":-73.96259758},{"lat":40.7902914,"lng":-73.96349685999999}],"holes":[]},{"shell":[{"lat":40.758366960000004,"lng":-73.95135658},{"lat":40.757917320000004,"lng":-73.95180622},{"lat":40.757917320000004,"lng":-73.95360477999999},{"lat":40.757018040000005,"lng":-73.95450405999999},{"lat":40.758366960000004,"lng":-73.95585297999999},{"lat":40.761514440000006,"lng":-73.9527055},{"lat":40.76016552,"lng":-73.95135658},{"lat":40.758366960000004,"lng":-73.95135658}],"holes":[]}],"properties":{}},{"search_id":"casey_15_min","shapes":[{"shell":[{"lat":40.71397668,"lng":-74.01587252},{"lat":40.7153256,"lng":-74.01722144},{"lat":40.716224880000006,"lng":-74.01632216},{"lat":40.71802344,"lng":-74.01632216},{"lat":40.71847308,"lng":-74.01587252},{"lat":40.71847308,"lng":-74.01407396},{"lat":40.719822,"lng":-74.01272504},{"lat":40.721620560000005,"lng":-74.01272504},{"lat":40.722519840000004,"lng":-74.01362432},{"lat":40.7243184,"lng":-74.01362432},{"lat":40.727016240000005,"lng":-74.01092648},{"lat":40.7288148,"lng":-74.01092648},{"lat":40.72971408,"lng":-74.01182576},{"lat":40.7333112,"lng":-74.01182576},{"lat":40.73600904,"lng":-74.00912792},{"lat":40.74320328,"lng":-74.00912792},{"lat":40.74410256,"lng":-74.0100272},{"lat":40.74545148,"lng":-74.00867828},{"lat":40.74545148,"lng":-74.00418188},{"lat":40.74590112,"lng":-74.00373224},{"lat":40.746800400000005,"lng":-74.00373224},{"lat":40.74859896,"lng":-74.0055308},{"lat":40.7490486,"lng":-74.00508116},{"lat":40.7490486,"lng":-74.0032826},{"lat":40.74994788,"lng":-74.00238332},{"lat":40.74994788,"lng":-74.00058476},{"lat":40.75084716,"lng":-73.99968548},{"lat":40.74994788,"lng":-73.9987862},{"lat":40.74994788,"lng":-73.99788692},{"lat":40.75039752,"lng":-73.99743728},{"lat":40.7512968,"lng":-73.99743728},{"lat":40.75309536,"lng":-73.99923584},{"lat":40.75534356,"lng":-73.99698764},{"lat":40.75444428,"lng":-73.99608836},{"lat":40.75444428,"lng":-73.99518908},{"lat":40.75489392,"lng":-73.99473944},{"lat":40.7557932,"lng":-73.99473944},{"lat":40.756692480000005,"lng":-73.99384016},{"lat":40.75849104,"lng":-73.99384016},{"lat":40.75983996,"lng":-73.99249124},{"lat":40.75894068,"lng":-73.99159196},{"lat":40.75894068,"lng":-73.9897934},{"lat":40.76118888,"lng":-73.9875452},{"lat":40.76298744,"lng":-73.9875452},{"lat":40.76343708,"lng":-73.98709556},{"lat":40.76343708,"lng":-73.985297},{"lat":40.762088160000005,"lng":-73.98394808},{"lat":40.761638520000005,"lng":-73.98439772},{"lat":40.761638520000005,"lng":-73.98619628},{"lat":40.76118888,"lng":-73.98664592},{"lat":40.757591760000004,"lng":-73.98664592},{"lat":40.757142120000005,"lng":-73.98619628},{"lat":40.757142120000005,"lng":-73.98439772},{"lat":40.757591760000004,"lng":-73.98394808},{"lat":40.75939032,"lng":-73.98394808},{"lat":40.7602896,"lng":-73.98484736},{"lat":40.76073924,"lng":-73.98439772},{"lat":40.76073924,"lng":-73.98259916},{"lat":40.75983996,"lng":-73.98169988000001},{"lat":40.75983996,"lng":-73.97990132},{"lat":40.76073924,"lng":-73.97900204},{"lat":40.7602896,"lng":-73.9785524},{"lat":40.75849104,"lng":-73.9785524},{"lat":40.757591760000004,"lng":-73.97765312},{"lat":40.757142120000005,"lng":-73.97810276},{"lat":40.757142120000005,"lng":-73.97990132},{"lat":40.75624284,"lng":-73.9808006},{"lat":40.75624284,"lng":-73.98169988000001},{"lat":40.7557932,"lng":-73.98214952000001},{"lat":40.75489392,"lng":-73.98214952000001},{"lat":40.75309536,"lng":-73.98035096},{"lat":40.751746440000005,"lng":-73.98169988000001},{"lat":40.751746440000005,"lng":-73.98439772},{"lat":40.7512968,"lng":-73.98484736},{"lat":40.75084716,"lng":-73.98439772},{"lat":40.75084716,"lng":-73.98349844},{"lat":40.75039752,"lng":-73.9830488},{"lat":40.74859896,"lng":-73.9830488},{"lat":40.74814932,"lng":-73.98259916},{"lat":40.74814932,"lng":-73.9808006},{"lat":40.747699680000004,"lng":-73.98035096},{"lat":40.74635076,"lng":-73.98169988000001},{"lat":40.74635076,"lng":-73.985297},{"lat":40.74590112,"lng":-73.98574664},{"lat":40.74410256,"lng":-73.98574664},{"lat":40.74365292,"lng":-73.98619628},{"lat":40.74365292,"lng":-73.98709556},{"lat":40.74320328,"lng":-73.9875452},{"lat":40.742304000000004,"lng":-73.9875452},{"lat":40.73960616,"lng":-73.98484736},{"lat":40.73915652,"lng":-73.985297},{"lat":40.73915652,"lng":-73.98709556},{"lat":40.7400558,"lng":-73.98799484},{"lat":40.7400558,"lng":-73.9897934},{"lat":40.73870688,"lng":-73.99114232},{"lat":40.737807600000004,"lng":-73.99114232},{"lat":40.737357960000004,"lng":-73.99069268},{"lat":40.737357960000004,"lng":-73.9897934},{"lat":40.7333112,"lng":-73.98574664},{"lat":40.73286156,"lng":-73.98619628},{"lat":40.73286156,"lng":-73.98889412},{"lat":40.732411920000004,"lng":-73.98934376},{"lat":40.731512640000005,"lng":-73.98934376},{"lat":40.731063000000006,"lng":-73.98889412},{"lat":40.731063000000006,"lng":-73.98709556},{"lat":40.73061336,"lng":-73.98664592},{"lat":40.72791552,"lng":-73.98664592},{"lat":40.727465880000004,"lng":-73.98619628},{"lat":40.727465880000004,"lng":-73.985297},{"lat":40.727016240000005,"lng":-73.98484736},{"lat":40.726116960000006,"lng":-73.98574664},{"lat":40.72521768,"lng":-73.98574664},{"lat":40.72476804,"lng":-73.985297},{"lat":40.72476804,"lng":-73.98439772},{"lat":40.7243184,"lng":-73.98394808},{"lat":40.72341912,"lng":-73.98394808},{"lat":40.722519840000004,"lng":-73.9830488},{"lat":40.71892272,"lng":-73.9830488},{"lat":40.71802344,"lng":-73.98214952000001},{"lat":40.716224880000006,"lng":-73.98214952000001},{"lat":40.71397668,"lng":-73.98439772},{"lat":40.71397668,"lng":-73.98619628},{"lat":40.71352704,"lng":-73.98664592},{"lat":40.711728480000005,"lng":-73.98664592},{"lat":40.70948028,"lng":-73.98889412},{"lat":40.70948028,"lng":-73.99069268},{"lat":40.71397668,"lng":-73.99518908},{"lat":40.71397668,"lng":-73.99608836},{"lat":40.7130774,"lng":-73.99698764},{"lat":40.7130774,"lng":-73.9987862},{"lat":40.71352704,"lng":-73.99923584},{"lat":40.716224880000006,"lng":-73.99923584},{"lat":40.716674520000005,"lng":-73.99968548},{"lat":40.716674520000005,"lng":-74.00058476},{"lat":40.71442632,"lng":-74.00283296},{"lat":40.710829200000006,"lng":-74.00283296},{"lat":40.70992992,"lng":-74.00193368000001},{"lat":40.70813136,"lng":-74.00373224},{"lat":40.706332800000006,"lng":-74.00373224},{"lat":40.705883160000006,"lng":-74.00418188},{"lat":40.705883160000006,"lng":-74.01137612},{"lat":40.706782440000005,"lng":-74.01227540000001},{"lat":40.706782440000005,"lng":-74.01407396},{"lat":40.707232080000004,"lng":-74.0145236},{"lat":40.70813136,"lng":-74.01362432},{"lat":40.70992992,"lng":-74.01362432},{"lat":40.711728480000005,"lng":-74.01542288},{"lat":40.71352704,"lng":-74.01542288},{"lat":40.71397668,"lng":-74.01587252}],"holes":[]},{"shell":[{"lat":40.70003784,"lng":-73.99204160000001},{"lat":40.702286040000004,"lng":-73.9897934},{"lat":40.702286040000004,"lng":-73.98799484},{"lat":40.70318532,"lng":-73.98709556},{"lat":40.70318532,"lng":-73.985297},{"lat":40.70273568,"lng":-73.98484736},{"lat":40.700937120000006,"lng":-73.98484736},{"lat":40.6995882,"lng":-73.98619628},{"lat":40.6995882,"lng":-73.98799484},{"lat":40.70048748000001,"lng":-73.98889412},{"lat":40.70003784,"lng":-73.98934376},{"lat":40.69823928,"lng":-73.98934376},{"lat":40.69778964,"lng":-73.9897934},{"lat":40.69778964,"lng":-73.99159196},{"lat":40.69823928,"lng":-73.99204160000001},{"lat":40.70003784,"lng":-73.99204160000001}],"holes":[]},{"shell":[{"lat":40.6928436,"lng":-73.98844448},{"lat":40.69329324,"lng":-73.98799484},{"lat":40.69329324,"lng":-73.98619628},{"lat":40.6928436,"lng":-73.98574664},{"lat":40.691944320000005,"lng":-73.98574664},{"lat":40.69059540000001,"lng":-73.98709556},{"lat":40.69059540000001,"lng":-73.98799484},{"lat":40.691045040000006,"lng":-73.98844448},{"lat":40.6928436,"lng":-73.98844448}],"holes":[]},{"shell":[{"lat":40.764786,"lng":-73.97945168},{"lat":40.76388672,"lng":-73.98035096},{"lat":40.762088160000005,"lng":-73.98035096},{"lat":40.761638520000005,"lng":-73.9808006},{"lat":40.761638520000005,"lng":-73.98259916},{"lat":40.762088160000005,"lng":-73.9830488},{"lat":40.76388672,"lng":-73.98125024},{"lat":40.76568528,"lng":-73.98125024},{"lat":40.76613492,"lng":-73.9808006},{"lat":40.764786,"lng":-73.97945168}],"holes":[]}],"properties":{}},{"search_id":"casey_20_min","shapes":[{"shell":[{"lat":40.71521242,"lng":-74.0340448},{"lat":40.71521242,"lng":-74.03494408},{"lat":40.71566206,"lng":-74.03539372},{"lat":40.71656134,"lng":-74.03449444},{"lat":40.7183599,"lng":-74.03449444},{"lat":40.71880954,"lng":-74.0340448},{"lat":40.71880954,"lng":-74.03224624},{"lat":40.71656134,"lng":-74.02999804000001},{"lat":40.71431314,"lng":-74.03224624},{"lat":40.71521242,"lng":-74.03314552},{"lat":40.71521242,"lng":-74.0340448}],"holes":[]},{"shell":[{"lat":40.73499658,"lng":-74.02864912},{"lat":40.73499658,"lng":-74.02954840000001},{"lat":40.7363455,"lng":-74.03089732000001},{"lat":40.73679514,"lng":-74.03044768000001},{"lat":40.73679514,"lng":-74.02774984},{"lat":40.7363455,"lng":-74.0273002},{"lat":40.73544622,"lng":-74.0273002},{"lat":40.73499658,"lng":-74.02774984},{"lat":40.73499658,"lng":-74.02864912}],"holes":[]},{"shell":[{"lat":40.7183599,"lng":-74.02010596000001},{"lat":40.71970882,"lng":-74.01875704},{"lat":40.71970882,"lng":-74.01785776},{"lat":40.72015846,"lng":-74.01740812},{"lat":40.72105774,"lng":-74.01740812},{"lat":40.72150738,"lng":-74.01695848},{"lat":40.72150738,"lng":-74.0160592},{"lat":40.722856300000004,"lng":-74.01471028},{"lat":40.72375558,"lng":-74.01560956},{"lat":40.72555414,"lng":-74.01560956},{"lat":40.73005054,"lng":-74.01111316000001},{"lat":40.73139946,"lng":-74.01246208},{"lat":40.73139946,"lng":-74.01426064},{"lat":40.732748380000004,"lng":-74.01560956},{"lat":40.73454694,"lng":-74.01560956},{"lat":40.73589586,"lng":-74.01426064},{"lat":40.73589586,"lng":-74.01246208},{"lat":40.7363455,"lng":-74.01201244},{"lat":40.73904334,"lng":-74.01201244},{"lat":40.74039226,"lng":-74.01066352000001},{"lat":40.74039226,"lng":-74.00976424000001},{"lat":40.7408419,"lng":-74.00931460000001},{"lat":40.74174118,"lng":-74.00931460000001},{"lat":40.74353974,"lng":-74.01111316000001},{"lat":40.74443902,"lng":-74.01021388000001},{"lat":40.74623758,"lng":-74.01021388000001},{"lat":40.74668722,"lng":-74.00976424000001},{"lat":40.74668722,"lng":-74.00886496000001},{"lat":40.74713686,"lng":-74.00841532},{"lat":40.7498347,"lng":-74.00841532},{"lat":40.75028434,"lng":-74.00796568},{"lat":40.75028434,"lng":-74.00616712},{"lat":40.7520829,"lng":-74.00436856},{"lat":40.7520829,"lng":-74.00346928},{"lat":40.75253254,"lng":-74.00301964},{"lat":40.753431819999996,"lng":-74.00301964},{"lat":40.7543311,"lng":-74.00391892},{"lat":40.75612966,"lng":-74.00391892},{"lat":40.75702894,"lng":-74.0048182},{"lat":40.75747858,"lng":-74.00436856},{"lat":40.75747858,"lng":-74.00257},{"lat":40.758377859999996,"lng":-74.00167072},{"lat":40.75747858,"lng":-74.00077144000001},{"lat":40.75747858,"lng":-73.9980736},{"lat":40.75792822,"lng":-73.99762396},{"lat":40.76062606,"lng":-73.99762396},{"lat":40.7610757,"lng":-73.99717432},{"lat":40.7610757,"lng":-73.99537576},{"lat":40.76197498,"lng":-73.99447648},{"lat":40.7610757,"lng":-73.9935772},{"lat":40.7610757,"lng":-73.99177864},{"lat":40.76242462,"lng":-73.99042972000001},{"lat":40.76422318,"lng":-73.99042972000001},{"lat":40.76647138,"lng":-73.98818152},{"lat":40.7655721,"lng":-73.98728224},{"lat":40.7655721,"lng":-73.98548368},{"lat":40.76602174,"lng":-73.98503404},{"lat":40.7678203,"lng":-73.98503404},{"lat":40.768719579999996,"lng":-73.98593332},{"lat":40.77141742,"lng":-73.98323548},{"lat":40.773215979999996,"lng":-73.98323548},{"lat":40.7745649,"lng":-73.98188656},{"lat":40.774115259999995,"lng":-73.98143692000001},{"lat":40.7723167,"lng":-73.98143692000001},{"lat":40.77186706,"lng":-73.98098728000001},{"lat":40.77186706,"lng":-73.97739016},{"lat":40.77141742,"lng":-73.97694052},{"lat":40.7678203,"lng":-73.97694052},{"lat":40.76602174,"lng":-73.97514196},{"lat":40.76512246,"lng":-73.97604124},{"lat":40.76242462,"lng":-73.97604124},{"lat":40.76197498,"lng":-73.9755916},{"lat":40.76197498,"lng":-73.97469232},{"lat":40.76287426,"lng":-73.97379304},{"lat":40.76242462,"lng":-73.9733434},{"lat":40.76062606,"lng":-73.9733434},{"lat":40.7588275,"lng":-73.97514196},{"lat":40.75702894,"lng":-73.97514196},{"lat":40.75612966,"lng":-73.97424268},{"lat":40.75478074,"lng":-73.9755916},{"lat":40.75478074,"lng":-73.97739016},{"lat":40.7543311,"lng":-73.9778398},{"lat":40.753431819999996,"lng":-73.9778398},{"lat":40.75298218,"lng":-73.97739016},{"lat":40.75298218,"lng":-73.97469232},{"lat":40.75073398,"lng":-73.97244412},{"lat":40.75028434,"lng":-73.97289376},{"lat":40.75028434,"lng":-73.97469232},{"lat":40.74893542,"lng":-73.97604124},{"lat":40.74713686,"lng":-73.97604124},{"lat":40.7453383,"lng":-73.97424268},{"lat":40.7430901,"lng":-73.97649088},{"lat":40.74353974,"lng":-73.97694052},{"lat":40.74443902,"lng":-73.97694052},{"lat":40.74488866,"lng":-73.97739016},{"lat":40.74443902,"lng":-73.9778398},{"lat":40.74353974,"lng":-73.9778398},{"lat":40.7430901,"lng":-73.97828944},{"lat":40.74398938,"lng":-73.97918872000001},{"lat":40.74398938,"lng":-73.98008800000001},{"lat":40.74353974,"lng":-73.98053764000001},{"lat":40.74264046,"lng":-73.98053764000001},{"lat":40.74174118,"lng":-73.98143692000001},{"lat":40.73814406,"lng":-73.98143692000001},{"lat":40.73679514,"lng":-73.98278584},{"lat":40.73679514,"lng":-73.98368512},{"lat":40.7363455,"lng":-73.98413476},{"lat":40.7340973,"lng":-73.98188656},{"lat":40.7340973,"lng":-73.98098728000001},{"lat":40.73544622,"lng":-73.97963836000001},{"lat":40.73724478,"lng":-73.97963836000001},{"lat":40.73769442,"lng":-73.97918872000001},{"lat":40.73769442,"lng":-73.97739016},{"lat":40.7363455,"lng":-73.97604124},{"lat":40.73499658,"lng":-73.97739016},{"lat":40.73499658,"lng":-73.97828944},{"lat":40.73454694,"lng":-73.97873908},{"lat":40.73364766,"lng":-73.9778398},{"lat":40.7318491,"lng":-73.9778398},{"lat":40.73094982,"lng":-73.97694052},{"lat":40.73005054,"lng":-73.97694052},{"lat":40.7296009,"lng":-73.97739016},{"lat":40.7296009,"lng":-73.97828944},{"lat":40.72870162,"lng":-73.97918872000001},{"lat":40.72870162,"lng":-73.98098728000001},{"lat":40.73050018,"lng":-73.98278584},{"lat":40.73005054,"lng":-73.98323548},{"lat":40.72825198,"lng":-73.98323548},{"lat":40.727802340000004,"lng":-73.98278584},{"lat":40.727802340000004,"lng":-73.98188656},{"lat":40.7273527,"lng":-73.98143692000001},{"lat":40.722856300000004,"lng":-73.98143692000001},{"lat":40.722406660000004,"lng":-73.98098728000001},{"lat":40.722406660000004,"lng":-73.97918872000001},{"lat":40.72015846,"lng":-73.97694052},{"lat":40.71925918,"lng":-73.97694052},{"lat":40.71880954,"lng":-73.97739016},{"lat":40.71880954,"lng":-73.97828944},{"lat":40.7183599,"lng":-73.97873908},{"lat":40.71296422,"lng":-73.97873908},{"lat":40.712514580000004,"lng":-73.97918872000001},{"lat":40.712514580000004,"lng":-73.98098728000001},{"lat":40.707568540000004,"lng":-73.98593332},{"lat":40.70576998,"lng":-73.98593332},{"lat":40.70532034,"lng":-73.98548368},{"lat":40.70532034,"lng":-73.98368512},{"lat":40.70397142,"lng":-73.9823362},{"lat":40.702172860000005,"lng":-73.9823362},{"lat":40.701723220000005,"lng":-73.98188656},{"lat":40.701723220000005,"lng":-73.98098728000001},{"lat":40.70127358,"lng":-73.98053764000001},{"lat":40.69947502,"lng":-73.98053764000001},{"lat":40.69902538,"lng":-73.98098728000001},{"lat":40.69902538,"lng":-73.98548368},{"lat":40.69857574,"lng":-73.98593332},{"lat":40.6958779,"lng":-73.98593332},{"lat":40.6936297,"lng":-73.98368512},{"lat":40.6936297,"lng":-73.98188656},{"lat":40.691381500000006,"lng":-73.97963836000001},{"lat":40.68868366,"lng":-73.97963836000001},{"lat":40.687784380000004,"lng":-73.98053764000001},{"lat":40.686885100000005,"lng":-73.98053764000001},{"lat":40.686435460000006,"lng":-73.98098728000001},{"lat":40.686435460000006,"lng":-73.9845844},{"lat":40.68553618,"lng":-73.98548368},{"lat":40.685985820000006,"lng":-73.98593332},{"lat":40.686885100000005,"lng":-73.98593332},{"lat":40.687334740000004,"lng":-73.98638296},{"lat":40.687334740000004,"lng":-73.98818152},{"lat":40.686885100000005,"lng":-73.98863116},{"lat":40.685985820000006,"lng":-73.98863116},{"lat":40.68508654,"lng":-73.98773188},{"lat":40.6846369,"lng":-73.98818152},{"lat":40.6846369,"lng":-73.99177864},{"lat":40.68373762,"lng":-73.99267792},{"lat":40.68418726,"lng":-73.99312756},{"lat":40.68508654,"lng":-73.99222828},{"lat":40.686885100000005,"lng":-73.99222828},{"lat":40.687334740000004,"lng":-73.99177864},{"lat":40.687334740000004,"lng":-73.98998008000001},{"lat":40.687784380000004,"lng":-73.98953044000001},{"lat":40.68958294,"lng":-73.98953044000001},{"lat":40.692280780000004,"lng":-73.99222828},{"lat":40.69407934,"lng":-73.99222828},{"lat":40.6958779,"lng":-73.99402684},{"lat":40.697676460000004,"lng":-73.99402684},{"lat":40.69947502,"lng":-73.9958254},{"lat":40.70397142,"lng":-73.9958254},{"lat":40.70442106,"lng":-73.99537576},{"lat":40.70442106,"lng":-73.99177864},{"lat":40.70532034,"lng":-73.99087936000001},{"lat":40.70532034,"lng":-73.98908080000001},{"lat":40.70666926,"lng":-73.98773188},{"lat":40.707118900000005,"lng":-73.98818152},{"lat":40.707118900000005,"lng":-73.98908080000001},{"lat":40.707568540000004,"lng":-73.98953044000001},{"lat":40.70846782,"lng":-73.98953044000001},{"lat":40.70891746,"lng":-73.98998008000001},{"lat":40.70891746,"lng":-73.99177864},{"lat":40.70801818,"lng":-73.99267792},{"lat":40.70801818,"lng":-73.99447648},{"lat":40.70981674,"lng":-73.99627504},{"lat":40.70981674,"lng":-73.9980736},{"lat":40.7093671,"lng":-73.99852324},{"lat":40.707568540000004,"lng":-73.99852324},{"lat":40.70666926,"lng":-73.99942252000001},{"lat":40.7048707,"lng":-73.99942252000001},{"lat":40.70442106,"lng":-73.99987216000001},{"lat":40.70442106,"lng":-74.00346928},{"lat":40.70352178,"lng":-74.00436856},{"lat":40.70352178,"lng":-74.00616712},{"lat":40.702622500000004,"lng":-74.0070664},{"lat":40.702622500000004,"lng":-74.01066352000001},{"lat":40.69992466,"lng":-74.01336136},{"lat":40.69992466,"lng":-74.01426064},{"lat":40.701723220000005,"lng":-74.0160592},{"lat":40.701723220000005,"lng":-74.01785776},{"lat":40.702622500000004,"lng":-74.01875704},{"lat":40.702622500000004,"lng":-74.02055560000001},{"lat":40.70307214,"lng":-74.02100524000001},{"lat":40.70666926,"lng":-74.02100524000001},{"lat":40.707118900000005,"lng":-74.02055560000001},{"lat":40.707118900000005,"lng":-74.01875704},{"lat":40.707568540000004,"lng":-74.0183074},{"lat":40.7093671,"lng":-74.0183074},{"lat":40.71026638,"lng":-74.01920668000001},{"lat":40.71116566,"lng":-74.0183074},{"lat":40.712064940000005,"lng":-74.0183074},{"lat":40.712514580000004,"lng":-74.01875704},{"lat":40.712514580000004,"lng":-74.01965632000001},{"lat":40.7138635,"lng":-74.02100524000001},{"lat":40.717460620000004,"lng":-74.02100524000001},{"lat":40.7183599,"lng":-74.02010596000001}],"holes":[]},{"shell":[{"lat":40.67969086,"lng":-73.99492612},{"lat":40.67924122,"lng":-73.99537576},{"lat":40.67969086,"lng":-73.9958254},{"lat":40.681489420000005,"lng":-73.9958254},{"lat":40.681939060000005,"lng":-73.99537576},{"lat":40.68059014,"lng":-73.99402684},{"lat":40.67969086,"lng":-73.99492612}],"holes":[]},{"shell":[{"lat":40.6846369,"lng":-73.97649088},{"lat":40.68328798,"lng":-73.97514196},{"lat":40.68283834,"lng":-73.9755916},{"lat":40.68283834,"lng":-73.97739016},{"lat":40.68418726,"lng":-73.97873908},{"lat":40.6846369,"lng":-73.97828944},{"lat":40.6846369,"lng":-73.97649088}],"holes":[]},{"shell":[{"lat":40.6778923,"lng":-73.97424268},{"lat":40.67834194,"lng":-73.97379304},{"lat":40.676093740000006,"lng":-73.97154484},{"lat":40.67564410000001,"lng":-73.97199448},{"lat":40.6778923,"lng":-73.97424268}],"holes":[]},{"shell":[{"lat":40.7565793,"lng":-73.96929664000001},{"lat":40.7565793,"lng":-73.97199448},{"lat":40.75702894,"lng":-73.97244412},{"lat":40.758377859999996,"lng":-73.97109520000001},{"lat":40.758377859999996,"lng":-73.96929664000001},{"lat":40.75702894,"lng":-73.96794772},{"lat":40.7565793,"lng":-73.96839736},{"lat":40.7565793,"lng":-73.96929664000001}],"holes":[]},{"shell":[{"lat":40.76287426,"lng":-73.96569952},{"lat":40.76287426,"lng":-73.96749808},{"lat":40.76422318,"lng":-73.968847},{"lat":40.76602174,"lng":-73.968847},{"lat":40.76647138,"lng":-73.96839736},{"lat":40.7655721,"lng":-73.96749808},{"lat":40.7655721,"lng":-73.9665988},{"lat":40.76602174,"lng":-73.96614916},{"lat":40.76692102,"lng":-73.96614916},{"lat":40.76737066,"lng":-73.96569952},{"lat":40.76512246,"lng":-73.96345132},{"lat":40.76287426,"lng":-73.96569952}],"holes":[]},{"shell":[{"lat":40.7093671,"lng":-73.96075348000001},{"lat":40.70981674,"lng":-73.96030384000001},{"lat":40.70981674,"lng":-73.95670672},{"lat":40.70846782,"lng":-73.9553578},{"lat":40.707118900000005,"lng":-73.95670672},{"lat":40.707118900000005,"lng":-73.95850528},{"lat":40.7093671,"lng":-73.96075348000001}],"holes":[]},{"shell":[{"lat":40.758377859999996,"lng":-73.95400888},{"lat":40.7588275,"lng":-73.95445852},{"lat":40.76017642,"lng":-73.9531096},{"lat":40.7588275,"lng":-73.95176068},{"lat":40.758377859999996,"lng":-73.95221032},{"lat":40.758377859999996,"lng":-73.95400888}],"holes":[]},{"shell":[{"lat":40.7700685,"lng":-73.957606},{"lat":40.76961886,"lng":-73.95715636},{"lat":40.7678203,"lng":-73.95715636},{"lat":40.76737066,"lng":-73.957606},{"lat":40.76737066,"lng":-73.95940456000001},{"lat":40.7678203,"lng":-73.95985420000001},{"lat":40.768719579999996,"lng":-73.95895492000001},{"lat":40.77051814,"lng":-73.95895492000001},{"lat":40.77096778,"lng":-73.95850528},{"lat":40.7700685,"lng":-73.957606}],"holes":[]},{"shell":[{"lat":40.7161117,"lng":-73.95670672},{"lat":40.717460620000004,"lng":-73.95805564},{"lat":40.717910260000004,"lng":-73.957606},{"lat":40.717910260000004,"lng":-73.95580744},{"lat":40.717460620000004,"lng":-73.9553578},{"lat":40.7161117,"lng":-73.95670672}],"holes":[]},{"shell":[{"lat":40.77501454,"lng":-73.97514196},{"lat":40.773665619999996,"lng":-73.97649088},{"lat":40.77501454,"lng":-73.9778398},{"lat":40.7768131,"lng":-73.9778398},{"lat":40.77726274,"lng":-73.97739016},{"lat":40.77636346,"lng":-73.97649088},{"lat":40.77636346,"lng":-73.97469232},{"lat":40.77591382,"lng":-73.97424268},{"lat":40.77501454,"lng":-73.97514196}],"holes":[]}],"properties":{}},{"search_id":"casey_25_min","shapes":[{"shell":[{"lat":40.73177064000001,"lng":-74.06310252},{"lat":40.733119560000006,"lng":-74.06445144},{"lat":40.734468480000004,"lng":-74.06310252},{"lat":40.733119560000006,"lng":-74.0617536},{"lat":40.73177064000001,"lng":-74.06310252}],"holes":[]},{"shell":[{"lat":40.72142892000001,"lng":-74.04736512},{"lat":40.722777840000006,"lng":-74.0460162},{"lat":40.722777840000006,"lng":-74.04241908},{"lat":40.72187856000001,"lng":-74.0415198},{"lat":40.72187856000001,"lng":-74.04062052},{"lat":40.72142892000001,"lng":-74.04017088},{"lat":40.72052964000001,"lng":-74.04017088},{"lat":40.719180720000004,"lng":-74.03882195999999},{"lat":40.719180720000004,"lng":-74.03792268},{"lat":40.722777840000006,"lng":-74.03432556},{"lat":40.722777840000006,"lng":-74.032527},{"lat":40.72187856000001,"lng":-74.03162772},{"lat":40.72187856000001,"lng":-74.02982916},{"lat":40.719630360000004,"lng":-74.02758096},{"lat":40.718731080000005,"lng":-74.02758096},{"lat":40.718281440000005,"lng":-74.0280306},{"lat":40.718281440000005,"lng":-74.02892987999999},{"lat":40.717831800000006,"lng":-74.02937951999999},{"lat":40.71693252000001,"lng":-74.02937951999999},{"lat":40.713335400000005,"lng":-74.0257824},{"lat":40.71198648000001,"lng":-74.02713132},{"lat":40.71198648000001,"lng":-74.032527},{"lat":40.71108720000001,"lng":-74.03342628},{"lat":40.71108720000001,"lng":-74.03432556},{"lat":40.71153684000001,"lng":-74.0347752},{"lat":40.71243612000001,"lng":-74.0347752},{"lat":40.71648288000001,"lng":-74.03882195999999},{"lat":40.71648288000001,"lng":-74.04421764},{"lat":40.719630360000004,"lng":-74.04736512},{"lat":40.72142892000001,"lng":-74.04736512}],"holes":[]},{"shell":[{"lat":40.70704044000001,"lng":-74.0257824},{"lat":40.708389360000005,"lng":-74.02443348},{"lat":40.708389360000005,"lng":-74.0235342},{"lat":40.708839000000005,"lng":-74.02308456},{"lat":40.70973828,"lng":-74.02308456},{"lat":40.71243612000001,"lng":-74.02038672},{"lat":40.714234680000004,"lng":-74.02038672},{"lat":40.71513396000001,"lng":-74.021286},{"lat":40.71603324000001,"lng":-74.02038672},{"lat":40.719630360000004,"lng":-74.02038672},{"lat":40.72008000000001,"lng":-74.01993708},{"lat":40.72008000000001,"lng":-74.01903779999999},{"lat":40.723227480000006,"lng":-74.01589032},{"lat":40.72502604000001,"lng":-74.01589032},{"lat":40.72592532000001,"lng":-74.01499104},{"lat":40.72637496000001,"lng":-74.01544068},{"lat":40.72637496000001,"lng":-74.01723924},{"lat":40.72682460000001,"lng":-74.01768888},{"lat":40.73042172000001,"lng":-74.01768888},{"lat":40.73222028000001,"lng":-74.01948743999999},{"lat":40.73491812000001,"lng":-74.01948743999999},{"lat":40.73626704000001,"lng":-74.01813852},{"lat":40.73626704000001,"lng":-74.01633996},{"lat":40.73671668000001,"lng":-74.01589032},{"lat":40.73761596000001,"lng":-74.01589032},{"lat":40.738065600000006,"lng":-74.01544068},{"lat":40.738065600000006,"lng":-74.01274284},{"lat":40.738515240000005,"lng":-74.0122932},{"lat":40.74031380000001,"lng":-74.0122932},{"lat":40.74121308000001,"lng":-74.01319248},{"lat":40.74166272000001,"lng":-74.01274284},{"lat":40.74166272000001,"lng":-74.01184356},{"lat":40.74211236000001,"lng":-74.01139392},{"lat":40.743910920000005,"lng":-74.01139392},{"lat":40.74436056000001,"lng":-74.01184356},{"lat":40.74436056000001,"lng":-74.01274284},{"lat":40.74570948000001,"lng":-74.01409176},{"lat":40.74615912000001,"lng":-74.01364212},{"lat":40.74615912000001,"lng":-74.01184356},{"lat":40.74660876000001,"lng":-74.01139392},{"lat":40.748407320000005,"lng":-74.01139392},{"lat":40.74930660000001,"lng":-74.01049464},{"lat":40.75020588000001,"lng":-74.01049464},{"lat":40.75065552000001,"lng":-74.01094428},{"lat":40.75065552000001,"lng":-74.01184356},{"lat":40.75110516000001,"lng":-74.0122932},{"lat":40.75245408000001,"lng":-74.01094428},{"lat":40.75245408000001,"lng":-74.010045},{"lat":40.75470228000001,"lng":-74.0077968},{"lat":40.75560156000001,"lng":-74.00869607999999},{"lat":40.75919868000001,"lng":-74.00869607999999},{"lat":40.75964832000001,"lng":-74.00824644},{"lat":40.75964832000001,"lng":-74.00464932},{"lat":40.76054760000001,"lng":-74.00375004},{"lat":40.76054760000001,"lng":-74.00285076},{"lat":40.76009796000001,"lng":-74.00240112},{"lat":40.75919868000001,"lng":-74.00240112},{"lat":40.758749040000005,"lng":-74.00195148},{"lat":40.758749040000005,"lng":-74.0010522},{"lat":40.75919868000001,"lng":-74.00060256},{"lat":40.76099724000001,"lng":-74.00060256},{"lat":40.76189652000001,"lng":-74.00150184},{"lat":40.76234616000001,"lng":-74.0010522},{"lat":40.76234616000001,"lng":-74.00015292},{"lat":40.763245440000006,"lng":-73.99925363999999},{"lat":40.763245440000006,"lng":-73.9965558},{"lat":40.762795800000006,"lng":-73.99610616},{"lat":40.76189652000001,"lng":-73.99610616},{"lat":40.76144688000001,"lng":-73.99565652},{"lat":40.76189652000001,"lng":-73.99520688},{"lat":40.76459436000001,"lng":-73.99520688},{"lat":40.76549364000001,"lng":-73.99610616},{"lat":40.76774184000001,"lng":-73.99385796},{"lat":40.76774184000001,"lng":-73.99026084},{"lat":40.76909076000001,"lng":-73.98891191999999},{"lat":40.77088932000001,"lng":-73.98891191999999},{"lat":40.77268788000001,"lng":-73.99071048},{"lat":40.773137520000006,"lng":-73.99026084},{"lat":40.77223824000001,"lng":-73.98936156},{"lat":40.77223824000001,"lng":-73.987563},{"lat":40.77358716000001,"lng":-73.98621408},{"lat":40.77538572000001,"lng":-73.98621408},{"lat":40.77583536000001,"lng":-73.98576444},{"lat":40.77583536000001,"lng":-73.98486516},{"lat":40.77628500000001,"lng":-73.98441552},{"lat":40.77718428000001,"lng":-73.98441552},{"lat":40.77898284000001,"lng":-73.98261696},{"lat":40.78078140000001,"lng":-73.98261696},{"lat":40.78123104000001,"lng":-73.98216732},{"lat":40.78123104000001,"lng":-73.98036876},{"lat":40.78078140000001,"lng":-73.97991912},{"lat":40.77988212000001,"lng":-73.97991912},{"lat":40.77943248000001,"lng":-73.97946948},{"lat":40.77943248000001,"lng":-73.9785702},{"lat":40.77853320000001,"lng":-73.97767092},{"lat":40.77853320000001,"lng":-73.97677164},{"lat":40.77898284000001,"lng":-73.976322},{"lat":40.78168068000001,"lng":-73.976322},{"lat":40.78257996000001,"lng":-73.97542272},{"lat":40.78437852000001,"lng":-73.97542272},{"lat":40.78482816000001,"lng":-73.97497308},{"lat":40.78482816000001,"lng":-73.97227524},{"lat":40.78527780000001,"lng":-73.9718256},{"lat":40.78617708000001,"lng":-73.9718256},{"lat":40.78707636000001,"lng":-73.97272488},{"lat":40.78842528000001,"lng":-73.97137596},{"lat":40.787975640000006,"lng":-73.97092632},{"lat":40.78707636000001,"lng":-73.97092632},{"lat":40.78662672000001,"lng":-73.97047668},{"lat":40.78662672000001,"lng":-73.9695774},{"lat":40.78842528000001,"lng":-73.96777884},{"lat":40.787975640000006,"lng":-73.9673292},{"lat":40.78617708000001,"lng":-73.9673292},{"lat":40.78437852000001,"lng":-73.96553064},{"lat":40.78392888000001,"lng":-73.96598028},{"lat":40.78392888000001,"lng":-73.96777884},{"lat":40.78347924000001,"lng":-73.96822848},{"lat":40.78168068000001,"lng":-73.96822848},{"lat":40.78078140000001,"lng":-73.9673292},{"lat":40.78033176000001,"lng":-73.96777884},{"lat":40.78033176000001,"lng":-73.96867811999999},{"lat":40.77988212000001,"lng":-73.96912775999999},{"lat":40.77898284000001,"lng":-73.96912775999999},{"lat":40.77763392000001,"lng":-73.97047668},{"lat":40.77763392000001,"lng":-73.97227524},{"lat":40.77718428000001,"lng":-73.97272488},{"lat":40.77628500000001,"lng":-73.97272488},{"lat":40.77583536000001,"lng":-73.97227524},{"lat":40.77583536000001,"lng":-73.97137596},{"lat":40.77538572000001,"lng":-73.97092632},{"lat":40.77358716000001,"lng":-73.97092632},{"lat":40.77178860000001,"lng":-73.97272488},{"lat":40.76729220000001,"lng":-73.97272488},{"lat":40.76684256000001,"lng":-73.97227524},{"lat":40.76684256000001,"lng":-73.97137596},{"lat":40.76774184000001,"lng":-73.97047668},{"lat":40.76774184000001,"lng":-73.96867811999999},{"lat":40.76864112000001,"lng":-73.96777884},{"lat":40.76864112000001,"lng":-73.96687956},{"lat":40.77043968000001,"lng":-73.965081},{"lat":40.77043968000001,"lng":-73.96418172},{"lat":40.77223824000001,"lng":-73.96238316},{"lat":40.77133896000001,"lng":-73.96148388},{"lat":40.77133896000001,"lng":-73.9605846},{"lat":40.77223824000001,"lng":-73.95968532},{"lat":40.77223824000001,"lng":-73.95698748},{"lat":40.77448644000001,"lng":-73.95473928},{"lat":40.778083560000006,"lng":-73.95473928},{"lat":40.77988212000001,"lng":-73.95653784},{"lat":40.78033176000001,"lng":-73.9560882},{"lat":40.77943248000001,"lng":-73.95518892},{"lat":40.77943248000001,"lng":-73.95339036},{"lat":40.78033176000001,"lng":-73.95249108},{"lat":40.78033176000001,"lng":-73.95069252},{"lat":40.78168068000001,"lng":-73.9493436},{"lat":40.78347924000001,"lng":-73.9493436},{"lat":40.78437852000001,"lng":-73.95024288},{"lat":40.78662672000001,"lng":-73.94799468},{"lat":40.78662672000001,"lng":-73.94619612},{"lat":40.78437852000001,"lng":-73.94394792},{"lat":40.783029600000006,"lng":-73.94529684},{"lat":40.783029600000006,"lng":-73.9470954},{"lat":40.78168068000001,"lng":-73.94844432},{"lat":40.77988212000001,"lng":-73.94844432},{"lat":40.77898284000001,"lng":-73.94754504},{"lat":40.77718428000001,"lng":-73.94754504},{"lat":40.77493608000001,"lng":-73.94979324},{"lat":40.77493608000001,"lng":-73.95339036},{"lat":40.77358716000001,"lng":-73.95473928},{"lat":40.77178860000001,"lng":-73.95473928},{"lat":40.76999004000001,"lng":-73.95294072},{"lat":40.768191480000006,"lng":-73.95294072},{"lat":40.76729220000001,"lng":-73.95204144},{"lat":40.76594328000001,"lng":-73.95339036},{"lat":40.76594328000001,"lng":-73.95698748},{"lat":40.76414472000001,"lng":-73.95878603999999},{"lat":40.76414472000001,"lng":-73.95968532},{"lat":40.763695080000005,"lng":-73.96013496},{"lat":40.762795800000006,"lng":-73.96013496},{"lat":40.76189652000001,"lng":-73.95923568},{"lat":40.76144688000001,"lng":-73.95968532},{"lat":40.76144688000001,"lng":-73.96148388},{"lat":40.76054760000001,"lng":-73.96238316},{"lat":40.76054760000001,"lng":-73.96328244},{"lat":40.76009796000001,"lng":-73.96373208},{"lat":40.75740012000001,"lng":-73.96373208},{"lat":40.75650084000001,"lng":-73.9628328},{"lat":40.75425264000001,"lng":-73.965081},{"lat":40.75515192000001,"lng":-73.96598028},{"lat":40.75515192000001,"lng":-73.96777884},{"lat":40.753353360000006,"lng":-73.9695774},{"lat":40.753353360000006,"lng":-73.97047668},{"lat":40.752903720000006,"lng":-73.97092632},{"lat":40.75200444000001,"lng":-73.97092632},{"lat":40.74930660000001,"lng":-73.96822848},{"lat":40.74750804000001,"lng":-73.96822848},{"lat":40.74705840000001,"lng":-73.96867811999999},{"lat":40.74705840000001,"lng":-73.97047668},{"lat":40.747957680000006,"lng":-73.97137596},{"lat":40.747957680000006,"lng":-73.97227524},{"lat":40.74750804000001,"lng":-73.97272488},{"lat":40.74660876000001,"lng":-73.97272488},{"lat":40.743910920000005,"lng":-73.97002704},{"lat":40.743011640000006,"lng":-73.97002704},{"lat":40.74166272000001,"lng":-73.97137596},{"lat":40.74166272000001,"lng":-73.9740738},{"lat":40.74031380000001,"lng":-73.97542272},{"lat":40.73941452000001,"lng":-73.97542272},{"lat":40.738964880000005,"lng":-73.97497308},{"lat":40.738964880000005,"lng":-73.97317452},{"lat":40.738515240000005,"lng":-73.97272488},{"lat":40.733119560000006,"lng":-73.97272488},{"lat":40.73222028000001,"lng":-73.97362416},{"lat":40.73087136000001,"lng":-73.97227524},{"lat":40.73177064000001,"lng":-73.97137596},{"lat":40.73177064000001,"lng":-73.9695774},{"lat":40.728623160000005,"lng":-73.96642992},{"lat":40.72682460000001,"lng":-73.96642992},{"lat":40.72637496000001,"lng":-73.96687956},{"lat":40.72637496000001,"lng":-73.96867811999999},{"lat":40.723677120000005,"lng":-73.97137596},{"lat":40.723677120000005,"lng":-73.97317452},{"lat":40.72232820000001,"lng":-73.97452344},{"lat":40.72052964000001,"lng":-73.97452344},{"lat":40.719630360000004,"lng":-73.97362416},{"lat":40.717831800000006,"lng":-73.97362416},{"lat":40.71603324000001,"lng":-73.97542272},{"lat":40.714234680000004,"lng":-73.97542272},{"lat":40.71243612000001,"lng":-73.97722128},{"lat":40.71063756000001,"lng":-73.97722128},{"lat":40.71018792000001,"lng":-73.97767092},{"lat":40.71018792000001,"lng":-73.97946948},{"lat":40.71108720000001,"lng":-73.98036876},{"lat":40.70973828,"lng":-73.98171768},{"lat":40.707939720000006,"lng":-73.98171768},{"lat":40.70524188,"lng":-73.97901983999999},{"lat":40.703443320000005,"lng":-73.97901983999999},{"lat":40.70254404000001,"lng":-73.97991912},{"lat":40.70209440000001,"lng":-73.97946948},{"lat":40.70209440000001,"lng":-73.97767092},{"lat":40.70074548000001,"lng":-73.976322},{"lat":40.6998462,"lng":-73.976322},{"lat":40.698946920000004,"lng":-73.97722128},{"lat":40.698047640000006,"lng":-73.97722128},{"lat":40.69714836000001,"lng":-73.97812056},{"lat":40.694450520000004,"lng":-73.97812056},{"lat":40.694000880000004,"lng":-73.97767092},{"lat":40.694000880000004,"lng":-73.97587236},{"lat":40.693551240000005,"lng":-73.97542272},{"lat":40.69175268000001,"lng":-73.97542272},{"lat":40.69130304000001,"lng":-73.97587236},{"lat":40.69130304000001,"lng":-73.97677164},{"lat":40.69085340000001,"lng":-73.97722128},{"lat":40.69040376,"lng":-73.97677164},{"lat":40.69040376,"lng":-73.97587236},{"lat":40.68995412,"lng":-73.97542272},{"lat":40.689054840000004,"lng":-73.97542272},{"lat":40.687705920000006,"lng":-73.9740738},{"lat":40.687705920000006,"lng":-73.97317452},{"lat":40.68725628000001,"lng":-73.97272488},{"lat":40.68545772,"lng":-73.97272488},{"lat":40.682759880000006,"lng":-73.97002704},{"lat":40.68231024000001,"lng":-73.97047668},{"lat":40.68231024000001,"lng":-73.97227524},{"lat":40.68096132000001,"lng":-73.97362416},{"lat":40.68006204,"lng":-73.97362416},{"lat":40.6796124,"lng":-73.97317452},{"lat":40.6796124,"lng":-73.97227524},{"lat":40.68051168,"lng":-73.97137596},{"lat":40.67736420000001,"lng":-73.96822848},{"lat":40.67556564,"lng":-73.96822848},{"lat":40.67466636,"lng":-73.9673292},{"lat":40.673317440000005,"lng":-73.96867811999999},{"lat":40.673317440000005,"lng":-73.97047668},{"lat":40.67241816000001,"lng":-73.97137596},{"lat":40.674216720000004,"lng":-73.97317452},{"lat":40.674216720000004,"lng":-73.97497308},{"lat":40.67556564,"lng":-73.976322},{"lat":40.67646492000001,"lng":-73.97542272},{"lat":40.67736420000001,"lng":-73.97542272},{"lat":40.677813840000006,"lng":-73.97587236},{"lat":40.677813840000006,"lng":-73.97677164},{"lat":40.678263480000005,"lng":-73.97722128},{"lat":40.68006204,"lng":-73.97722128},{"lat":40.68051168,"lng":-73.97767092},{"lat":40.68051168,"lng":-73.9785702},{"lat":40.68141096000001,"lng":-73.97946948},{"lat":40.68141096000001,"lng":-73.98036876},{"lat":40.68186060000001,"lng":-73.9808184},{"lat":40.682759880000006,"lng":-73.9808184},{"lat":40.683209520000005,"lng":-73.98126804},{"lat":40.683209520000005,"lng":-73.98576444},{"lat":40.68231024000001,"lng":-73.98666372},{"lat":40.68231024000001,"lng":-73.987563},{"lat":40.68141096000001,"lng":-73.98846228},{"lat":40.68141096000001,"lng":-73.98936156},{"lat":40.68096132000001,"lng":-73.9898112},{"lat":40.68006204,"lng":-73.9898112},{"lat":40.679162760000004,"lng":-73.98891191999999},{"lat":40.678263480000005,"lng":-73.98891191999999},{"lat":40.677813840000006,"lng":-73.98936156},{"lat":40.677813840000006,"lng":-73.99295868},{"lat":40.67646492000001,"lng":-73.9943076},{"lat":40.673767080000005,"lng":-73.9943076},{"lat":40.673317440000005,"lng":-73.99385796},{"lat":40.673317440000005,"lng":-73.99295868},{"lat":40.672867800000006,"lng":-73.99250904},{"lat":40.67196852000001,"lng":-73.99250904},{"lat":40.67151888000001,"lng":-73.99295868},{"lat":40.67151888000001,"lng":-73.99475724},{"lat":40.6706196,"lng":-73.99565652},{"lat":40.6706196,"lng":-73.99745508},{"lat":40.67106924,"lng":-73.99790472},{"lat":40.673767080000005,"lng":-73.99790472},{"lat":40.67466636,"lng":-73.99880399999999},{"lat":40.67556564,"lng":-73.99880399999999},{"lat":40.67646492000001,"lng":-73.99790472},{"lat":40.67736420000001,"lng":-73.99880399999999},{"lat":40.679162760000004,"lng":-73.99880399999999},{"lat":40.6796124,"lng":-73.99925363999999},{"lat":40.6796124,"lng":-74.00015292},{"lat":40.68006204,"lng":-74.00060256},{"lat":40.68186060000001,"lng":-74.00060256},{"lat":40.68231024000001,"lng":-74.00015292},{"lat":40.68231024000001,"lng":-73.99925363999999},{"lat":40.683209520000005,"lng":-73.99835436},{"lat":40.683209520000005,"lng":-73.9965558},{"lat":40.683659160000005,"lng":-73.99610616},{"lat":40.68455844,"lng":-73.99610616},{"lat":40.68635700000001,"lng":-73.99790472},{"lat":40.688155560000006,"lng":-73.99790472},{"lat":40.688605200000005,"lng":-73.99745508},{"lat":40.688605200000005,"lng":-73.99565652},{"lat":40.689054840000004,"lng":-73.99520688},{"lat":40.69175268000001,"lng":-73.99520688},{"lat":40.69220232000001,"lng":-73.99565652},{"lat":40.69220232000001,"lng":-73.9965558},{"lat":40.692651960000006,"lng":-73.99700544},{"lat":40.693551240000005,"lng":-73.99700544},{"lat":40.694450520000004,"lng":-73.99790472},{"lat":40.69624908000001,"lng":-73.99790472},{"lat":40.69714836000001,"lng":-73.99700544},{"lat":40.698047640000006,"lng":-73.99790472},{"lat":40.6998462,"lng":-73.99790472},{"lat":40.70209440000001,"lng":-74.00015292},{"lat":40.70209440000001,"lng":-74.00195148},{"lat":40.702993680000006,"lng":-74.00285076},{"lat":40.70254404000001,"lng":-74.0033004},{"lat":40.70074548000001,"lng":-74.0033004},{"lat":40.70029584,"lng":-74.00375004},{"lat":40.70029584,"lng":-74.00914571999999},{"lat":40.6998462,"lng":-74.00959536},{"lat":40.698047640000006,"lng":-74.00959536},{"lat":40.69669872000001,"lng":-74.01094428},{"lat":40.69669872000001,"lng":-74.0145414},{"lat":40.697598000000006,"lng":-74.01544068},{"lat":40.697598000000006,"lng":-74.01633996},{"lat":40.698497280000005,"lng":-74.01723924},{"lat":40.698497280000005,"lng":-74.02263492},{"lat":40.698946920000004,"lng":-74.02308456},{"lat":40.6998462,"lng":-74.02308456},{"lat":40.70029584,"lng":-74.0235342},{"lat":40.70029584,"lng":-74.02443348},{"lat":40.70074548000001,"lng":-74.02488312},{"lat":40.70254404000001,"lng":-74.02488312},{"lat":40.703443320000005,"lng":-74.0257824},{"lat":40.70704044000001,"lng":-74.0257824}],"holes":[[{"lat":40.70659080000001,"lng":-73.99295868},{"lat":40.70659080000001,"lng":-73.98936156},{"lat":40.70704044000001,"lng":-73.98891191999999},{"lat":40.709288640000004,"lng":-73.99116012},{"lat":40.70704044000001,"lng":-73.99340832},{"lat":40.70659080000001,"lng":-73.99295868}],[{"lat":40.72637496000001,"lng":-73.97497308},{"lat":40.72637496000001,"lng":-73.97317452},{"lat":40.72682460000001,"lng":-73.97272488},{"lat":40.728173520000006,"lng":-73.9740738},{"lat":40.72682460000001,"lng":-73.97542272},{"lat":40.72637496000001,"lng":-73.97497308}]]},{"shell":[{"lat":40.68051168,"lng":-74.00464932},{"lat":40.68096132000001,"lng":-74.00509896},{"lat":40.68141096000001,"lng":-74.00464932},{"lat":40.68141096000001,"lng":-74.00285076},{"lat":40.68096132000001,"lng":-74.00240112},{"lat":40.68006204,"lng":-74.00240112},{"lat":40.6796124,"lng":-74.00285076},{"lat":40.6796124,"lng":-74.00375004},{"lat":40.68051168,"lng":-74.00464932}],"holes":[]},{"shell":[{"lat":40.71108720000001,"lng":-73.96418172},{"lat":40.71108720000001,"lng":-73.96328244},{"lat":40.71198648000001,"lng":-73.96238316},{"lat":40.71198648000001,"lng":-73.95878603999999},{"lat":40.712885760000006,"lng":-73.95788676},{"lat":40.712885760000006,"lng":-73.95428964},{"lat":40.71243612000001,"lng":-73.95384},{"lat":40.71153684000001,"lng":-73.95384},{"lat":40.70973828,"lng":-73.95204144},{"lat":40.708839000000005,"lng":-73.95294072},{"lat":40.707939720000006,"lng":-73.95294072},{"lat":40.70749008000001,"lng":-73.95249108},{"lat":40.70749008000001,"lng":-73.9515918},{"lat":40.70704044000001,"lng":-73.95114216},{"lat":40.70569152000001,"lng":-73.95249108},{"lat":40.70569152000001,"lng":-73.95428964},{"lat":40.703892960000005,"lng":-73.9560882},{"lat":40.703892960000005,"lng":-73.95968532},{"lat":40.702993680000006,"lng":-73.9605846},{"lat":40.703892960000005,"lng":-73.96148388},{"lat":40.703892960000005,"lng":-73.96238316},{"lat":40.704342600000004,"lng":-73.9628328},{"lat":40.70614116000001,"lng":-73.9628328},{"lat":40.707939720000006,"lng":-73.96463136},{"lat":40.71063756000001,"lng":-73.96463136},{"lat":40.71108720000001,"lng":-73.96418172}],"holes":[]},{"shell":[{"lat":40.68051168,"lng":-73.95428964},{"lat":40.68096132000001,"lng":-73.95473928},{"lat":40.68141096000001,"lng":-73.95428964},{"lat":40.68141096000001,"lng":-73.95249108},{"lat":40.68231024000001,"lng":-73.9515918},{"lat":40.68231024000001,"lng":-73.94979324},{"lat":40.68186060000001,"lng":-73.9493436},{"lat":40.68006204,"lng":-73.9493436},{"lat":40.6796124,"lng":-73.94979324},{"lat":40.6796124,"lng":-73.95339036},{"lat":40.68051168,"lng":-73.95428964}],"holes":[]},{"shell":[{"lat":40.713335400000005,"lng":-73.94664576},{"lat":40.712885760000006,"lng":-73.9470954},{"lat":40.712885760000006,"lng":-73.94889395999999},{"lat":40.71198648000001,"lng":-73.94979324},{"lat":40.712885760000006,"lng":-73.95069252},{"lat":40.712885760000006,"lng":-73.95249108},{"lat":40.713785040000005,"lng":-73.95339036},{"lat":40.713785040000005,"lng":-73.9560882},{"lat":40.71468432,"lng":-73.95698748},{"lat":40.71468432,"lng":-73.95878603999999},{"lat":40.71558360000001,"lng":-73.95968532},{"lat":40.71558360000001,"lng":-73.9605846},{"lat":40.71603324000001,"lng":-73.96103424},{"lat":40.719630360000004,"lng":-73.96103424},{"lat":40.72008000000001,"lng":-73.9605846},{"lat":40.72008000000001,"lng":-73.95788676},{"lat":40.719180720000004,"lng":-73.95698748},{"lat":40.72008000000001,"lng":-73.9560882},{"lat":40.72008000000001,"lng":-73.95339036},{"lat":40.719630360000004,"lng":-73.95294072},{"lat":40.71513396000001,"lng":-73.95294072},{"lat":40.71468432,"lng":-73.95249108},{"lat":40.71468432,"lng":-73.95069252},{"lat":40.71513396000001,"lng":-73.95024288},{"lat":40.71603324000001,"lng":-73.95024288},{"lat":40.71648288000001,"lng":-73.94979324},{"lat":40.71558360000001,"lng":-73.94889395999999},{"lat":40.71558360000001,"lng":-73.94529684},{"lat":40.71648288000001,"lng":-73.94439756},{"lat":40.71603324000001,"lng":-73.94394792},{"lat":40.71513396000001,"lng":-73.94394792},{"lat":40.71468432,"lng":-73.94349828},{"lat":40.71468432,"lng":-73.942599},{"lat":40.71558360000001,"lng":-73.94169972},{"lat":40.71513396000001,"lng":-73.94125008},{"lat":40.714234680000004,"lng":-73.94125008},{"lat":40.713785040000005,"lng":-73.94080044},{"lat":40.713785040000005,"lng":-73.93990116},{"lat":40.71243612000001,"lng":-73.93855223999999},{"lat":40.71018792000001,"lng":-73.94080044},{"lat":40.71108720000001,"lng":-73.94169972},{"lat":40.71108720000001,"lng":-73.94349828},{"lat":40.71243612000001,"lng":-73.9448472},{"lat":40.713335400000005,"lng":-73.9448472},{"lat":40.713785040000005,"lng":-73.94529684},{"lat":40.713785040000005,"lng":-73.94619612},{"lat":40.713335400000005,"lng":-73.94664576}],"holes":[]},{"shell":[{"lat":40.70119512000001,"lng":-73.94169972},{"lat":40.70119512000001,"lng":-73.93990116},{"lat":40.70074548000001,"lng":-73.93945152},{"lat":40.698497280000005,"lng":-73.94169972},{"lat":40.698946920000004,"lng":-73.94214936},{"lat":40.70074548000001,"lng":-73.94214936},{"lat":40.70119512000001,"lng":-73.94169972}],"holes":[]},{"shell":[{"lat":40.697598000000006,"lng":-73.93900187999999},{"lat":40.698047640000006,"lng":-73.93945152},{"lat":40.699396560000004,"lng":-73.9381026},{"lat":40.698497280000005,"lng":-73.93720332},{"lat":40.698497280000005,"lng":-73.93540476},{"lat":40.699396560000004,"lng":-73.93450548},{"lat":40.69714836000001,"lng":-73.93225728},{"lat":40.69490016,"lng":-73.93450548},{"lat":40.69490016,"lng":-73.93630404},{"lat":40.697598000000006,"lng":-73.93900187999999}],"holes":[]},{"shell":[{"lat":40.678713120000005,"lng":-73.93180764},{"lat":40.678713120000005,"lng":-73.93270692},{"lat":40.679162760000004,"lng":-73.93315656},{"lat":40.68051168,"lng":-73.93180764},{"lat":40.679162760000004,"lng":-73.93045872},{"lat":40.678713120000005,"lng":-73.93090836},{"lat":40.678713120000005,"lng":-73.93180764}],"holes":[]},{"shell":[{"lat":40.692651960000006,"lng":-73.92776088},{"lat":40.69220232000001,"lng":-73.92821052},{"lat":40.693551240000005,"lng":-73.92955944},{"lat":40.694000880000004,"lng":-73.9291098},{"lat":40.694000880000004,"lng":-73.92731124},{"lat":40.693551240000005,"lng":-73.9268616},{"lat":40.692651960000006,"lng":-73.92776088}],"holes":[]},{"shell":[{"lat":40.75515192000001,"lng":-73.94080044},{"lat":40.75515192000001,"lng":-73.93990116},{"lat":40.75470228000001,"lng":-73.93945152},{"lat":40.752903720000006,"lng":-73.93945152},{"lat":40.75245408000001,"lng":-73.93990116},{"lat":40.753353360000006,"lng":-73.94080044},{"lat":40.753353360000006,"lng":-73.94169972},{"lat":40.752903720000006,"lng":-73.94214936},{"lat":40.75200444000001,"lng":-73.94214936},{"lat":40.75155480000001,"lng":-73.942599},{"lat":40.75155480000001,"lng":-73.94439756},{"lat":40.75200444000001,"lng":-73.9448472},{"lat":40.752903720000006,"lng":-73.9448472},{"lat":40.753803000000005,"lng":-73.94394792},{"lat":40.75425264000001,"lng":-73.94439756},{"lat":40.75425264000001,"lng":-73.94529684},{"lat":40.75470228000001,"lng":-73.94574648},{"lat":40.75560156000001,"lng":-73.94574648},{"lat":40.75605120000001,"lng":-73.94529684},{"lat":40.75605120000001,"lng":-73.94349828},{"lat":40.75515192000001,"lng":-73.942599},{"lat":40.75515192000001,"lng":-73.94080044}],"holes":[]},{"shell":[{"lat":40.81000800000001,"lng":-73.95249108},{"lat":40.81225620000001,"lng":-73.95473928},{"lat":40.812705840000014,"lng":-73.95428964},{"lat":40.812705840000014,"lng":-73.95249108},{"lat":40.81360512000001,"lng":-73.9515918},{"lat":40.81135692000001,"lng":-73.9493436},{"lat":40.81000800000001,"lng":-73.95069252},{"lat":40.81000800000001,"lng":-73.95249108}],"holes":[]},{"shell":[{"lat":40.80641088000001,"lng":-73.95518892},{"lat":40.80506196000001,"lng":-73.95384},{"lat":40.80371304000001,"lng":-73.95518892},{"lat":40.80416268000001,"lng":-73.95563856},{"lat":40.80596124000001,"lng":-73.95563856},{"lat":40.80641088000001,"lng":-73.95518892}],"holes":[]},{"shell":[{"lat":40.79606916000001,"lng":-73.95923568},{"lat":40.79472024000001,"lng":-73.9605846},{"lat":40.79606916000001,"lng":-73.96193352},{"lat":40.79651880000001,"lng":-73.96148388},{"lat":40.79651880000001,"lng":-73.95968532},{"lat":40.79606916000001,"lng":-73.95923568}],"holes":[]},{"shell":[{"lat":40.79382096000001,"lng":-73.96418172},{"lat":40.79157276000001,"lng":-73.96193352},{"lat":40.78932456000001,"lng":-73.96418172},{"lat":40.78932456000001,"lng":-73.96598028},{"lat":40.79067348000001,"lng":-73.9673292},{"lat":40.79247204000001,"lng":-73.9673292},{"lat":40.79382096000001,"lng":-73.96598028},{"lat":40.79382096000001,"lng":-73.96418172}],"holes":[]},{"shell":[{"lat":40.78213032000001,"lng":-73.9785702},{"lat":40.78213032000001,"lng":-73.98036876},{"lat":40.78257996000001,"lng":-73.9808184},{"lat":40.78437852000001,"lng":-73.9808184},{"lat":40.78572744000001,"lng":-73.97946948},{"lat":40.78527780000001,"lng":-73.97901983999999},{"lat":40.78347924000001,"lng":-73.97901983999999},{"lat":40.78257996000001,"lng":-73.97812056},{"lat":40.78213032000001,"lng":-73.9785702}],"holes":[]},{"shell":[{"lat":40.73222028000001,"lng":-74.02398384},{"lat":40.73177064000001,"lng":-74.02443348},{"lat":40.73177064000001,"lng":-74.0280306},{"lat":40.732669920000006,"lng":-74.02892987999999},{"lat":40.732669920000006,"lng":-74.02982916},{"lat":40.733119560000006,"lng":-74.0302788},{"lat":40.734018840000005,"lng":-74.0302788},{"lat":40.734468480000004,"lng":-74.03072844},{"lat":40.734468480000004,"lng":-74.03522484},{"lat":40.73491812000001,"lng":-74.03567448},{"lat":40.73671668000001,"lng":-74.03567448},{"lat":40.73716632000001,"lng":-74.03522484},{"lat":40.73716632000001,"lng":-74.03432556},{"lat":40.738065600000006,"lng":-74.03342628},{"lat":40.738065600000006,"lng":-74.03162772},{"lat":40.738515240000005,"lng":-74.03117808},{"lat":40.73941452000001,"lng":-74.03117808},{"lat":40.73986416000001,"lng":-74.03072844},{"lat":40.73986416000001,"lng":-74.02623204},{"lat":40.738065600000006,"lng":-74.02443348},{"lat":40.738065600000006,"lng":-74.0235342},{"lat":40.73761596000001,"lng":-74.02308456},{"lat":40.73491812000001,"lng":-74.02308456},{"lat":40.734018840000005,"lng":-74.02398384},{"lat":40.73222028000001,"lng":-74.02398384}],"holes":[]},{"shell":[{"lat":40.728623160000005,"lng":-74.03657376},{"lat":40.729072800000004,"lng":-74.03612412},{"lat":40.729072800000004,"lng":-74.03432556},{"lat":40.728173520000006,"lng":-74.03342628},{"lat":40.728173520000006,"lng":-74.03162772},{"lat":40.727723880000006,"lng":-74.03117808},{"lat":40.72502604000001,"lng":-74.03117808},{"lat":40.724576400000004,"lng":-74.03162772},{"lat":40.724576400000004,"lng":-74.03612412},{"lat":40.72502604000001,"lng":-74.03657376},{"lat":40.728623160000005,"lng":-74.03657376}],"holes":[]},{"shell":[{"lat":40.74705840000001,"lng":-73.94529684},{"lat":40.74750804000001,"lng":-73.94574648},{"lat":40.748407320000005,"lng":-73.94574648},{"lat":40.748856960000005,"lng":-73.94529684},{"lat":40.748856960000005,"lng":-73.94439756},{"lat":40.74750804000001,"lng":-73.94304864},{"lat":40.74705840000001,"lng":-73.94349828},{"lat":40.74705840000001,"lng":-73.94529684}],"holes":[]},{"shell":[{"lat":40.74166272000001,"lng":-73.94889395999999},{"lat":40.74166272000001,"lng":-73.95518892},{"lat":40.74211236000001,"lng":-73.95563856},{"lat":40.743011640000006,"lng":-73.95563856},{"lat":40.743461280000005,"lng":-73.95518892},{"lat":40.743461280000005,"lng":-73.95069252},{"lat":40.74256200000001,"lng":-73.94979324},{"lat":40.74256200000001,"lng":-73.94889395999999},{"lat":40.74211236000001,"lng":-73.94844432},{"lat":40.74166272000001,"lng":-73.94889395999999}],"holes":[]},{"shell":[{"lat":40.763245440000006,"lng":-73.9515918},{"lat":40.76189652000001,"lng":-73.95024288},{"lat":40.758299400000006,"lng":-73.95024288},{"lat":40.75515192000001,"lng":-73.95339036},{"lat":40.75605120000001,"lng":-73.95428964},{"lat":40.75605120000001,"lng":-73.9560882},{"lat":40.75515192000001,"lng":-73.95698748},{"lat":40.75560156000001,"lng":-73.95743712},{"lat":40.75740012000001,"lng":-73.95563856},{"lat":40.76009796000001,"lng":-73.95563856},{"lat":40.763245440000006,"lng":-73.95249108},{"lat":40.763245440000006,"lng":-73.9515918}],"holes":[]}],"properties":{}},{"search_id":"casey_30_min","shapes":[{"shell":[{"lat":40.73237964,"lng":-74.06673372},{"lat":40.733728559999996,"lng":-74.06808264},{"lat":40.734178199999995,"lng":-74.067633},{"lat":40.73327892,"lng":-74.06673372},{"lat":40.73327892,"lng":-74.06583444},{"lat":40.733728559999996,"lng":-74.0653848},{"lat":40.734627839999995,"lng":-74.0653848},{"lat":40.73552712,"lng":-74.06628408},{"lat":40.7364264,"lng":-74.06628408},{"lat":40.73687604,"lng":-74.06583444},{"lat":40.73687604,"lng":-74.06313659999999},{"lat":40.73597676,"lng":-74.06223732},{"lat":40.73597676,"lng":-74.06043876},{"lat":40.73193,"lng":-74.056392},{"lat":40.73058108,"lng":-74.05774092},{"lat":40.73058108,"lng":-74.06133804},{"lat":40.728782519999996,"lng":-74.06313659999999},{"lat":40.73237964,"lng":-74.06673372}],"holes":[]},{"shell":[{"lat":40.72698396,"lng":-74.05594236},{"lat":40.7274336,"lng":-74.056392},{"lat":40.728782519999996,"lng":-74.05504308},{"lat":40.72788324,"lng":-74.0541438},{"lat":40.72788324,"lng":-74.04874812},{"lat":40.72608468,"lng":-74.04694956},{"lat":40.72608468,"lng":-74.04335244},{"lat":40.72788324,"lng":-74.04155388},{"lat":40.72788324,"lng":-74.03885604},{"lat":40.728332879999996,"lng":-74.0384064},{"lat":40.728782519999996,"lng":-74.03885604},{"lat":40.728782519999996,"lng":-74.03975532},{"lat":40.729232159999995,"lng":-74.04020496},{"lat":40.73237964,"lng":-74.03705748},{"lat":40.73237964,"lng":-74.03525892},{"lat":40.73327892,"lng":-74.03435964},{"lat":40.73237964,"lng":-74.03346036},{"lat":40.73237964,"lng":-74.0316618},{"lat":40.73282928,"lng":-74.03121216},{"lat":40.733728559999996,"lng":-74.03121216},{"lat":40.734178199999995,"lng":-74.0316618},{"lat":40.734178199999995,"lng":-74.03525892},{"lat":40.73507748,"lng":-74.0361582},{"lat":40.73507748,"lng":-74.03885604},{"lat":40.7364264,"lng":-74.04020496},{"lat":40.73732568,"lng":-74.04020496},{"lat":40.73777532,"lng":-74.03975532},{"lat":40.73777532,"lng":-74.03795676},{"lat":40.73822496,"lng":-74.03750712},{"lat":40.739124239999995,"lng":-74.0384064},{"lat":40.740023519999994,"lng":-74.0384064},{"lat":40.74047316,"lng":-74.03795676},{"lat":40.74047316,"lng":-74.03705748},{"lat":40.74182208,"lng":-74.03570856},{"lat":40.74272136,"lng":-74.03570856},{"lat":40.74586884,"lng":-74.03256108},{"lat":40.7454192,"lng":-74.03211144},{"lat":40.743620639999996,"lng":-74.03211144},{"lat":40.74272136,"lng":-74.03121216},{"lat":40.74182208,"lng":-74.03121216},{"lat":40.74137244,"lng":-74.03076252},{"lat":40.74272136,"lng":-74.0294136},{"lat":40.743620639999996,"lng":-74.0294136},{"lat":40.744070279999995,"lng":-74.02896396},{"lat":40.744070279999995,"lng":-74.02626612},{"lat":40.74272136,"lng":-74.0249172},{"lat":40.74182208,"lng":-74.0249172},{"lat":40.73777532,"lng":-74.02087044},{"lat":40.73777532,"lng":-74.01907188},{"lat":40.73732568,"lng":-74.01862224},{"lat":40.7364264,"lng":-74.01952152},{"lat":40.73552712,"lng":-74.01952152},{"lat":40.73507748,"lng":-74.01907188},{"lat":40.73687604,"lng":-74.01727332},{"lat":40.73687604,"lng":-74.01547476},{"lat":40.73732568,"lng":-74.01502512},{"lat":40.73822496,"lng":-74.01502512},{"lat":40.738674599999996,"lng":-74.01547476},{"lat":40.738674599999996,"lng":-74.01727332},{"lat":40.739124239999995,"lng":-74.01772296},{"lat":40.7409228,"lng":-74.01772296},{"lat":40.74182208,"lng":-74.01682368},{"lat":40.74721776,"lng":-74.01682368},{"lat":40.74811704,"lng":-74.0159244},{"lat":40.75171416,"lng":-74.0159244},{"lat":40.75261344,"lng":-74.01502512},{"lat":40.754411999999995,"lng":-74.01502512},{"lat":40.75576092,"lng":-74.0136762},{"lat":40.75576092,"lng":-74.01277692},{"lat":40.75621056,"lng":-74.01232728},{"lat":40.75710984,"lng":-74.01232728},{"lat":40.75800912,"lng":-74.011428},{"lat":40.759807679999994,"lng":-74.011428},{"lat":40.76025732,"lng":-74.01097836},{"lat":40.76025732,"lng":-74.0091798},{"lat":40.7611566,"lng":-74.00828052},{"lat":40.7611566,"lng":-74.00558268},{"lat":40.76160624,"lng":-74.00513304},{"lat":40.76250552,"lng":-74.00513304},{"lat":40.76295516,"lng":-74.0046834},{"lat":40.76295516,"lng":-74.00378412},{"lat":40.763854439999996,"lng":-74.00288484},{"lat":40.763854439999996,"lng":-73.99748916},{"lat":40.764304079999995,"lng":-73.99703952},{"lat":40.76610264,"lng":-73.99703952},{"lat":40.76835084,"lng":-73.99479132},{"lat":40.76835084,"lng":-73.99299275999999},{"lat":40.76745156,"lng":-73.99209348},{"lat":40.7679012,"lng":-73.99164384},{"lat":40.768800479999996,"lng":-73.99164384},{"lat":40.769699759999995,"lng":-73.99254312},{"lat":40.77329688,"lng":-73.99254312},{"lat":40.773746519999996,"lng":-73.99209348},{"lat":40.773746519999996,"lng":-73.99029492},{"lat":40.774196159999995,"lng":-73.98984528},{"lat":40.77779328,"lng":-73.98984528},{"lat":40.77824292,"lng":-73.98939564},{"lat":40.77824292,"lng":-73.98849636},{"lat":40.778692559999996,"lng":-73.98804672},{"lat":40.7813904,"lng":-73.98804672},{"lat":40.78228968,"lng":-73.988946},{"lat":40.78273932,"lng":-73.98849636},{"lat":40.78184004,"lng":-73.98759708},{"lat":40.78184004,"lng":-73.98579852},{"lat":40.78228968,"lng":-73.98534888},{"lat":40.7858868,"lng":-73.98534888},{"lat":40.78678608,"lng":-73.98624816},{"lat":40.78723572,"lng":-73.98579852},{"lat":40.78633644,"lng":-73.98489924},{"lat":40.78633644,"lng":-73.98310068},{"lat":40.788135,"lng":-73.98130212},{"lat":40.788135,"lng":-73.97950356},{"lat":40.79218176,"lng":-73.9754568},{"lat":40.793980319999996,"lng":-73.9754568},{"lat":40.79577888,"lng":-73.97725536},{"lat":40.79622852,"lng":-73.97680572},{"lat":40.79622852,"lng":-73.9732086},{"lat":40.7971278,"lng":-73.97230932},{"lat":40.7971278,"lng":-73.97051076},{"lat":40.79622852,"lng":-73.96961148},{"lat":40.79622852,"lng":-73.96781292},{"lat":40.79532924,"lng":-73.96691364},{"lat":40.79532924,"lng":-73.96601436},{"lat":40.79577888,"lng":-73.96556472},{"lat":40.79667816,"lng":-73.96556472},{"lat":40.79802708,"lng":-73.96691364},{"lat":40.79802708,"lng":-73.9687122},{"lat":40.79847672,"lng":-73.96916184},{"lat":40.80027528,"lng":-73.96916184},{"lat":40.80072492,"lng":-73.9687122},{"lat":40.798926359999996,"lng":-73.96691364},{"lat":40.798926359999996,"lng":-73.96511508},{"lat":40.799825639999995,"lng":-73.9642158},{"lat":40.799825639999995,"lng":-73.96241724},{"lat":40.80117456,"lng":-73.96106832},{"lat":40.80297312,"lng":-73.96106832},{"lat":40.80342276,"lng":-73.96061868},{"lat":40.80342276,"lng":-73.95882012},{"lat":40.803872399999996,"lng":-73.95837048},{"lat":40.80567096,"lng":-73.95837048},{"lat":40.80657024,"lng":-73.95926976},{"lat":40.80701988,"lng":-73.95882012},{"lat":40.80701988,"lng":-73.95702156},{"lat":40.80746952,"lng":-73.95657192},{"lat":40.809268079999995,"lng":-73.95657192},{"lat":40.810167359999994,"lng":-73.9574712},{"lat":40.81556304,"lng":-73.9574712},{"lat":40.81601268,"lng":-73.95702156},{"lat":40.815113399999994,"lng":-73.95612228},{"lat":40.815113399999994,"lng":-73.95432372},{"lat":40.81601268,"lng":-73.95342444},{"lat":40.815113399999994,"lng":-73.95252516},{"lat":40.815113399999994,"lng":-73.9507266},{"lat":40.81556304,"lng":-73.95027696},{"lat":40.8173616,"lng":-73.95027696},{"lat":40.81826088,"lng":-73.95117624},{"lat":40.818710519999996,"lng":-73.9507266},{"lat":40.818710519999996,"lng":-73.94802876},{"lat":40.819160159999996,"lng":-73.94757912},{"lat":40.820059439999994,"lng":-73.94757912},{"lat":40.82095872,"lng":-73.9484784},{"lat":40.82230764,"lng":-73.94712948},{"lat":40.81826088,"lng":-73.94308272},{"lat":40.81646232,"lng":-73.94308272},{"lat":40.81556304,"lng":-73.94218344},{"lat":40.815113399999994,"lng":-73.94263308},{"lat":40.81601268,"lng":-73.94353236},{"lat":40.81601268,"lng":-73.94533092},{"lat":40.814663759999995,"lng":-73.94667984},{"lat":40.81106664,"lng":-73.94667984},{"lat":40.810167359999994,"lng":-73.94578056},{"lat":40.80791916,"lng":-73.94802876},{"lat":40.808818439999996,"lng":-73.94892804},{"lat":40.808818439999996,"lng":-73.9507266},{"lat":40.80746952,"lng":-73.95207552},{"lat":40.80657024,"lng":-73.95207552},{"lat":40.8061206,"lng":-73.95162588},{"lat":40.8061206,"lng":-73.9507266},{"lat":40.80567096,"lng":-73.95027696},{"lat":40.803872399999996,"lng":-73.95027696},{"lat":40.80297312,"lng":-73.94937768},{"lat":40.80252348,"lng":-73.94982732},{"lat":40.80252348,"lng":-73.95162588},{"lat":40.8016242,"lng":-73.95252516},{"lat":40.8016242,"lng":-73.955223},{"lat":40.80117456,"lng":-73.95567264},{"lat":40.80027528,"lng":-73.95567264},{"lat":40.79847672,"lng":-73.95387408},{"lat":40.79802708,"lng":-73.95432372},{"lat":40.79802708,"lng":-73.955223},{"lat":40.79757744,"lng":-73.95567264},{"lat":40.79667816,"lng":-73.95567264},{"lat":40.794879599999994,"lng":-73.9574712},{"lat":40.79308104,"lng":-73.9574712},{"lat":40.7926314,"lng":-73.95792084},{"lat":40.7926314,"lng":-73.95882012},{"lat":40.79218176,"lng":-73.95926976},{"lat":40.79173212,"lng":-73.95882012},{"lat":40.79173212,"lng":-73.95792084},{"lat":40.79128248,"lng":-73.9574712},{"lat":40.789483919999995,"lng":-73.9574712},{"lat":40.788135,"lng":-73.95882012},{"lat":40.788135,"lng":-73.96061868},{"lat":40.78633644,"lng":-73.96241724},{"lat":40.78633644,"lng":-73.96331652},{"lat":40.7858868,"lng":-73.96376616},{"lat":40.78543716,"lng":-73.96331652},{"lat":40.78543716,"lng":-73.96241724},{"lat":40.784987519999994,"lng":-73.9619676},{"lat":40.78228968,"lng":-73.9619676},{"lat":40.78184004,"lng":-73.96151796},{"lat":40.78184004,"lng":-73.96061868},{"lat":40.78273932,"lng":-73.9597194},{"lat":40.78273932,"lng":-73.95432372},{"lat":40.78318896,"lng":-73.95387408},{"lat":40.78678608,"lng":-73.95387408},{"lat":40.788584639999996,"lng":-73.95567264},{"lat":40.789034279999996,"lng":-73.955223},{"lat":40.788135,"lng":-73.95432372},{"lat":40.788135,"lng":-73.9507266},{"lat":40.789034279999996,"lng":-73.94982732},{"lat":40.788135,"lng":-73.94892804},{"lat":40.788135,"lng":-73.94802876},{"lat":40.788584639999996,"lng":-73.94757912},{"lat":40.789483919999995,"lng":-73.94757912},{"lat":40.7903832,"lng":-73.9484784},{"lat":40.79173212,"lng":-73.94712948},{"lat":40.79128248,"lng":-73.94667984},{"lat":40.789483919999995,"lng":-73.94667984},{"lat":40.789034279999996,"lng":-73.9462302},{"lat":40.789034279999996,"lng":-73.94443164},{"lat":40.79083284,"lng":-73.94263308},{"lat":40.7903832,"lng":-73.94218344},{"lat":40.788584639999996,"lng":-73.94218344},{"lat":40.78768536,"lng":-73.94128416},{"lat":40.78678608,"lng":-73.94128416},{"lat":40.78633644,"lng":-73.94083452},{"lat":40.78633644,"lng":-73.93993524},{"lat":40.7858868,"lng":-73.9394856},{"lat":40.784088239999996,"lng":-73.9394856},{"lat":40.783638599999996,"lng":-73.93993524},{"lat":40.783638599999996,"lng":-73.94083452},{"lat":40.78318896,"lng":-73.94128416},{"lat":40.78228968,"lng":-73.94128416},{"lat":40.78184004,"lng":-73.9417338},{"lat":40.78184004,"lng":-73.94263308},{"lat":40.7813904,"lng":-73.94308272},{"lat":40.78049112,"lng":-73.94308272},{"lat":40.779591839999995,"lng":-73.943982},{"lat":40.77599472,"lng":-73.943982},{"lat":40.775095439999994,"lng":-73.94308272},{"lat":40.773746519999996,"lng":-73.94443164},{"lat":40.773746519999996,"lng":-73.94802876},{"lat":40.77284724,"lng":-73.94892804},{"lat":40.77284724,"lng":-73.94982732},{"lat":40.7723976,"lng":-73.95027696},{"lat":40.769699759999995,"lng":-73.95027696},{"lat":40.768800479999996,"lng":-73.94937768},{"lat":40.7679012,"lng":-73.94937768},{"lat":40.76700192,"lng":-73.95027696},{"lat":40.76655228,"lng":-73.94982732},{"lat":40.76655228,"lng":-73.94892804},{"lat":40.76520336,"lng":-73.94757912},{"lat":40.764304079999995,"lng":-73.94757912},{"lat":40.763404799999996,"lng":-73.94667984},{"lat":40.76160624,"lng":-73.94667984},{"lat":40.76025732,"lng":-73.94802876},{"lat":40.76025732,"lng":-73.94982732},{"lat":40.759807679999994,"lng":-73.95027696},{"lat":40.75800912,"lng":-73.95027696},{"lat":40.75710984,"lng":-73.95117624},{"lat":40.75531128,"lng":-73.95117624},{"lat":40.7521638,"lng":-73.95432372},{"lat":40.7521638,"lng":-73.95882012},{"lat":40.753512719999996,"lng":-73.96016904},{"lat":40.754411999999995,"lng":-73.95926976},{"lat":40.75531128,"lng":-73.95926976},{"lat":40.75576092,"lng":-73.95882012},{"lat":40.75576092,"lng":-73.95792084},{"lat":40.75710984,"lng":-73.95657192},{"lat":40.75800912,"lng":-73.95657192},{"lat":40.759807679999994,"lng":-73.95477336},{"lat":40.76025732,"lng":-73.955223},{"lat":40.76025732,"lng":-73.95792084},{"lat":40.758908399999996,"lng":-73.95926976},{"lat":40.75710984,"lng":-73.95926976},{"lat":40.75576092,"lng":-73.96061868},{"lat":40.75576092,"lng":-73.96151796},{"lat":40.75531128,"lng":-73.9619676},{"lat":40.75261344,"lng":-73.9619676},{"lat":40.7521638,"lng":-73.96241724},{"lat":40.7521638,"lng":-73.9642158},{"lat":40.75171416,"lng":-73.96466544},{"lat":40.749915599999994,"lng":-73.96466544},{"lat":40.748566679999996,"lng":-73.96601436},{"lat":40.748566679999996,"lng":-73.96781292},{"lat":40.74721776,"lng":-73.96916184},{"lat":40.743620639999996,"lng":-73.96916184},{"lat":40.74272136,"lng":-73.96826256},{"lat":40.7409228,"lng":-73.97006112},{"lat":40.739124239999995,"lng":-73.97006112},{"lat":40.7364264,"lng":-73.96736328},{"lat":40.734627839999995,"lng":-73.96736328},{"lat":40.733728559999996,"lng":-73.96826256},{"lat":40.73103072,"lng":-73.96826256},{"lat":40.73013144,"lng":-73.96736328},{"lat":40.728332879999996,"lng":-73.96736328},{"lat":40.7274336,"lng":-73.966464},{"lat":40.72563504,"lng":-73.966464},{"lat":40.723836479999996,"lng":-73.96826256},{"lat":40.7229372,"lng":-73.96826256},{"lat":40.72248756,"lng":-73.9687122},{"lat":40.72248756,"lng":-73.96961148},{"lat":40.72203792,"lng":-73.97006112},{"lat":40.718440799999996,"lng":-73.97006112},{"lat":40.71754152,"lng":-73.9709604},{"lat":40.71664224,"lng":-73.9709604},{"lat":40.7161926,"lng":-73.97141004},{"lat":40.7161926,"lng":-73.97230932},{"lat":40.713944399999995,"lng":-73.97455752},{"lat":40.71304512,"lng":-73.97365824},{"lat":40.71124656,"lng":-73.97365824},{"lat":40.709447999999995,"lng":-73.9754568},{"lat":40.70764944,"lng":-73.9754568},{"lat":40.7071998,"lng":-73.97590644},{"lat":40.7071998,"lng":-73.97950356},{"lat":40.70675016,"lng":-73.9799532},{"lat":40.70585088,"lng":-73.97905392},{"lat":40.704052319999995,"lng":-73.97905392},{"lat":40.703602679999996,"lng":-73.97860428},{"lat":40.703602679999996,"lng":-73.97500716},{"lat":40.70315304,"lng":-73.97455752},{"lat":40.70135448,"lng":-73.97455752},{"lat":40.70090484,"lng":-73.97410788},{"lat":40.70090484,"lng":-73.97230932},{"lat":40.7004552,"lng":-73.97185968},{"lat":40.698656639999996,"lng":-73.97185968},{"lat":40.698207,"lng":-73.97141004},{"lat":40.698207,"lng":-73.96961148},{"lat":40.69640844,"lng":-73.96781292},{"lat":40.69640844,"lng":-73.96601436},{"lat":40.69730772,"lng":-73.96511508},{"lat":40.69730772,"lng":-73.9642158},{"lat":40.69685808,"lng":-73.96376616},{"lat":40.69505952,"lng":-73.96376616},{"lat":40.694609879999994,"lng":-73.9642158},{"lat":40.694609879999994,"lng":-73.96511508},{"lat":40.694160239999995,"lng":-73.96556472},{"lat":40.69236168,"lng":-73.96556472},{"lat":40.6914624,"lng":-73.966464},{"lat":40.69101276,"lng":-73.96601436},{"lat":40.69101276,"lng":-73.9642158},{"lat":40.69011348,"lng":-73.96331652},{"lat":40.69011348,"lng":-73.96151796},{"lat":40.69191204,"lng":-73.9597194},{"lat":40.69191204,"lng":-73.95792084},{"lat":40.69011348,"lng":-73.95612228},{"lat":40.69056312,"lng":-73.95567264},{"lat":40.6914624,"lng":-73.95567264},{"lat":40.69191204,"lng":-73.955223},{"lat":40.69191204,"lng":-73.95432372},{"lat":40.69056312,"lng":-73.9529748},{"lat":40.689663839999994,"lng":-73.9529748},{"lat":40.689214199999995,"lng":-73.95342444},{"lat":40.689214199999995,"lng":-73.95432372},{"lat":40.68741564,"lng":-73.95612228},{"lat":40.68741564,"lng":-73.95792084},{"lat":40.68561708,"lng":-73.9597194},{"lat":40.68561708,"lng":-73.96151796},{"lat":40.68651636,"lng":-73.96241724},{"lat":40.684268159999995,"lng":-73.96466544},{"lat":40.683818519999996,"lng":-73.9642158},{"lat":40.683818519999996,"lng":-73.96331652},{"lat":40.683368879999996,"lng":-73.96286688},{"lat":40.6824696,"lng":-73.96376616},{"lat":40.68067104,"lng":-73.96376616},{"lat":40.6802214,"lng":-73.9642158},{"lat":40.6802214,"lng":-73.96511508},{"lat":40.679771759999994,"lng":-73.96556472},{"lat":40.678872479999995,"lng":-73.96466544},{"lat":40.67707392,"lng":-73.96466544},{"lat":40.67662428,"lng":-73.9642158},{"lat":40.67662428,"lng":-73.96241724},{"lat":40.67617464,"lng":-73.9619676},{"lat":40.674376079999995,"lng":-73.9619676},{"lat":40.673926439999995,"lng":-73.96241724},{"lat":40.673926439999995,"lng":-73.9642158},{"lat":40.673476799999996,"lng":-73.96466544},{"lat":40.67212788,"lng":-73.96331652},{"lat":40.67212788,"lng":-73.9597194},{"lat":40.67257752,"lng":-73.95926976},{"lat":40.67527536,"lng":-73.95926976},{"lat":40.675725,"lng":-73.95882012},{"lat":40.675725,"lng":-73.95792084},{"lat":40.67527536,"lng":-73.9574712},{"lat":40.674376079999995,"lng":-73.9574712},{"lat":40.67257752,"lng":-73.95567264},{"lat":40.67077896,"lng":-73.95567264},{"lat":40.66987968,"lng":-73.95477336},{"lat":40.669430039999995,"lng":-73.955223},{"lat":40.669430039999995,"lng":-73.95612228},{"lat":40.66763148,"lng":-73.95792084},{"lat":40.66763148,"lng":-73.95882012},{"lat":40.66808112,"lng":-73.95926976},{"lat":40.668980399999995,"lng":-73.95926976},{"lat":40.669430039999995,"lng":-73.9597194},{"lat":40.669430039999995,"lng":-73.96151796},{"lat":40.6712286,"lng":-73.96331652},{"lat":40.67032932,"lng":-73.9642158},{"lat":40.67032932,"lng":-73.96601436},{"lat":40.6712286,"lng":-73.96691364},{"lat":40.67077896,"lng":-73.96736328},{"lat":40.66987968,"lng":-73.96736328},{"lat":40.669430039999995,"lng":-73.96781292},{"lat":40.669430039999995,"lng":-73.9687122},{"lat":40.67032932,"lng":-73.96961148},{"lat":40.669430039999995,"lng":-73.97051076},{"lat":40.669430039999995,"lng":-73.97230932},{"lat":40.67032932,"lng":-73.9732086},{"lat":40.67032932,"lng":-73.97500716},{"lat":40.67077896,"lng":-73.9754568},{"lat":40.67167824,"lng":-73.9754568},{"lat":40.67212788,"lng":-73.97590644},{"lat":40.67212788,"lng":-73.977705},{"lat":40.67257752,"lng":-73.97815464},{"lat":40.673476799999996,"lng":-73.97815464},{"lat":40.67662428,"lng":-73.98130212},{"lat":40.674825719999994,"lng":-73.98310068},{"lat":40.674825719999994,"lng":-73.98489924},{"lat":40.675725,"lng":-73.98579852},{"lat":40.675725,"lng":-73.9866978},{"lat":40.67617464,"lng":-73.98714744},{"lat":40.67707392,"lng":-73.98714744},{"lat":40.67752356,"lng":-73.98759708},{"lat":40.67617464,"lng":-73.988946},{"lat":40.67167824,"lng":-73.9844496},{"lat":40.66987968,"lng":-73.9844496},{"lat":40.668980399999995,"lng":-73.98355032},{"lat":40.66808112,"lng":-73.98355032},{"lat":40.66763148,"lng":-73.98399996},{"lat":40.66763148,"lng":-73.98489924},{"lat":40.6667322,"lng":-73.98579852},{"lat":40.6667322,"lng":-73.98759708},{"lat":40.66763148,"lng":-73.98849636},{"lat":40.6667322,"lng":-73.98939564},{"lat":40.6667322,"lng":-73.99029492},{"lat":40.668530759999996,"lng":-73.99209348},{"lat":40.668530759999996,"lng":-73.9956906},{"lat":40.66763148,"lng":-73.99658988},{"lat":40.67257752,"lng":-74.00153592},{"lat":40.673476799999996,"lng":-74.00153592},{"lat":40.674376079999995,"lng":-74.0024352},{"lat":40.67617464,"lng":-74.0024352},{"lat":40.67752356,"lng":-74.00378412},{"lat":40.67752356,"lng":-74.0046834},{"lat":40.6779732,"lng":-74.00513304},{"lat":40.679771759999994,"lng":-74.00513304},{"lat":40.6824696,"lng":-74.00783088},{"lat":40.683368879999996,"lng":-74.0069316},{"lat":40.684268159999995,"lng":-74.0069316},{"lat":40.684717799999994,"lng":-74.00648196},{"lat":40.684717799999994,"lng":-74.00108628},{"lat":40.68606672,"lng":-73.99973736},{"lat":40.68786528,"lng":-74.00153592},{"lat":40.689663839999994,"lng":-74.00153592},{"lat":40.69011348,"lng":-74.00108628},{"lat":40.69011348,"lng":-73.99928772},{"lat":40.69056312,"lng":-73.99883808},{"lat":40.6914624,"lng":-73.99973736},{"lat":40.693260959999996,"lng":-73.99973736},{"lat":40.693710599999996,"lng":-74.000187},{"lat":40.693710599999996,"lng":-74.00108628},{"lat":40.694160239999995,"lng":-74.00153592},{"lat":40.694609879999994,"lng":-74.00108628},{"lat":40.694609879999994,"lng":-74.000187},{"lat":40.69505952,"lng":-73.99973736},{"lat":40.69775736,"lng":-73.99973736},{"lat":40.70000556,"lng":-74.00198556},{"lat":40.70000556,"lng":-74.00378412},{"lat":40.698207,"lng":-74.00558268},{"lat":40.698207,"lng":-74.00738124},{"lat":40.69281132,"lng":-74.01277692},{"lat":40.69281132,"lng":-74.01457548},{"lat":40.693710599999996,"lng":-74.01547476},{"lat":40.693710599999996,"lng":-74.01907188},{"lat":40.694160239999995,"lng":-74.01952152},{"lat":40.69505952,"lng":-74.01952152},{"lat":40.69640844,"lng":-74.02087044},{"lat":40.69640844,"lng":-74.022669},{"lat":40.69730772,"lng":-74.02356828},{"lat":40.69730772,"lng":-74.02446756},{"lat":40.699555919999995,"lng":-74.02671576},{"lat":40.709447999999995,"lng":-74.02671576},{"lat":40.709897639999994,"lng":-74.0271654},{"lat":40.708998359999995,"lng":-74.02806468},{"lat":40.708998359999995,"lng":-74.02986324},{"lat":40.709897639999994,"lng":-74.03076252},{"lat":40.709897639999994,"lng":-74.03346036},{"lat":40.71259548,"lng":-74.0361582},{"lat":40.71259548,"lng":-74.03975532},{"lat":40.713494759999996,"lng":-74.0406546},{"lat":40.713494759999996,"lng":-74.04245316},{"lat":40.71259548,"lng":-74.04335244},{"lat":40.714394039999995,"lng":-74.045151},{"lat":40.714394039999995,"lng":-74.04694956},{"lat":40.7161926,"lng":-74.04874812},{"lat":40.7161926,"lng":-74.05054668},{"lat":40.71664224,"lng":-74.05099632},{"lat":40.71754152,"lng":-74.05009704},{"lat":40.719340079999995,"lng":-74.05009704},{"lat":40.72203792,"lng":-74.05279488},{"lat":40.7229372,"lng":-74.05279488},{"lat":40.723386839999996,"lng":-74.05324451999999},{"lat":40.723386839999996,"lng":-74.0541438},{"lat":40.723836479999996,"lng":-74.05459344},{"lat":40.724735759999994,"lng":-74.05369416},{"lat":40.72653432,"lng":-74.05369416},{"lat":40.72698396,"lng":-74.0541438},{"lat":40.72698396,"lng":-74.05594236}],"holes":[[{"lat":40.724286119999995,"lng":-74.02986324},{"lat":40.723836479999996,"lng":-74.03031288},{"lat":40.72203792,"lng":-74.03031288},{"lat":40.72158828,"lng":-74.02986324},{"lat":40.72158828,"lng":-74.02806468},{"lat":40.72248756,"lng":-74.0271654},{"lat":40.72248756,"lng":-74.02356828},{"lat":40.72203792,"lng":-74.02311863999999},{"lat":40.719340079999995,"lng":-74.02311863999999},{"lat":40.718890439999996,"lng":-74.022669},{"lat":40.720689,"lng":-74.02087044},{"lat":40.720689,"lng":-74.01907188},{"lat":40.72248756,"lng":-74.01727332},{"lat":40.72248756,"lng":-74.01547476},{"lat":40.7229372,"lng":-74.01502512},{"lat":40.723836479999996,"lng":-74.0159244},{"lat":40.72563504,"lng":-74.0159244},{"lat":40.7274336,"lng":-74.01772296},{"lat":40.729232159999995,"lng":-74.01772296},{"lat":40.73103072,"lng":-74.01952152},{"lat":40.73282928,"lng":-74.01952152},{"lat":40.73327892,"lng":-74.01997116},{"lat":40.73148036,"lng":-74.02176972},{"lat":40.73148036,"lng":-74.0271654},{"lat":40.73237964,"lng":-74.02806468},{"lat":40.73193,"lng":-74.02851432},{"lat":40.73103072,"lng":-74.02851432},{"lat":40.73058108,"lng":-74.02896396},{"lat":40.73058108,"lng":-74.02986324},{"lat":40.73013144,"lng":-74.03031288},{"lat":40.7274336,"lng":-74.02761504},{"lat":40.72563504,"lng":-74.02761504},{"lat":40.724286119999995,"lng":-74.02896396},{"lat":40.724286119999995,"lng":-74.02986324}],[{"lat":40.774196159999995,"lng":-73.96736328},{"lat":40.773746519999996,"lng":-73.96691364},{"lat":40.773746519999996,"lng":-73.96511508},{"lat":40.774196159999995,"lng":-73.96466544},{"lat":40.776894,"lng":-73.96466544},{"lat":40.77734364,"lng":-73.96511508},{"lat":40.77734364,"lng":-73.96601436},{"lat":40.77599472,"lng":-73.96736328},{"lat":40.774196159999995,"lng":-73.96736328}],[{"lat":40.77644436,"lng":-73.96061868},{"lat":40.77644436,"lng":-73.95882012},{"lat":40.776894,"lng":-73.95837048},{"lat":40.778692559999996,"lng":-73.95837048},{"lat":40.780041479999994,"lng":-73.9597194},{"lat":40.780041479999994,"lng":-73.96151796},{"lat":40.78094076,"lng":-73.96241724},{"lat":40.78094076,"lng":-73.9642158},{"lat":40.78049112,"lng":-73.96466544},{"lat":40.779591839999995,"lng":-73.96466544},{"lat":40.779142199999995,"lng":-73.9642158},{"lat":40.779142199999995,"lng":-73.96331652},{"lat":40.77644436,"lng":-73.96061868}],[{"lat":40.76295516,"lng":-73.95342444},{"lat":40.76295516,"lng":-73.95432372},{"lat":40.76250552,"lng":-73.95477336},{"lat":40.7611566,"lng":-73.95342444},{"lat":40.76250552,"lng":-73.95207552},{"lat":40.76295516,"lng":-73.95252516},{"lat":40.76295516,"lng":-73.95342444}],[{"lat":40.71664224,"lng":-74.02132008},{"lat":40.71799116,"lng":-74.022669},{"lat":40.71664224,"lng":-74.02401792},{"lat":40.71574296,"lng":-74.02401792},{"lat":40.71529332,"lng":-74.02446756},{"lat":40.71529332,"lng":-74.02536684},{"lat":40.714843679999994,"lng":-74.02581648},{"lat":40.713944399999995,"lng":-74.02581648},{"lat":40.713494759999996,"lng":-74.02536684},{"lat":40.713494759999996,"lng":-74.022669},{"lat":40.714843679999994,"lng":-74.02132008},{"lat":40.71664224,"lng":-74.02132008}]]},{"shell":[{"lat":40.65639048,"lng":-74.00513304},{"lat":40.65684012,"lng":-74.0046834},{"lat":40.65684012,"lng":-74.00288484},{"lat":40.6554912,"lng":-74.00153592},{"lat":40.653242999999996,"lng":-74.00378412},{"lat":40.6554912,"lng":-74.00603232},{"lat":40.65639048,"lng":-74.00513304}],"holes":[]},{"shell":[{"lat":40.66043724,"lng":-74.000187},{"lat":40.66178616,"lng":-74.00153592},{"lat":40.663584719999996,"lng":-74.00153592},{"lat":40.66583292,"lng":-73.99928772},{"lat":40.66583292,"lng":-73.9956906},{"lat":40.66538328,"lng":-73.99524096},{"lat":40.66178616,"lng":-73.99524096},{"lat":40.659537959999994,"lng":-73.99748916},{"lat":40.66043724,"lng":-73.99838844},{"lat":40.66043724,"lng":-74.000187}],"holes":[]},{"shell":[{"lat":40.66628256,"lng":-73.98265104},{"lat":40.6667322,"lng":-73.9822014},{"lat":40.6667322,"lng":-73.98130212},{"lat":40.66763148,"lng":-73.98040284},{"lat":40.66763148,"lng":-73.977705},{"lat":40.66718184,"lng":-73.97725536},{"lat":40.66628256,"lng":-73.97725536},{"lat":40.66538328,"lng":-73.97635608},{"lat":40.664483999999995,"lng":-73.97635608},{"lat":40.664034359999995,"lng":-73.97680572},{"lat":40.664034359999995,"lng":-73.977705},{"lat":40.663584719999996,"lng":-73.97815464},{"lat":40.66178616,"lng":-73.97815464},{"lat":40.66088688,"lng":-73.97725536},{"lat":40.6599876,"lng":-73.97725536},{"lat":40.659537959999994,"lng":-73.977705},{"lat":40.659537959999994,"lng":-73.97860428},{"lat":40.658638679999996,"lng":-73.97950356},{"lat":40.658638679999996,"lng":-73.98130212},{"lat":40.659088319999995,"lng":-73.98175176},{"lat":40.66088688,"lng":-73.98175176},{"lat":40.66268544,"lng":-73.9799532},{"lat":40.663135079999996,"lng":-73.98040284},{"lat":40.663135079999996,"lng":-73.98130212},{"lat":40.663584719999996,"lng":-73.98175176},{"lat":40.664483999999995,"lng":-73.98175176},{"lat":40.66538328,"lng":-73.98265104},{"lat":40.66628256,"lng":-73.98265104}],"holes":[]},{"shell":[{"lat":40.64784732,"lng":-73.96601436},{"lat":40.648296959999996,"lng":-73.966464},{"lat":40.65009552,"lng":-73.966464},{"lat":40.6509948,"lng":-73.96736328},{"lat":40.65234372,"lng":-73.96601436},{"lat":40.65234372,"lng":-73.96511508},{"lat":40.65279336,"lng":-73.96466544},{"lat":40.653692639999996,"lng":-73.96466544},{"lat":40.654591919999994,"lng":-73.96556472},{"lat":40.65594084,"lng":-73.9642158},{"lat":40.65594084,"lng":-73.96061868},{"lat":40.65684012,"lng":-73.9597194},{"lat":40.65639048,"lng":-73.95926976},{"lat":40.654591919999994,"lng":-73.95926976},{"lat":40.654142279999995,"lng":-73.9597194},{"lat":40.654142279999995,"lng":-73.96061868},{"lat":40.653692639999996,"lng":-73.96106832},{"lat":40.653242999999996,"lng":-73.96061868},{"lat":40.653242999999996,"lng":-73.9597194},{"lat":40.65279336,"lng":-73.95926976},{"lat":40.649196239999995,"lng":-73.95926976},{"lat":40.64784732,"lng":-73.96061868},{"lat":40.64784732,"lng":-73.96601436}],"holes":[]},{"shell":[{"lat":40.63615668,"lng":-73.96511508},{"lat":40.63615668,"lng":-73.96241724},{"lat":40.63705596,"lng":-73.96151796},{"lat":40.63660632,"lng":-73.96106832},{"lat":40.63480776,"lng":-73.96106832},{"lat":40.634358119999995,"lng":-73.96151796},{"lat":40.634358119999995,"lng":-73.96331652},{"lat":40.633458839999996,"lng":-73.9642158},{"lat":40.63480776,"lng":-73.96556472},{"lat":40.63570704,"lng":-73.96556472},{"lat":40.63615668,"lng":-73.96511508}],"holes":[]},{"shell":[{"lat":40.659088319999995,"lng":-73.95837048},{"lat":40.6577394,"lng":-73.9597194},{"lat":40.6577394,"lng":-73.96151796},{"lat":40.65684012,"lng":-73.96241724},{"lat":40.65728976,"lng":-73.96286688},{"lat":40.658189039999996,"lng":-73.96286688},{"lat":40.658638679999996,"lng":-73.96331652},{"lat":40.658638679999996,"lng":-73.9642158},{"lat":40.6599876,"lng":-73.96556472},{"lat":40.66088688,"lng":-73.96466544},{"lat":40.66178616,"lng":-73.96466544},{"lat":40.6622358,"lng":-73.96511508},{"lat":40.6622358,"lng":-73.96601436},{"lat":40.66268544,"lng":-73.966464},{"lat":40.66538328,"lng":-73.966464},{"lat":40.6667322,"lng":-73.96511508},{"lat":40.6667322,"lng":-73.96151796},{"lat":40.66493364,"lng":-73.9597194},{"lat":40.66493364,"lng":-73.95882012},{"lat":40.664483999999995,"lng":-73.95837048},{"lat":40.663584719999996,"lng":-73.95926976},{"lat":40.66178616,"lng":-73.95926976},{"lat":40.66088688,"lng":-73.95837048},{"lat":40.659088319999995,"lng":-73.95837048}],"holes":[]},{"shell":[{"lat":40.67662428,"lng":-73.95342444},{"lat":40.6802214,"lng":-73.95702156},{"lat":40.6802214,"lng":-73.95882012},{"lat":40.68067104,"lng":-73.95926976},{"lat":40.683818519999996,"lng":-73.95612228},{"lat":40.683818519999996,"lng":-73.955223},{"lat":40.684268159999995,"lng":-73.95477336},{"lat":40.68516744,"lng":-73.95477336},{"lat":40.68561708,"lng":-73.95432372},{"lat":40.68561708,"lng":-73.95342444},{"lat":40.684717799999994,"lng":-73.95252516},{"lat":40.684717799999994,"lng":-73.95162588},{"lat":40.68561708,"lng":-73.9507266},{"lat":40.68561708,"lng":-73.94892804},{"lat":40.68516744,"lng":-73.9484784},{"lat":40.684268159999995,"lng":-73.9484784},{"lat":40.683818519999996,"lng":-73.94802876},{"lat":40.683818519999996,"lng":-73.94712948},{"lat":40.68201996,"lng":-73.94533092},{"lat":40.68201996,"lng":-73.94353236},{"lat":40.68112068,"lng":-73.94263308},{"lat":40.68112068,"lng":-73.9417338},{"lat":40.68157032,"lng":-73.94128416},{"lat":40.6824696,"lng":-73.94128416},{"lat":40.68291924,"lng":-73.94083452},{"lat":40.678872479999995,"lng":-73.93678776},{"lat":40.678422839999996,"lng":-73.9372374},{"lat":40.678422839999996,"lng":-73.93903596},{"lat":40.67662428,"lng":-73.94083452},{"lat":40.67662428,"lng":-73.94263308},{"lat":40.6779732,"lng":-73.943982},{"lat":40.678872479999995,"lng":-73.943982},{"lat":40.679322119999995,"lng":-73.94443164},{"lat":40.678872479999995,"lng":-73.94488128},{"lat":40.6779732,"lng":-73.94488128},{"lat":40.67752356,"lng":-73.94533092},{"lat":40.67752356,"lng":-73.94802876},{"lat":40.67707392,"lng":-73.9484784},{"lat":40.67617464,"lng":-73.9484784},{"lat":40.675725,"lng":-73.94892804},{"lat":40.675725,"lng":-73.95252516},{"lat":40.67662428,"lng":-73.95342444}],"holes":[]},{"shell":[{"lat":40.693710599999996,"lng":-73.94982732},{"lat":40.694160239999995,"lng":-73.95027696},{"lat":40.6959588,"lng":-73.95027696},{"lat":40.69640844,"lng":-73.94982732},{"lat":40.69640844,"lng":-73.94802876},{"lat":40.6959588,"lng":-73.94757912},{"lat":40.693260959999996,"lng":-73.94757912},{"lat":40.69281132,"lng":-73.94802876},{"lat":40.69281132,"lng":-73.94892804},{"lat":40.693710599999996,"lng":-73.94982732}],"holes":[]},{"shell":[{"lat":40.70000556,"lng":-73.9507266},{"lat":40.7004552,"lng":-73.95117624},{"lat":40.70225376,"lng":-73.95117624},{"lat":40.7027034,"lng":-73.95162588},{"lat":40.70180412,"lng":-73.95252516},{"lat":40.70180412,"lng":-73.95432372},{"lat":40.70090484,"lng":-73.955223},{"lat":40.70090484,"lng":-73.95882012},{"lat":40.70000556,"lng":-73.9597194},{"lat":40.70000556,"lng":-73.96241724},{"lat":40.7004552,"lng":-73.96286688},{"lat":40.70135448,"lng":-73.96286688},{"lat":40.7027034,"lng":-73.9642158},{"lat":40.7027034,"lng":-73.96511508},{"lat":40.70315304,"lng":-73.96556472},{"lat":40.70585088,"lng":-73.96556472},{"lat":40.70630052,"lng":-73.96601436},{"lat":40.70630052,"lng":-73.96781292},{"lat":40.708548719999996,"lng":-73.97006112},{"lat":40.71034728,"lng":-73.97006112},{"lat":40.71124656,"lng":-73.96916184},{"lat":40.71214584,"lng":-73.96916184},{"lat":40.71259548,"lng":-73.9687122},{"lat":40.71259548,"lng":-73.96781292},{"lat":40.713494759999996,"lng":-73.96691364},{"lat":40.713494759999996,"lng":-73.96511508},{"lat":40.714843679999994,"lng":-73.96376616},{"lat":40.71664224,"lng":-73.96376616},{"lat":40.71754152,"lng":-73.96466544},{"lat":40.72203792,"lng":-73.96466544},{"lat":40.72248756,"lng":-73.9642158},{"lat":40.72248756,"lng":-73.96151796},{"lat":40.723386839999996,"lng":-73.96061868},{"lat":40.723386839999996,"lng":-73.95882012},{"lat":40.72158828,"lng":-73.95702156},{"lat":40.723386839999996,"lng":-73.955223},{"lat":40.723386839999996,"lng":-73.95342444},{"lat":40.72248756,"lng":-73.95252516},{"lat":40.72248756,"lng":-73.94982732},{"lat":40.72203792,"lng":-73.94937768},{"lat":40.72113864,"lng":-73.94937768},{"lat":40.72023936,"lng":-73.95027696},{"lat":40.719789719999994,"lng":-73.94982732},{"lat":40.719789719999994,"lng":-73.94802876},{"lat":40.72023936,"lng":-73.94757912},{"lat":40.72113864,"lng":-73.94757912},{"lat":40.72158828,"lng":-73.94712948},{"lat":40.72158828,"lng":-73.9462302},{"lat":40.720689,"lng":-73.94533092},{"lat":40.720689,"lng":-73.94353236},{"lat":40.72023936,"lng":-73.94308272},{"lat":40.718440799999996,"lng":-73.94308272},{"lat":40.71799116,"lng":-73.94263308},{"lat":40.718890439999996,"lng":-73.9417338},{"lat":40.718890439999996,"lng":-73.93993524},{"lat":40.71709188,"lng":-73.93813668},{"lat":40.71709188,"lng":-73.9372374},{"lat":40.71664224,"lng":-73.93678776},{"lat":40.71574296,"lng":-73.93678776},{"lat":40.71529332,"lng":-73.93633812},{"lat":40.71529332,"lng":-73.93543884},{"lat":40.714843679999994,"lng":-73.9349892},{"lat":40.713944399999995,"lng":-73.9349892},{"lat":40.71304512,"lng":-73.93408992},{"lat":40.71124656,"lng":-73.93408992},{"lat":40.709447999999995,"lng":-73.93588848},{"lat":40.708998359999995,"lng":-73.93543884},{"lat":40.708998359999995,"lng":-73.93274100000001},{"lat":40.70585088,"lng":-73.92959352},{"lat":40.704501959999995,"lng":-73.93094244},{"lat":40.704501959999995,"lng":-73.93184172},{"lat":40.704052319999995,"lng":-73.93229136},{"lat":40.70315304,"lng":-73.93229136},{"lat":40.7027034,"lng":-73.93274100000001},{"lat":40.703602679999996,"lng":-73.93364028},{"lat":40.703602679999996,"lng":-73.93453956},{"lat":40.704501959999995,"lng":-73.93543884},{"lat":40.704052319999995,"lng":-73.93588848},{"lat":40.70315304,"lng":-73.93588848},{"lat":40.70180412,"lng":-73.93453956},{"lat":40.70180412,"lng":-73.93274100000001},{"lat":40.70135448,"lng":-73.93229136},{"lat":40.7004552,"lng":-73.93229136},{"lat":40.70000556,"lng":-73.93184172},{"lat":40.70000556,"lng":-73.93094244},{"lat":40.699106279999995,"lng":-73.93004316},{"lat":40.699106279999995,"lng":-73.9282446},{"lat":40.69505952,"lng":-73.92419784},{"lat":40.69236168,"lng":-73.92419784},{"lat":40.69191204,"lng":-73.9237482},{"lat":40.69191204,"lng":-73.92284892},{"lat":40.69281132,"lng":-73.92194964},{"lat":40.69281132,"lng":-73.92015108},{"lat":40.69236168,"lng":-73.91970144},{"lat":40.69056312,"lng":-73.91970144},{"lat":40.689663839999994,"lng":-73.91880216},{"lat":40.688764559999996,"lng":-73.91880216},{"lat":40.688314919999996,"lng":-73.91835252},{"lat":40.688314919999996,"lng":-73.91745324},{"lat":40.689214199999995,"lng":-73.91655396},{"lat":40.686966,"lng":-73.91430576},{"lat":40.68606672,"lng":-73.91430576},{"lat":40.68561708,"lng":-73.91385612},{"lat":40.68561708,"lng":-73.91295684},{"lat":40.684717799999994,"lng":-73.91205756},{"lat":40.684717799999994,"lng":-73.910259},{"lat":40.6824696,"lng":-73.9080108},{"lat":40.68112068,"lng":-73.90935972},{"lat":40.68112068,"lng":-73.91115828},{"lat":40.6824696,"lng":-73.9125072},{"lat":40.683368879999996,"lng":-73.9125072},{"lat":40.683818519999996,"lng":-73.91295684},{"lat":40.683818519999996,"lng":-73.91385612},{"lat":40.684717799999994,"lng":-73.9147554},{"lat":40.684717799999994,"lng":-73.91655396},{"lat":40.683818519999996,"lng":-73.91745324},{"lat":40.68561708,"lng":-73.9192518},{"lat":40.68561708,"lng":-73.92284892},{"lat":40.684717799999994,"lng":-73.9237482},{"lat":40.68516744,"lng":-73.92419784},{"lat":40.686966,"lng":-73.92419784},{"lat":40.68786528,"lng":-73.92509712},{"lat":40.688764559999996,"lng":-73.92509712},{"lat":40.689214199999995,"lng":-73.92554676},{"lat":40.689214199999995,"lng":-73.92644604},{"lat":40.68741564,"lng":-73.9282446},{"lat":40.68741564,"lng":-73.93004316},{"lat":40.689663839999994,"lng":-73.93229136},{"lat":40.69056312,"lng":-73.93229136},{"lat":40.69101276,"lng":-73.93274100000001},{"lat":40.69101276,"lng":-73.93543884},{"lat":40.69011348,"lng":-73.93633812},{"lat":40.6914624,"lng":-73.93768704},{"lat":40.69236168,"lng":-73.93768704},{"lat":40.69281132,"lng":-73.93813668},{"lat":40.69281132,"lng":-73.93993524},{"lat":40.69550916,"lng":-73.94263308},{"lat":40.69550916,"lng":-73.94353236},{"lat":40.6959588,"lng":-73.943982},{"lat":40.69685808,"lng":-73.943982},{"lat":40.69775736,"lng":-73.94488128},{"lat":40.699555919999995,"lng":-73.94488128},{"lat":40.70000556,"lng":-73.94533092},{"lat":40.70000556,"lng":-73.94712948},{"lat":40.70090484,"lng":-73.94802876},{"lat":40.70000556,"lng":-73.94892804},{"lat":40.70000556,"lng":-73.9507266}],"holes":[[{"lat":40.708998359999995,"lng":-73.94712948},{"lat":40.708548719999996,"lng":-73.94757912},{"lat":40.7071998,"lng":-73.9462302},{"lat":40.7071998,"lng":-73.94443164},{"lat":40.70764944,"lng":-73.943982},{"lat":40.709447999999995,"lng":-73.943982},{"lat":40.709897639999994,"lng":-73.94443164},{"lat":40.709897639999994,"lng":-73.9462302},{"lat":40.708998359999995,"lng":-73.94712948}]]},{"shell":[{"lat":40.74272136,"lng":-73.95926976},{"lat":40.744519919999995,"lng":-73.95926976},{"lat":40.744969559999994,"lng":-73.95882012},{"lat":40.744969559999994,"lng":-73.95702156},{"lat":40.74586884,"lng":-73.95612228},{"lat":40.74586884,"lng":-73.95342444},{"lat":40.744969559999994,"lng":-73.95252516},{"lat":40.744969559999994,"lng":-73.9507266},{"lat":40.7454192,"lng":-73.95027696},{"lat":40.749016319999996,"lng":-73.95027696},{"lat":40.75036524,"lng":-73.94892804},{"lat":40.75036524,"lng":-73.94802876},{"lat":40.75081488,"lng":-73.94757912},{"lat":40.754411999999995,"lng":-73.94757912},{"lat":40.75621056,"lng":-73.94937768},{"lat":40.75710984,"lng":-73.94937768},{"lat":40.75755948,"lng":-73.94892804},{"lat":40.75755948,"lng":-73.94263308},{"lat":40.75800912,"lng":-73.94218344},{"lat":40.758908399999996,"lng":-73.94218344},{"lat":40.759358039999995,"lng":-73.9417338},{"lat":40.758458759999996,"lng":-73.94083452},{"lat":40.758458759999996,"lng":-73.93903596},{"lat":40.76025732,"lng":-73.9372374},{"lat":40.759807679999994,"lng":-73.93678776},{"lat":40.753512719999996,"lng":-73.93678776},{"lat":40.75306308,"lng":-73.93633812},{"lat":40.75306308,"lng":-73.93543884},{"lat":40.7521638,"lng":-73.93453956},{"lat":40.7521638,"lng":-73.93274100000001},{"lat":40.75171416,"lng":-73.93229136},{"lat":40.75081488,"lng":-73.93229136},{"lat":40.74811704,"lng":-73.9349892},{"lat":40.74721776,"lng":-73.93408992},{"lat":40.74631848,"lng":-73.93408992},{"lat":40.74586884,"lng":-73.93453956},{"lat":40.74586884,"lng":-73.93543884},{"lat":40.74676812,"lng":-73.93633812},{"lat":40.74676812,"lng":-73.93903596},{"lat":40.74631848,"lng":-73.9394856},{"lat":40.7454192,"lng":-73.9394856},{"lat":40.744969559999994,"lng":-73.93993524},{"lat":40.744969559999994,"lng":-73.94083452},{"lat":40.744070279999995,"lng":-73.9417338},{"lat":40.744070279999995,"lng":-73.94353236},{"lat":40.743620639999996,"lng":-73.943982},{"lat":40.74182208,"lng":-73.943982},{"lat":40.74137244,"lng":-73.94443164},{"lat":40.74137244,"lng":-73.94892804},{"lat":40.7409228,"lng":-73.94937768},{"lat":40.740023519999994,"lng":-73.94937768},{"lat":40.738674599999996,"lng":-73.9507266},{"lat":40.739573879999995,"lng":-73.95162588},{"lat":40.739573879999995,"lng":-73.95342444},{"lat":40.74137244,"lng":-73.955223},{"lat":40.74137244,"lng":-73.95882012},{"lat":40.74182208,"lng":-73.95926976},{"lat":40.74272136,"lng":-73.95926976}],"holes":[]},{"shell":[{"lat":40.70540124,"lng":-73.92105036},{"lat":40.70540124,"lng":-73.92284892},{"lat":40.70764944,"lng":-73.92509712},{"lat":40.70809908,"lng":-73.92464748},{"lat":40.7071998,"lng":-73.9237482},{"lat":40.7071998,"lng":-73.92194964},{"lat":40.70585088,"lng":-73.92060072},{"lat":40.70540124,"lng":-73.92105036}],"holes":[]},{"shell":[{"lat":40.678872479999995,"lng":-73.90711152},{"lat":40.6802214,"lng":-73.9057626},{"lat":40.6802214,"lng":-73.90396404},{"lat":40.678872479999995,"lng":-73.90261512000001},{"lat":40.675725,"lng":-73.9057626},{"lat":40.6779732,"lng":-73.9080108},{"lat":40.678872479999995,"lng":-73.90711152}],"holes":[]},{"shell":[{"lat":40.74586884,"lng":-73.89317268},{"lat":40.74631848,"lng":-73.89362232},{"lat":40.74721776,"lng":-73.89362232},{"lat":40.7476674,"lng":-73.89317268},{"lat":40.7476674,"lng":-73.88957556},{"lat":40.74721776,"lng":-73.88912592},{"lat":40.74631848,"lng":-73.88912592},{"lat":40.74586884,"lng":-73.88957556},{"lat":40.74586884,"lng":-73.89047484},{"lat":40.7454192,"lng":-73.89092448},{"lat":40.744519919999995,"lng":-73.89092448},{"lat":40.744070279999995,"lng":-73.89137412},{"lat":40.744519919999995,"lng":-73.89182376},{"lat":40.7454192,"lng":-73.89182376},{"lat":40.74586884,"lng":-73.8922734},{"lat":40.74586884,"lng":-73.89317268}],"holes":[]},{"shell":[{"lat":40.6779732,"lng":-73.91070864},{"lat":40.67752356,"lng":-73.91115828},{"lat":40.67752356,"lng":-73.91295684},{"lat":40.6779732,"lng":-73.91340648},{"lat":40.6802214,"lng":-73.91115828},{"lat":40.678872479999995,"lng":-73.90980936},{"lat":40.6779732,"lng":-73.91070864}],"holes":[]},{"shell":[{"lat":40.679322119999995,"lng":-73.91565468},{"lat":40.678872479999995,"lng":-73.91520504},{"lat":40.67752356,"lng":-73.91655396},{"lat":40.67752356,"lng":-73.91745324},{"lat":40.67707392,"lng":-73.91790288},{"lat":40.67617464,"lng":-73.91790288},{"lat":40.675725,"lng":-73.91835252},{"lat":40.675725,"lng":-73.92194964},{"lat":40.67752356,"lng":-73.9237482},{"lat":40.67752356,"lng":-73.92554676},{"lat":40.67662428,"lng":-73.92644604},{"lat":40.67662428,"lng":-73.9282446},{"lat":40.674825719999994,"lng":-73.93004316},{"lat":40.674825719999994,"lng":-73.93184172},{"lat":40.67527536,"lng":-73.93229136},{"lat":40.67617464,"lng":-73.93229136},{"lat":40.67662428,"lng":-73.93274100000001},{"lat":40.67662428,"lng":-73.93364028},{"lat":40.678872479999995,"lng":-73.93588848},{"lat":40.68067104,"lng":-73.93588848},{"lat":40.68201996,"lng":-73.93453956},{"lat":40.68201996,"lng":-73.93364028},{"lat":40.6824696,"lng":-73.93319064},{"lat":40.683368879999996,"lng":-73.93319064},{"lat":40.683818519999996,"lng":-73.93274100000001},{"lat":40.683818519999996,"lng":-73.93004316},{"lat":40.684717799999994,"lng":-73.92914388},{"lat":40.684717799999994,"lng":-73.9282446},{"lat":40.684268159999995,"lng":-73.92779496},{"lat":40.6824696,"lng":-73.92779496},{"lat":40.68112068,"lng":-73.92644604},{"lat":40.68112068,"lng":-73.92464748},{"lat":40.68067104,"lng":-73.92419784},{"lat":40.679771759999994,"lng":-73.92419784},{"lat":40.679322119999995,"lng":-73.9237482},{"lat":40.679771759999994,"lng":-73.92329856},{"lat":40.68067104,"lng":-73.92329856},{"lat":40.68201996,"lng":-73.92194964},{"lat":40.68201996,"lng":-73.92015108},{"lat":40.6802214,"lng":-73.91835252},{"lat":40.6802214,"lng":-73.91655396},{"lat":40.679322119999995,"lng":-73.91565468}],"holes":[]},{"shell":[{"lat":40.765653,"lng":-73.93004316},{"lat":40.76520336,"lng":-73.92959352},{"lat":40.763854439999996,"lng":-73.93094244},{"lat":40.763854439999996,"lng":-73.93274100000001},{"lat":40.76520336,"lng":-73.93408992},{"lat":40.765653,"lng":-73.93364028},{"lat":40.765653,"lng":-73.93184172},{"lat":40.76655228,"lng":-73.93094244},{"lat":40.765653,"lng":-73.93004316}],"holes":[]},{"shell":[{"lat":40.83219972,"lng":-73.9417338},{"lat":40.83219972,"lng":-73.94083452},{"lat":40.83175008,"lng":-73.94038488},{"lat":40.829951519999995,"lng":-73.94038488},{"lat":40.828602599999996,"lng":-73.9417338},{"lat":40.8308508,"lng":-73.943982},{"lat":40.83219972,"lng":-73.94263308},{"lat":40.83219972,"lng":-73.9417338}],"holes":[]},{"shell":[{"lat":40.82680404,"lng":-73.94533092},{"lat":40.82680404,"lng":-73.94263308},{"lat":40.8263544,"lng":-73.94218344},{"lat":40.82275728,"lng":-73.94218344},{"lat":40.82230764,"lng":-73.94263308},{"lat":40.82320692,"lng":-73.94353236},{"lat":40.82320692,"lng":-73.94533092},{"lat":40.8263544,"lng":-73.9484784},{"lat":40.82680404,"lng":-73.94802876},{"lat":40.82680404,"lng":-73.94533092}],"holes":[]},{"shell":[{"lat":40.77824292,"lng":-74.0136762},{"lat":40.77779328,"lng":-74.01322656},{"lat":40.77599472,"lng":-74.01322656},{"lat":40.77554508,"lng":-74.0136762},{"lat":40.77554508,"lng":-74.01457548},{"lat":40.77599472,"lng":-74.01502512},{"lat":40.77779328,"lng":-74.01502512},{"lat":40.77824292,"lng":-74.01457548},{"lat":40.77824292,"lng":-74.0136762}],"holes":[]},{"shell":[{"lat":40.77284724,"lng":-74.01727332},{"lat":40.77284724,"lng":-74.01637404},{"lat":40.774645799999995,"lng":-74.01457548},{"lat":40.774196159999995,"lng":-74.01412584},{"lat":40.77329688,"lng":-74.01502512},{"lat":40.77149832,"lng":-74.01502512},{"lat":40.7701494,"lng":-74.01637404},{"lat":40.7701494,"lng":-74.01907188},{"lat":40.768800479999996,"lng":-74.0204208},{"lat":40.76700192,"lng":-74.0204208},{"lat":40.76610264,"lng":-74.02132008},{"lat":40.76520336,"lng":-74.02132008},{"lat":40.764753719999995,"lng":-74.02176972},{"lat":40.764753719999995,"lng":-74.022669},{"lat":40.76520336,"lng":-74.02311863999999},{"lat":40.76700192,"lng":-74.02311863999999},{"lat":40.76745156,"lng":-74.02356828},{"lat":40.76745156,"lng":-74.02446756},{"lat":40.7679012,"lng":-74.0249172},{"lat":40.76835084,"lng":-74.02446756},{"lat":40.76835084,"lng":-74.02356828},{"lat":40.768800479999996,"lng":-74.02311863999999},{"lat":40.769699759999995,"lng":-74.02401792},{"lat":40.77149832,"lng":-74.02401792},{"lat":40.77194796,"lng":-74.02356828},{"lat":40.77194796,"lng":-74.02176972},{"lat":40.773746519999996,"lng":-74.01997116},{"lat":40.773746519999996,"lng":-74.01907188},{"lat":40.77284724,"lng":-74.0181726},{"lat":40.77284724,"lng":-74.01727332}],"holes":[]},{"shell":[{"lat":40.7701494,"lng":-74.02626612},{"lat":40.769699759999995,"lng":-74.02581648},{"lat":40.769250119999995,"lng":-74.02626612},{"lat":40.769250119999995,"lng":-74.02806468},{"lat":40.77059904,"lng":-74.0294136},{"lat":40.77104868,"lng":-74.02896396},{"lat":40.77104868,"lng":-74.0271654},{"lat":40.7701494,"lng":-74.02626612}],"holes":[]},{"shell":[{"lat":40.76025732,"lng":-74.03076252},{"lat":40.76025732,"lng":-74.02806468},{"lat":40.76070696,"lng":-74.02761504},{"lat":40.76160624,"lng":-74.02761504},{"lat":40.76205588,"lng":-74.0271654},{"lat":40.76205588,"lng":-74.02536684},{"lat":40.76160624,"lng":-74.0249172},{"lat":40.759807679999994,"lng":-74.0249172},{"lat":40.758458759999996,"lng":-74.02626612},{"lat":40.758458759999996,"lng":-74.02806468},{"lat":40.75755948,"lng":-74.02896396},{"lat":40.759807679999994,"lng":-74.03121216},{"lat":40.76025732,"lng":-74.03076252}],"holes":[]},{"shell":[{"lat":40.754861639999994,"lng":-74.02896396},{"lat":40.754861639999994,"lng":-74.02986324},{"lat":40.75531128,"lng":-74.03031288},{"lat":40.7566602,"lng":-74.02896396},{"lat":40.75531128,"lng":-74.02761504},{"lat":40.754861639999994,"lng":-74.02806468},{"lat":40.754861639999994,"lng":-74.02896396}],"holes":[]}],"properties":{}}]}'
    data = json.loads(cached_api_response_raw)
    return {
        '15_min': get_polygon_from_api(data['results'][0]),
        '20_min': get_polygon_from_api(data['results'][1]),
        '25_min': get_polygon_from_api(data['results'][2]),
        '30_min': get_polygon_from_api(data['results'][3]),
    }
    
def get_polygon_from_api(api_result):
    polygons = [
        Polygon(
            [coordinate_object_to_point(coord) for coord in shape['shell']],
            list(map(
                lambda hole: [coordinate_object_to_point(coord) for coord in hole],
                shape['holes']
            ))
        ) for shape in api_result['shapes']
    ]
    return MultiPolygon(polygons)

def coordinate_object_to_point(coordinate_object):
    return (coordinate_object['lat'], coordinate_object['lng'])

In [4]:
from shapely.geometry import Point

commute_polygons = get_commute_polygons()

def is_within_commute(listing):
    return commute_polygons['30_min'].contains(Point(listing['latitude'], listing['longitude']))

def is_2br_or_larger(listing):
    return listing['br'] != "1"
    
listings_within_commute = list(filter(
    lambda listing: is_within_commute(listing) and is_2br_or_larger(listing),
    listings
))

In [7]:
import functools
import re

SCORES = {"Wet Bar": 0, "Bridge Views": 0, "Air Conditioning": 10, "Attended Elevator": 0, "Attended Lobby": 1.5, "Balcony": 3, "Bay Window": 0, "Beamed Ceilings": 1, "Central Air Conditioning": 10, "Central Park Views": 3, "City Views": 5, "Common Courtyard": 3, "Common Garden": 3, "Concierge": 6, "Corner": 5, "Decorative Fireplace": 0, "Dining Alcove": 3, "Dining Area": 8, "Dishwasher": 10, "Doorman": 0, "Dressing Area": 1, "Duplex": -1, "E/W Exposures": 0, "Eastern Exposure": 3, "Eat-In Kitchen": 3, "Elevator": 8, "Entry Foyer": 0, "Exposed Brick": 3, "Floor-To-Ceiling Windows": 8, "Formal Dining Room": 0, "French Doors": 2, "Full Floor": 6, "Full Service": 10, "Furnished": 4, "Gallery": 0, "Garage": 0, "Gym": 3, "Hardwood Floors": 7, "High Ceilings": 3, "Highrise": 10, "Home Office": 10, "Juliet Balcony": 0, "Keyed Elevator": 0, "Laundry Room": 0, "Live/Work": 0, "Luxury": 10, "Maisonette": 10, "Moldings": 1, "N/E Exposures": 3, "N/E/W Exposures": 0, "N/S Exposures": 0, "N/S/E Exposures": 4, "N/S/E/W Exposures": 0, "N/S/W Exposures": 0, "N/W Exposures": 0, "Northern Exposure": 1, "Oversized Windows": 2, "Park Views": 3, "Parlor": 2, "Part-Time Doorman": 0, "Patio": 6, "Pets (Case By Case)": 0, "Pets Allowed": 0, "Private Entrance": 3, "Private Garden": 3, "Private Roof": 3, "River Views": 3, "Roof Deck": 5, "S/E Exposures": 3, "S/E/W Exposures": 0, "S/W Exposures": 0, "Skylight": 0, "Skyline Views": -3, "Sleeping Loft": 0, "Southern Exposure": 7, "Stained Glass": 2, "Storage Loft": 3, "Sunken Living Room": 3, "Swimming Pool": 10, "Terrace": 5, "Triplex": -3, "Video Intercom": 1, "Virtual Doorman": 0, "Voice Intercom": 1, "Walk-In Closet": 6, "Washer/Dryer": 10, "Western Exposure": 0, "Windowed Kitchen": 3, "Wood Fireplace": -1,
}

MAX_SQ_FT = max(list(map(
    lambda listing: listing['sqft'] or 0,
    listings
)))

MAX_PRICE = max(list(map(
    lambda listing: listing['price'],
    listings
)))

def get_normalized_size(listing):
    return (listing['sqft'] or 0) / MAX_SQ_FT

def get_normalized_price(listing):
    return 1 - (listing['price'] / MAX_PRICE)

def get_normalized_commute_time(listing):
    lat_lng = Point(listing['latitude'], listing['longitude'])
    if commute_polygons['15_min'].contains(lat_lng):
        return 1
    elif commute_polygons['20_min'].contains(lat_lng):
        return 0.666
    elif commute_polygons['25_min'].contains(lat_lng):
        return 0.333
    else:
        return 0
    
def get_neighborhood_score(listing):
    if listing['borough'].lower() == 'manhattan':
        return 10
    else:
        return 0

def get_score_for_listing(listing):
    features = set(listing['features'] + extract_features_from_description(listing))
    feature_score = sum(list(map(
        lambda feature: SCORES[feature],
        features,
    )))
    price_score = get_normalized_price(listing) * 0.2 * feature_score
    size_score = get_normalized_size(listing) * 30
    commute_score = get_normalized_commute_time(listing) * 75
    neighborhood_score = get_neighborhood_score(listing)
    score_dict = {
        'features': feature_score,
        'price': price_score,
        'size': size_score,
        'commute': commute_score,
        'neighborhood': neighborhood_score,
    }
    return sum(score_dict.values())

def extract_features_from_description(listing):
    description_normalized = listing['description'].lower()
    feature_presence = {
        'Air Conditioning': (
            'air conditioning' in description_normalized
            or 'a/c' in description_normalized
            or 'conditioning' in description_normalized
            or 'air con' in description_normalized
        ),
        'Central Air Conditioning': (
            'central air conditioning' in description_normalized
            or 'central air' in description_normalized
            or 'centralair' in description_normalized
            or 'central A/C' in description_normalized
            or 'centralA/C' in description_normalized
            or 'self-controlled' in description_normalized
            or 'self controlled' in description_normalized
        ),
        'Elevator': 'elevator' in description_normalized,
        'Gym': 'gym' in description_normalized,
        'Highrise': listing['floors'] and listing['floors'] > 10,
        'Luxury': 'luxury' in description_normalized,
        'Washer/Dryer': 'washer' in description_normalized or 'dryer' in description_normalized,
    }
    return list(filter(lambda k: feature_presence[k], feature_presence.keys()))
    

def is_walk_up(listing):
    description_normalized = listing['description'].lower()
    floor_num = get_floor_number(listing)
    walk_up_in_desc = (
        "walk up" in description_normalized 
        or "walk-up" in description_normalized
        or "walkup" in description_normalized
        or len(re.findall(r"(\d+) flight", description_normalized)) > 0
    ) 
    no_elevator_multi_unit = (
        (not 'Elevator' in listing['features']) 
        and not (listing['floors'] <= 2 or listing['units'] <= 3)
    )
    is_brownstone = 'brownstone' in description_normalized
    return (not is_brownstone) and (walk_up_in_desc or no_elevator_multi_unit) and (floor_num > 2)

def get_floor_number(listing):
    description_normalized = listing['description'].lower()
    
    flights = re.findall(r"(\d+) flight", description_normalized)
    if len(flights) > 0:
        return int(flights[0]) + 1
    
    (apartment_num_raw, unit_indicator) = (re.findall(r"^(\d+)([a-zA-Z]?)", listing['apartmentNumber']) or [(1000, 'Z')])[0]
    apartment_num = int(apartment_num_raw)
    units_per_floor = (listing['units'] or 1) / (listing['floors'] or 1)
    if unit_indicator and len(unit_indicator) > 0:
        return apartment_num
    else:
        return int(apartment_num / units_per_floor) + 1

def get_score(listing_tuple):
    return listing_tuple[1]

scored_listings = list(map(
    lambda listing: (listing, get_score_for_listing(listing), get_floor_number(listing)),
    listings_within_commute
))

walk_ups = [listing for listing in scored_listings if is_walk_up(listing[0])]
non_walk_ups = [listing for listing in scored_listings if not is_walk_up(listing[0])]

walk_ups.sort(key=get_score, reverse=True)
non_walk_ups.sort(key=get_score, reverse=True)

ranked_listings = non_walk_ups + walk_ups

In [8]:
from IPython.core.display import display, HTML

listing_lis = "".join(list(map(
    lambda listing: '<li><a href="https://www.coopercooper.com/listing//{id}">{address} ({score}) {floor}</a></li>'.format(
        id=listing[0]['id'],
        address=listing[0]['displayAddress'],
        score=listing[1],
        floor=listing[2],
    ),
    ranked_listings
)))
display(HTML('<ul>{}</ul>'.format(listing_lis)))

In [ ]:
re.findall("(\d+) foo", "32 foo 7 foo")

In [ ]:
urllib.parse.urlencode({"foo": 1, "foo":2})